In [1]:
%matplotlib inline
import math as math
import random as rand
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

from datetime import datetime
import statistics 
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from keras.datasets import mnist
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
import cProfile
import re

def debug(detail,the_output):
    if detail < 1:
        print(datetime.now(),the_output)
    return

pd.set_option('display.max_rows', 120)
pd.set_option('display.max_columns', 999)
pd.set_option('display.width', 1000)
np.set_printoptions(edgeitems=150,linewidth=200)

In [3]:
class Config:
    NUM_X_PIXELS=28
    NUM_Y_PIXELS=28
    PIXEL_CUTOFF=50
    MAX_NOISE_ADD_TO_PIXEL=30
    NUM_SPHERES=8
    NUM_SECTORS_QUAD_1=3
    NUM_SECTORS=4*NUM_SECTORS_QUAD_1
    NUM_POWER_SUMS=1
    KNOWN_NUM_CLUSTERS=10
    TRAIN_NUM_ROWS=50_000#
    TEST_NUM_ROWS=10_000#
    NUM_NEIGHBORS=9
    USE_AREA_INVAR=False
    SIMILARITY_VECTOR_TYPE=1
    NUM_EPOCHS=5
    NUM_KERAS_TRAIN_IMAGES=60_000
    NUM_KERAS_TRAIN_LABELS=NUM_KERAS_TRAIN_IMAGES
    NUM_KERAS_TEST_IMAGES=10_000
    NUM_KERAS_TEST_LABELS=NUM_KERAS_TEST_IMAGES
    SCALE_SIMILARITY_VECTORS=True
    USE_CENTROIDS=False
    TRANSFORM_SHRINK=False
    TRANSFORM_PCT_IMAGES=50
    DO_PROFILING=False

# Get the similarity vector, the object under investigation


In [4]:
slope_bdy_list=[np.tan(2*np.pi*kth_sector/Config.NUM_SECTORS) for kth_sector \
                in range(-Config.NUM_SECTORS_QUAD_1+1,Config.NUM_SECTORS_QUAD_1+1)]
slope_bdy_np=np.array(slope_bdy_list)

#
# Increment the entry that corresponds to the sector and sphere that a
# single point occurs in. Use (slope,dist,direction) to determine that
# entry num.
#
def incr_entry_num(all_vectors_df,max_dist):
    slope=all_vectors_df['slope']
    dist=all_vectors_df['dist']
    direction=all_vectors_df['direction']

    if Config.USE_AREA_INVAR:
        if len(np.where(dist<=radius_bdy_np)):
            which_sphere_num=int(Config.NUM_SPHERES-1)
        else:
            which_sphere_num=np.min(np.where(dist<=radius_bdy_np))
    else:
        which_sphere_num=np.where(dist >= max_dist,Config.NUM_SPHERES-1,\
                              (dist*Config.NUM_SPHERES)//max_dist)
    slope_in_array_ndx=[np.min(np.where(one_slope<=slope_bdy_np),\
                        initial=(len(slope_bdy_np)-1)) \
                        for one_slope in slope]
    closest_slope_ndx=np.where(np.isnan(slope),0,slope_in_array_ndx)
        
    which_entries=(2*closest_slope_ndx+direction+\
                        Config.NUM_SECTORS*which_sphere_num).astype(int)
    return which_entries
#
#
def sum_vector_powers(vector_np,vector_invariants_np):
    total_num_points=np.sum(vector_np)
    vector_density_np=(vector_np/total_num_points)
    for k in range(1,Config.NUM_POWER_SUMS+1):
        for j in range(Config.NUM_SPHERES):
            vector_invariants_np[j+(k-1)*Config.NUM_SPHERES]=np.float_power(\
                np.sum(np.float_power(\
                vector_density_np[j*Config.NUM_SECTORS:\
                                  (j+1)*Config.NUM_SECTORS],k))\
                                                       ,float(1/k))
    return vector_invariants_np

#
# get_one_similarity_vector(one_image)
#
def get_one_similarity_vector(one_image):
    ############
    one_image=one_image.T
    if one_image.sum()==0:
        print('image is all zeros')
        return
    #
    # make the similarity_vector 
    # a vector of random numbers to compare against "real" vectors
    #
    if Config.SIMILARITY_VECTOR_TYPE==-1:
        similarity_vector=np.random.randint(0,high=256,\
                            size=(Config.NUM_SECTORS*Config.NUM_SPHERES))
        return similarity_vector
    if Config.SIMILARITY_VECTOR_TYPE==-4:
        rand_vector=np.random.randint(0,high=256,\
                            size=(Config.NUM_SECTORS*Config.NUM_SPHERES))
        similarity_vector=np.random.randint(0,high=256,\
                            size=(Config.NUM_SECTORS*Config.NUM_SPHERES))
        return np.hstack((rand_vector, similarity_vector))
    #
    # add noise to the image
    #
    if Config.SIMILARITY_VECTOR_TYPE==-2:
        one_image+=np.random.randint(0,high=MAX_NOISE_ADD_TO_PIXEL,\
                        size=(Config.NUM_X_PIXELS*Config.NUM_Y_PIXELS))

    one_image_np=np.resize(one_image, (Config.NUM_X_PIXELS,\
                                       Config.NUM_Y_PIXELS))

    # Get the centroid
    coord_arrays=np.where(one_image_np>Config.PIXEL_CUTOFF)
    x_origin=np.mean(coord_arrays[0])
    y_origin=np.mean(coord_arrays[1])

    # Get all the columns that describe each (x,y) point relative to the centroid
    all_vectors_df=pd.DataFrame((coord_arrays[0]-x_origin,\
                                 coord_arrays[1]-y_origin)).T
    all_vectors_df['dist']=np.sqrt((all_vectors_df[0]*all_vectors_df[0]+\
        all_vectors_df[1]*all_vectors_df[1]))
    all_vectors_df['slope']=all_vectors_df[1]/all_vectors_df[0]
    all_vectors_df['direction']=(all_vectors_df[0]<np.abs(all_vectors_df[0]))

    # initialize np arrays to calculate sector and sphere placement
    # of each (x,y) point
    num_sector_sphere_pixels_np=\
        np.zeros(Config.NUM_SECTORS*Config.NUM_SPHERES,dtype=int)
    vector_invariants_np=\
        np.zeros(Config.NUM_SPHERES*Config.NUM_POWER_SUMS,dtype=float)
    max_dist=all_vectors_df['dist'].max()
    radius_bdy_list=[max_dist*np.sqrt((kth_sphere+1)/Config.NUM_SPHERES) \
                     for kth_sphere in range(Config.NUM_SPHERES)]
    radius_bdy_np=np.array(radius_bdy_list)

    if max_dist == 0:
        print('max_dist is 0 !!')
        return

    which_entries=incr_entry_num(all_vectors_df,max_dist)
    for which_entry in which_entries:
        num_sector_sphere_pixels_np[which_entry]+=1 
    
    similarity_vector=sum_vector_powers(num_sector_sphere_pixels_np,\
                                       vector_invariants_np)
    
    #
    # Choose whether to return the similarity_vector, the 
    # num_sector_sphere_pixels_np, the entire image
    # or some combination of these three choices.
    #
    if Config.SIMILARITY_VECTOR_TYPE==0:
        return np.hstack((num_sector_sphere_pixels_np, similarity_vector))
    elif Config.SIMILARITY_VECTOR_TYPE==1:
        return similarity_vector
    elif Config.SIMILARITY_VECTOR_TYPE==2:
        return num_sector_sphere_pixels_np
    elif Config.SIMILARITY_VECTOR_TYPE==3:
        return np.hstack((one_image.to_numpy(), num_sector_sphere_pixels_np, \
                          similarity_vector))
    elif Config.SIMILARITY_VECTOR_TYPE==4:
        return np.hstack((one_image.to_numpy(), similarity_vector))
    elif Config.SIMILARITY_VECTOR_TYPE==5:
        return one_image.to_numpy()
    elif Config.SIMILARITY_VECTOR_TYPE==-3:
        rand_vector=np.random.randint(0,high=256,\
                    size=(Config.NUM_SECTORS*Config.NUM_SPHERES))
        return np.hstack((rand_vector, similarity_vector))
    else:
        debug(0,('ERROR BAD SIMILARITY_VECTOR_TYPE'))

#
# get_similarity_vectors(all_images_no_label)
#
def get_similarity_vectors(all_images_no_label):
    debug(0,('Start get_similarity_vectors() processing'))
    all_similarity_vectors=pd.DataFrame()
    for kth_image in range(len(all_images_no_label)):
        if (kth_image % 1000)==999:
            debug(0,('we have processed ',kth_image,' images'))

        one_image=all_images_no_label.iloc[kth_image]
        similarity_vector_np=get_one_similarity_vector(one_image)
        similarity_vector_df = pd.DataFrame(np.resize(similarity_vector_np,\
                                                      (1,len(similarity_vector_np))))
        if np.isnan(similarity_vector_np).any():
            print('similarity_vector_np is NULL START==================================================')
            print(similarity_vector_df)
            print('similarity_vector_np is NULL END==================================================')
        else:
            all_similarity_vectors=all_similarity_vectors.append(similarity_vector_df)
    debug(0,('End get_similarity_vectors() processing'))
    return all_similarity_vectors

# are the class sizes unbalanced?
Not really as the next cell shows

In [5]:
def is_unbalanced(image_label):
    print('The range (min num reps of a digit, max num reps of a digit):')
    print(pd.value_counts(image_label).min(),pd.value_counts(image_label).max())
    return

# Train to recognize MNIST and return all similarity vectors

In [6]:
def training(image_label,image_no_label):
    print('image_label,image_no_label',len(image_label),len(image_no_label))
    is_unbalanced(image_label)

    all_similarity_vectors=pd.DataFrame()
    
    if Config.DO_PROFILING:
        pr = cProfile.Profile()
        pr.enable()
    all_similarity_vectors=get_similarity_vectors(image_no_label)

    if Config.DO_PROFILING:
        pr.disable()
        pr.print_stats()

    similarity_sc=StandardScaler()
    all_similarity_vectors_sc=similarity_sc.fit_transform(all_similarity_vectors)

    if Config.USE_CENTROIDS:
        kmeans_similarity=KMeans(n_clusters=Config.KNOWN_NUM_CLUSTERS,\
                                 random_state=42)
        kmeans_similarity.fit(all_similarity_vectors_sc)
        similarity_train_labels=kmeans_similarity.labels_
    else:
        knn=KNeighborsClassifier(n_neighbors=Config.NUM_NEIGHBORS)
        X_train_sc=all_similarity_vectors_sc
        y_train=image_label

        knn.fit(X_train_sc,y_train)
        train_score=knn.score(X_train_sc,y_train)
    print('The training accuracy is: ',train_score)
        
    return knn,all_similarity_vectors

# Test recognizing MNIST and return all similarity vectors

In [7]:
def testing(image_label,image_no_label,knn):
    is_unbalanced(image_label)
    
    base_sc=StandardScaler()
    image_no_label_sc=base_sc.fit_transform(image_no_label)

    X_test=get_similarity_vectors(image_no_label)
    
    y_test=image_label
    ss_test=StandardScaler()
    X_test_sc=ss_test.fit_transform(X_test)
    
    test_score=knn.score(X_test_sc,y_test)
    print('The testing accuracy is ',test_score)
    return X_test

# Get accuracy of kNN and of similarity vector in conjunction with kNN

In [8]:
def get_kNN_accuracy():
    debug(0,'Start of KNN model')
    all_images_csv=pd.read_csv('./mnist-in-csv/mnist_trainSCOTT.csv',\
        skiprows=0,\
        nrows=Config.TRAIN_NUM_ROWS+Config.TEST_NUM_ROWS)

    if len(all_images_csv)==0:
        print('MNIST is all used up')

    train_images_csv=all_images_csv[0:Config.TRAIN_NUM_ROWS]
    test_images_csv=all_images_csv[Config.TRAIN_NUM_ROWS:\
                                   Config.TRAIN_NUM_ROWS+Config.TEST_NUM_ROWS]
    train_image_label=train_images_csv['label']
    train_image_no_label=train_images_csv[train_images_csv.columns[1:]]

    test_image_label=test_images_csv['label']
    test_image_no_label=test_images_csv[test_images_csv.columns[1:]]

    knn,train_similarity_vectors=training(train_image_label,train_image_no_label)

    test_similarity_vectors=testing(test_image_label,test_image_no_label,knn)
    debug(0,'End of KNN model')
    return

# Verbatim from Deep Learning with Python...
...by Francois Chollet, 2018 (page 120-122) including the hardcoded constants.

In [9]:
def verbatim_from_book_CNN_orig():
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32,(3,3), activation='relu',input_shape=(28,28,1)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64,(3,3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(10,activation='softmax'))

    (train_images, train_labels),(test_images,test_labels)=mnist.load_data()

    train_images=train_images[0:Config.NUM_KERAS_TRAIN_IMAGES]
    train_labels=train_labels[0:Config.NUM_KERAS_TRAIN_LABELS]

    test_images=test_images[0:Config.NUM_KERAS_TEST_IMAGES]
    test_labels=test_labels[0:Config.NUM_KERAS_TEST_LABELS]

    train_images=train_images.reshape((Config.NUM_KERAS_TRAIN_IMAGES,28,28,1))#60000
    train_images=train_images.astype('float32')/255

    test_images=test_images.reshape((Config.NUM_KERAS_TEST_IMAGES,28,28,1))#10000
    test_images=test_images.astype('float32')/255

    train_labels=to_categorical(train_labels)
    test_labels=to_categorical(test_labels)

    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',\
                  metrics=['accuracy'])

    debug(0,'Start of fit')
    model.fit(train_images, train_labels,epochs=5,batch_size=64)
    debug(0,'End of fit')

    test_loss, test_acc = model.evaluate(test_images,test_labels)
    test_acc
    return

In [10]:
def create_CNN_model(num_x_pixels,num_y_pixels):
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32,(3,3), activation='relu',\
                            input_shape=(num_x_pixels,num_y_pixels,1)))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64,(3,3), activation='relu'))
    model.add(layers.MaxPooling2D((2,2)))
    model.add(layers.Conv2D(64,(3,3), activation='relu'))

    model.add(layers.Flatten())
    model.add(layers.Dense(64,activation='relu'))
    model.add(layers.Dense(Config.KNOWN_NUM_CLUSTERS,activation='softmax'))
    return model

In [11]:
def train_eval_CNN_model(model,train_images,train_labels,\
                         test_images,test_labels,num_extra_rows):
    train_images=train_images.reshape((len(train_images),\
                                       Config.NUM_X_PIXELS+num_extra_rows,\
                                       Config.NUM_Y_PIXELS,1))#60000
    train_images=train_images.astype('float32')/255

    test_images=test_images.reshape((len(test_images),\
                                     Config.NUM_X_PIXELS+num_extra_rows,\
                                     Config.NUM_Y_PIXELS,1))#10000
    test_images=test_images.astype('float32')/255

    train_labels=to_categorical(train_labels)
    test_labels=to_categorical(test_labels)

    model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

    debug(0,'Start of fit')
    model.fit(train_images,train_labels,epochs=5,batch_size=64,verbose=1)
    debug(0,'End of fit')

    scores = model.evaluate(test_images,test_labels,verbose=0)
    return scores

In [12]:
def shrink_transform(one_image):
    if np.random.randint(0,high=100) >= Config.TRANSFORM_PCT_IMAGES:
        return one_image
#     print(one_image.shape)
    image_shrunk=np.zeros((Config.NUM_X_PIXELS,Config.NUM_Y_PIXELS))
    for x_coord in range(Config.NUM_X_PIXELS):
        for y_coord in range(Config.NUM_Y_PIXELS):
            image_shrunk[x_coord//2,y_coord//2]+=one_image[x_coord,y_coord]/4
    image_shrunk=image_shrunk.astype(int)
#     print(image_shrunk)
    return image_shrunk

In [13]:
def do_transforms(train_images,test_images):
    if Config.TRANSFORM_SHRINK:
        for one_image_ndx in range(len(train_images)):
            if (one_image_ndx % 1000)==999:
                debug(0,('we have transformed ',one_image_ndx,' training images'))

            train_images[one_image_ndx]=shrink_transform(train_images[one_image_ndx])
        for one_image_ndx in range(len(test_images)):
            if (one_image_ndx % 1000)==999:
                debug(0,('we have transformed ',one_image_ndx,' testing images'))

            test_images[one_image_ndx]=shrink_transform(test_images[one_image_ndx])
    return train_images,test_images

In [14]:
def verbatim_from_book_CNN():
    debug(0,'Start of verbatim_from_book_CNN')
    (train_images, train_labels),(test_images,test_labels)=mnist.load_data()

    train_images=train_images[0:Config.NUM_KERAS_TRAIN_IMAGES]
    train_labels=train_labels[0:Config.NUM_KERAS_TRAIN_LABELS]

    test_images=test_images[0:Config.NUM_KERAS_TEST_IMAGES]
    test_labels=test_labels[0:Config.NUM_KERAS_TEST_LABELS]

    train_images,test_images=do_transforms(train_images,test_images)

    X=np.vstack((train_images,test_images))
    Y=np.hstack((train_labels,test_labels))

    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    cvscores = []
    for train, test in kfold.split(X, Y):
        model=None
        model=create_CNN_model(Config.NUM_X_PIXELS,Config.NUM_Y_PIXELS)
        scores=train_eval_CNN_model(model,X[train],Y[train],\
                                    X[test], Y[test],0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    debug(0,'end of verbatim_from_book_CNN')
    return cvscores

# Get StratifiedKFold accuracy of similarity vector in conjunction with verbatim_from_book_CNN

In [15]:
#
#
#
def append_sim_vec(images_np,similarity_vectors):
    num_floats_to_append=len(similarity_vectors.iloc[0])
    num_zero_floats=Config.NUM_X_PIXELS-\
                np.mod(num_floats_to_append,Config.NUM_X_PIXELS)

    similarity_vectors_padded=np.hstack((similarity_vectors,
        np.zeros((len(similarity_vectors),num_zero_floats),\
                 dtype=float)))
#     print('similarity_vectors_padded',similarity_vectors_padded)
#     num_extra_rows=(similarity_vectors_padded[0].nbytes//Config.NUM_X_PIXELS)
    num_extra_rows=((num_floats_to_append+num_zero_floats)//Config.NUM_X_PIXELS)

    images_plus_sim=np.hstack((images_np,similarity_vectors_padded))

#     for one_image_ndx in range(len(images_plus_sim)):
#         print('one_image_ndx',one_image_ndx)
#         print('images_plus_sim',images_plus_sim[one_image_ndx])
#     print('num_extra_rows',num_extra_rows)
    images=images_plus_sim.reshape((len(similarity_vectors),\
                                       Config.NUM_X_PIXELS+num_extra_rows,\
                                       Config.NUM_Y_PIXELS,\
                                       1))
    images=images.astype('float32')/255
    return images,num_extra_rows

def test_CNN_with_similarity_vector():
    debug(0,('Start of CNN model with similarity_vector TYPE=',\
             Config.SIMILARITY_VECTOR_TYPE))
    (train_images, train_labels),(test_images,test_labels)=mnist.load_data()
    train_images=train_images[0:Config.NUM_KERAS_TRAIN_IMAGES]
    train_labels=train_labels[0:Config.NUM_KERAS_TRAIN_LABELS]

    test_images=test_images[0:Config.NUM_KERAS_TEST_IMAGES]
    test_labels=test_labels[0:Config.NUM_KERAS_TEST_LABELS]
    print(type(train_images),type(train_labels))
    print(train_images.shape,train_labels.shape)
    
    train_images,test_images=do_transforms(train_images,test_images)

    train_images_np=np.resize(train_images,(len(train_images),\
                            Config.NUM_X_PIXELS*Config.NUM_Y_PIXELS))
    train_labels_np=np.resize(train_labels,(len(train_labels)))

    test_images_np=np.resize(test_images,(len(test_images),\
                            Config.NUM_X_PIXELS*Config.NUM_Y_PIXELS))
    test_labels_np=np.resize(test_labels,(len(test_labels)))

    train_images_df=pd.DataFrame(train_images_np)
    train_labels_df=pd.DataFrame(train_labels_np)

    test_images_df=pd.DataFrame(test_images_np)
    test_labels_df=pd.DataFrame(test_labels_np)

    knn,train_similarity_vectors=training(train_labels_np,train_images_df)

    test_similarity_vectors=testing(test_labels_np,test_images_df,knn)
    print('type(test_similarity_vectors=',type(test_similarity_vectors))
    print(train_images_df.shape,test_labels_np.shape)

    #
    #
    #
    train_images,num_extra_rows=\
        append_sim_vec(train_images_np,train_similarity_vectors)
    test_images,_=append_sim_vec(test_images_np,test_similarity_vectors)
    print(type(train_images),type(train_labels))
    print('train_images,train_labels: ',train_images.shape,train_labels.shape)
    X=np.vstack((train_images,test_images))
    Y=np.hstack((train_labels,test_labels))

    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
    cvscores = []
    for train, test in kfold.split(X, Y):
        model=None
        model=create_CNN_model(Config.NUM_X_PIXELS+num_extra_rows,\
                               Config.NUM_Y_PIXELS)
        model.compile(optimizer='rmsprop',loss='categorical_crossentropy',\
                      metrics=['accuracy'])
    
        debug(0,'Start of fit')
        model.fit(X[train],to_categorical(Y[train]),epochs=Config.NUM_EPOCHS,\
                  batch_size=64,verbose=1)
        debug(0,'End of fit')

        scores= model.evaluate(X[test],to_categorical(Y[test]),verbose=0)
        print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        cvscores.append(scores[1] * 100)
    print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
    debug(0,'End of CNN model with similarity_vector')
    return cvscores

# Get two baselines: (1) verbatim_from_book_CNN; (2) random numbers

# Baseline (1): Get StratifiedKFold accuracy of verbatim_from_book_CNN

In [17]:
# Config.NUM_KERAS_TRAIN_IMAGES=60
# Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
# Config.NUM_KERAS_TEST_IMAGES=10
# Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

# Config.TRANSFORM_SHRINK=True
# (train_images, train_labels),(test_images,test_labels)=mnist.load_data()

# train_images=train_images[0:Config.NUM_KERAS_TRAIN_IMAGES]
# train_labels=train_labels[0:Config.NUM_KERAS_TRAIN_LABELS]

# test_images=test_images[0:Config.NUM_KERAS_TEST_IMAGES]
# test_labels=test_labels[0:Config.NUM_KERAS_TEST_LABELS]

# train_images,test_images=do_transforms(train_images,test_images)


In [18]:
Config.NUM_KERAS_TRAIN_IMAGES=12_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=2_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=verbatim_from_book_CNN()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-21 19:46:55.497173 Start of verbatim_from_book_CNN
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
2019-12-21 19:46:56.072514 Start of fit
Epoch 1/5
12595/12595 [==============================] - 5s 383us/sample - loss: 0.4740 - acc: 0.8514s - loss: 1.6308 - ETA: 0s - loss: 0.5204 - a
Epoch 2/5
12595/12595 [==============================] - 4s 342us/sample - loss: 0.1306 - acc: 0.9591
Epoch 3/5
12595/12595 [==============================] - 4s 350us/sample - loss: 0.0782 - acc: 0.9755
Epoch 4/5
12595/12595 [==============================] - 4s 350us/sample - loss: 0.0535 - acc: 0.9822
Epoch 5/5
12595/12595 [==============================] - 4s 335us/sample - loss: 0.0394 - acc: 0.9881
2019-12-21 19:47:18.817527 End of fit
acc: 97.72%
2019-12-21 19:47:19.218339 Start of fit
Epoch 1/5
12597/12597 [==============================] - 5s 365us/sample - loss: 0.4792 - acc: 0.8477
Epoch 2/5
12597/12597 [=============

12595/12595 [==============================] - 4s 352us/sample - loss: 0.1270 - acc: 0.9622
Epoch 3/5
12595/12595 [==============================] - 4s 346us/sample - loss: 0.0811 - acc: 0.9753
Epoch 4/5
12595/12595 [==============================] - 4s 356us/sample - loss: 0.0559 - acc: 0.9826s - loss: 0.0548 - acc: 0.982 - ETA: 2s - loss: 0.054 - ETA: 1s - loss: 0.0557 -  - ETA: 0s - loss: 0.0562 - acc
Epoch 5/5
12595/12595 [==============================] - 4s 355us/sample - loss: 0.0425 - acc: 0.9858s - loss: 0.0421 - 
2019-12-21 19:55:18.811666 End of fit
acc: 97.51%
2019-12-21 19:55:19.501506 Start of fit
Epoch 1/5
12597/12597 [==============================] - 5s 391us/sample - loss: 0.4865 - acc: 0.8448s - loss: 1. - ETA: 3s - - ETA: 1s - loss
Epoch 2/5
12597/12597 [==============================] - 5s 365us/sample - loss: 0.1317 - acc: 0.9594
Epoch 3/5
12597/12597 [==============================] - 5s 367us/sample - loss: 0.0766 - acc: 0.9766s - loss: 0.0727 -  - ETA: 3s - los

12595/12595 [==============================] - 5s 365us/sample - loss: 0.0844 - acc: 0.9740s - loss: 0.0878 - acc:  - ETA: 2s - - ETA: 0s - loss: 0.0872 
Epoch 4/5
12595/12595 [==============================] - 4s 355us/sample - loss: 0.0620 - acc: 0.9812s - loss 
Epoch 5/5
12595/12595 [==============================] - 5s 358us/sample - loss: 0.0441 - acc: 0.9868s - loss: 
2019-12-21 20:03:43.390446 End of fit
acc: 97.22%
2019-12-21 20:03:44.378215 Start of fit
Epoch 1/5
12597/12597 [==============================] - 5s 382us/sample - loss: 0.4895 - acc: 0.8475s - loss: 0.6833 - ETA: 1s - loss: 0.5
Epoch 2/5
12597/12597 [==============================] - 4s 354us/sample - loss: 0.1278 - acc: 0.9621s 
Epoch 3/5
12597/12597 [==============================] - 4s 347us/sample - loss: 0.0823 - acc: 0.9741s - loss: 0.0896 - acc: 0.972 - ETA: 2s - - ETA: 0s - loss: 0.0845 - acc:
Epoch 4/5
12597/12597 [==============================] - 4s 351us/sample - loss: 0.0589 - acc: 0.9821s - loss: 0.0

In [19]:
Config.NUM_KERAS_TRAIN_IMAGES=24_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=4_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=verbatim_from_book_CNN()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-21 20:07:35.545050 Start of verbatim_from_book_CNN
2019-12-21 20:07:36.279894 Start of fit
Epoch 1/5
25195/25195 [==============================] - 10s 378us/sample - loss: 0.3097 - acc: 0.9047
Epoch 2/5
25195/25195 [==============================] - 9s 352us/sample - loss: 0.0794 - acc: 0.9747s - loss: 0.0874 - acc: 0. - ETA: 3s - loss: 0.0870 - acc: 0.97 - ETA: 3s - loss: 0.0857 - ETA: 2s - loss: 0.0836 - - ETA: 1s - loss: 0.0832 -  - ETA: 1s - loss:
Epoch 3/5
25195/25195 [==============================] - 9s 352us/sample - loss: 0.0516 - acc: 0.9834s - loss: 0.0  - ETA: 1s - loss: 0.0528 - acc: 0.982 - ETA: 1s - loss: 0.0529 - acc: 0.9 - ETA: 1s - los
Epoch 4/5
25195/25195 [==============================] - 9s 354us/sample - loss: 0.0357 - acc: 0.9894s - loss: 0.031 - ETA: 6 - ETA: 4 - ETA: 2s - lo - ETA: 0s - loss: 0.0362 - acc: 0.9 - ETA: 0s - loss: 0.0360 - acc: - ETA: 0s - loss: 0.0359 - acc: 0.9 - ETA: 0s - loss: 0.0355 - acc: 0.989
Epoch 5/5
25195/25195 [==============

25201/25201 [==============================] - 9s 368us/sample - loss: 0.0355 - acc: 0.9879s - loss: 0.0361 - acc - ETA: 3s - loss: 0.03
Epoch 5/5
25201/25201 [==============================] - 9s 362us/sample - loss: 0.0289 - acc: 0.9909
2019-12-21 20:20:57.149547 End of fit
acc: 98.04%
2019-12-21 20:20:58.828712 Start of fit
Epoch 1/5
25203/25203 [==============================] - 10s 389us/sample - loss: 0.3019 - acc: 0.9046 - loss - ETA: 6s - loss: 0.5828 - acc:  - ETA: 6s - loss: 0.5451 - ET
Epoch 2/5
25203/25203 [==============================] - 9s 368us/sample - loss: 0.0760 - acc: 0.9760s - - ETA: 0s - loss: 0.0765 - ac
Epoch 3/5
25203/25203 [==============================] - 9s 369us/sample - loss: 0.0507 - acc: 0.9838
Epoch 4/5
25203/25203 [==============================] - 9s 369us/sample - loss: 0.0345 - acc: 0.9883s - loss: 0.0374 - acc - ETA: 8s - loss: 0. - ETA: 7s - loss: 0.0362 - acc: 0 - ETA: 6s - loss: 0.0353 - acc:  - ETA: 6 -  - ETA: 1s - loss: 0.0348 - acc:  - ET

25195/25195 [==============================] - 10s 383us/sample - loss: 0.0253 - acc: 0.9921ETA: 6s - loss: 0.0277 - acc: - ETA: 6s - loss:  - ETA: 5s  - ETA: 0s - loss: 0.0254 - a
2019-12-21 20:34:14.367899 End of fit
acc: 98.54%
2019-12-21 20:34:16.306745 Start of fit
Epoch 1/5
25196/25196 [==============================] - 10s 412us/sample - loss: 0.3005 - acc: 0.9028 - loss: 0.3823  - E
Epoch 2/5
25196/25196 [==============================] - 10s 386us/sample - loss: 0.0801 - acc: 0.9758 - loss: 0.0812 - acc - ETA: 1s - loss: 0.0
Epoch 3/5
25196/25196 [==============================] - 10s 391us/sample - loss: 0.0501 - acc: 0.9838 - loss: - ETA: 8s - loss: 0 
Epoch 4/5
25196/25196 [==============================] - 10s 382us/sample - loss: 0.0378 - acc: 0.9882 - loss: 0.0398 - acc: 0.9 - ETA: 2s - lo - ETA: 1s - loss: 0.037 - ETA: 0s - loss: 0.0377 - ac
Epoch 5/5
25196/25196 [==============================] - 10s 385us/sample - loss: 0.0272 - acc: 0.9916 - loss:  - ETA: 0s - loss: 

25203/25203 [==============================] - 11s 420us/sample - loss: 0.0357 - acc: 0.9889 - loss: 0.0367 - acc: 0.98 - ETA: 3s - loss: 0.0375 - acc: 0.98 - ETA: 3s - loss: 0 - ET - ETA: 0s - loss: 0.0355 -
Epoch 5/5
25203/25203 [==============================] - 11s 419us/sample - loss: 0.0259 - acc: 0.9920 
2019-12-21 20:50:21.725216 End of fit
acc: 98.86%
2019-12-21 20:50:23.999695 Start of fit
Epoch 1/5
25204/25204 [==============================] - 11s 440us/sample - loss: 0.3158 - acc: 0.9031 - loss: 0.36 - ETA: 1s - loss: 0.3371 - acc: 0 - ETA: 0s - loss: 0.3310 - ac - ETA: 0s - loss: 0.3196 - acc: 0.9
Epoch 2/5
25204/25204 [==============================] - 11s 418us/sample - loss: 0.0796 - acc: 0.9754 - loss: 0.0879 - acc: 0.97 - ETA: 7s - loss - ETA: 5s - loss: 0.0850 - acc: 0. - ETA: 5s - loss:
Epoch 3/5
25204/25204 [==============================] - 10s 411us/sample - loss: 0.0479 - acc: 0.9845
Epoch 4/5
25204/25204 [==============================] - 10s 413us/sample - lo

In [20]:
Config.NUM_KERAS_TRAIN_IMAGES=36_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=6_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=verbatim_from_book_CNN()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-21 20:52:23.329484 Start of verbatim_from_book_CNN
2019-12-21 20:52:24.088475 Start of fit
Epoch 1/5
37797/37797 [==============================] - 17s 445us/sample - loss: 0.2362 - acc: 0.9250 - loss
Epoch 2/5
37797/37797 [==============================] - 16s 421us/sample - loss: 0.0602 - acc: 0.9810 - lo
Epoch 3/5
37797/37797 [==============================] - 16s 423us/sample - loss: 0.0394 - acc: 0.9871
Epoch 4/5
37797/37797 [==============================] - 16s 417us/sample - loss: 0.0294 - acc: 0.9908
Epoch 5/5
37797/37797 [==============================] - 16s 420us/sample - loss: 0.0218 - acc: 0.9932 - loss: 0.0220 - acc: 
2019-12-21 20:53:48.317455 End of fit
acc: 99.10%
2019-12-21 20:53:51.010328 Start of fit
Epoch 1/5
37797/37797 [==============================] - 17s 446us/sample - loss: 0.2309 - acc: 0.9292 - loss: 0.3444  - ETA: 7s - loss: 0. - ETA: 5s - loss:  - ETA: 1s - loss: 0.2492 - acc: 0.923 - ETA:
Epoch 2/5
37797/37797 [==============================] - 

37803/37803 [==============================] - 16s 434us/sample - loss: 0.0223 - acc: 0.9928 - loss:  - ETA: 2s - loss: 0.0219 - acc: - ETA: 2s - loss: 0.0215 - a - ETA: 1s - loss: 0 - ETA: 0s - loss: 0.0216 - acc: 0.9 - ETA: 0s - loss: 0.0217 - acc:
2019-12-21 21:21:13.748903 End of fit
acc: 98.83%
2019-12-21 21:21:16.892423 Start of fit
Epoch 1/5
37806/37806 [==============================] - 18s 465us/sample - loss: 0.2485 - acc: 0.9230 - loss: 0.3056 -  - ETA: 3s - loss: 0.29 - ETA: 2s - loss: 0.2801 - acc:
Epoch 2/5
37806/37806 [==============================] - 16s 435us/sample - loss: 0.0629 - acc: 0.9799 - loss: 0. - ETA: 1s - loss: 0
Epoch 3/5
37806/37806 [==============================] - 17s 437us/sample - loss: 0.0429 - acc: 0.9868 - loss: 0.0426  - ETA: 2s - ETA: 0s - loss: 0.0428 - - ETA: 0s - loss: 0.0429 - acc: 0.9
Epoch 4/5
37806/37806 [==============================] - 17s 437us/sample - loss: 0.0321 - acc: 0.9904TA: 0s - loss: 0.0317 -  - ETA: 0s - loss: 0.0319 - acc

37798/37798 [==============================] - 19s 509us/sample - loss: 0.2306 - acc: 0.928810s - loss: 0.3712 - acc: 0. - ETA: 10s - loss: 0.3683 - - ETA: 6s - loss: 0.3041 - - ETA: 5s - loss: 0.2919 - acc: 0.9 -  - ETA: 3s - loss: 0.2572 - ETA: 
Epoch 2/5
37798/37798 [==============================] - 18s 470us/sample - loss: 0.0570 - acc: 0.982311s - loss: 0.0627 - - ETA: 11s - loss - ETA: 10s - loss: 0.0618 - acc: 0.98 - ETA: 10s  - ETA: 9s - loss: 0.0616 - acc:  - ETA: 9s - loss: 0.0616 - acc: 0 -  - ETA: 4s - loss: 0.0590 - acc:  - ETA: 3s - loss: 0. - ETA: 2s - loss: 0.0589 - acc: 0.9 - ETA: 2s - loss: 0.0588 - acc: 0.98 - ETA: 2s - loss: 0.0588 -  - ETA: 1s - loss: 0.0582 - acc - ETA: 1s - loss: 0.0 - ETA: 0s - loss: 0.0568 - acc: 0.98
Epoch 3/5
37798/37798 [==============================] - 18s 465us/sample - loss: 0.0380 - acc: 0.9884 - loss: 0.0382 - ETA: 8s - loss: 0.0391 - acc: 0.98 - ETA: 8s - loss - ETA: 7s - loss: 0.0390 - acc: 0.988 - E - ETA: 2s - loss: 0.0382 - acc -

37800/37800 [==============================] - 18s 467us/sample - loss: 0.0286 - acc: 0.9906s - loss: 0.0294 - acc: 0. - ETA: 18s - loss: 0.0276 - acc - ETA - ETA: 14s  - ETA:  - ETA: 12s - loss: 0.0292 - acc: 0. - ETA: 12s - loss: 0. -  - ETA: 7s - loss: 0.0301 - acc - ETA: 6s - loss: 0.0298 - acc: 0 - ETA: 6s - loss: 0.0301 - acc: 0.99 - ETA: 6s - loss: 0.0303 - acc: 0.9 - ETA: 5s - loss: 0.0305 - acc: 0.990 - ETA: 5s - loss - ETA: 4s - loss: 0.0298 - acc: 0. - ETA: 4s - ETA: 2s - loss: 0.0289 - acc: 0. - ETA: 2s - loss: 0.0292 -  - ETA: 1s - loss: 0.0291 - ac - ETA: 1s - loss: 0.02
Epoch 5/5
37800/37800 [==============================] - 18s 468us/sample - loss: 0.0217 - acc: 0.9934 - loss: 0.0226 - ac - ETA: 8s - loss: 0.0229 - acc: 0.99 - ETA: 8s - loss:  - ETA: - ETA: - ETA: 2s - loss: 0.0224 - acc: 0. - ETA: 2s - loss: 0. - ETA: 1s - loss: 0.02 - ETA: 0s - loss: 0.0218 - a
2019-12-21 22:04:24.896378 End of fit
acc: 97.86%
2019-12-21 22:04:28.654032 Start of fit
Epoch 1/5
37801/3

In [21]:
Config.NUM_KERAS_TRAIN_IMAGES=48_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=8_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=verbatim_from_book_CNN()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-21 22:11:08.723758 Start of verbatim_from_book_CNN
2019-12-21 22:11:09.880075 Start of fit
Epoch 1/5
50395/50395 [==============================] - 25s 489us/sample - loss: 0.1943 - acc: 0.9377 - loss: 0.2703 - acc: 0.913 - ETA: 9s - loss: 0.2688 - - ETA: 8s - loss: 0.2605
Epoch 2/5
50395/50395 [==============================] - 23s 453us/sample - loss: 0.0511 - acc: 0.9841s - loss: 0.0535 - acc: 0.98 - ETA: 10s - loss: 0.0535 - acc: 0 -  - ETA: 2s - loss: 0.0510 -  - ETA: 2s - - ETA: 0s - loss: 0.0514 - acc:
Epoch 3/5
50395/50395 [==============================] - 23s 459us/sample - loss: 0.0347 - acc: 0.9888 - loss: 0.03 - ETA: 4s - loss: 0.0344 - acc: 0.98 - ETA: 4s - loss: 0.0344 - acc: 0.9 - ETA: 3s - loss: 0.0346 - acc: 0.98 - ETA: 3s - loss: 0.0346 - ETA: 2s - l - ETA: 1s - los
Epoch 4/5
50395/50395 [==============================] - 23s 461us/sample - loss: 0.0259 - acc: 0.9920 - loss: 0.0258 - a - ETA: 1s - loss: 0.0260 - acc: 0.992 - ETA: 1s - loss: 0.0259 - acc: - ET

50396/50396 [==============================] - 23s 462us/sample - loss: 0.0512 - acc: 0.9839 - loss: 0.0525 - acc: 0.98 - ETA: 5s - ETA: 3s - loss: 0.05 - ETA: 0s - loss: 0.0515 - acc: 0
Epoch 3/5
50396/50396 [==============================] - 24s 483us/sample - loss: 0.0343 - acc: 0.9890 2s - loss: 0.0346 -  - ETA: 1s - loss: 0.0343  - ETA: 0s - loss: 0.0348 - a
Epoch 4/5
50396/50396 [==============================] - 23s 461us/sample - loss: 0.0255 - acc: 0.9920 - loss: 0.0257 - acc: 0.99 - ETA: 0s - loss: 0.0257 - acc - ETA: 0s - loss: 0.0256 - acc: 0.9
Epoch 5/5
50396/50396 [==============================] - 23s 454us/sample - loss: 0.0196 - acc: 0.9940
2019-12-21 22:37:15.690927 End of fit
acc: 98.91%
2019-12-21 22:37:20.156829 Start of fit
Epoch 1/5
50398/50398 [==============================] - 25s 497us/sample - loss: 0.1957 - acc: 0.9373s - loss: 0.28 - ETA: 10s - loss: 0.2809 - acc: 0 - ETA: 8s - loss: 0.2536 - a - ETA: 7s - loss - ETA: 5s - loss:  - ETA: 4s - loss: 0 - ETA: 

50395/50395 [==============================] - 26s 516us/sample - loss: 0.1802 - acc: 0.9427s - loss: 0.2504 - acc: 0. - ETA: 10s - loss: 0.2 - ETA: 6s - loss: 0.2195  - ETA: 6s - loss: 0.2132 -  - ETA: 5s - loss: 0.2080 -  - ETA: 4s - loss: 0.2035 -  - ETA: 3s - loss: 0.1994 - acc: 0.93 - ETA: 3s -  - ETA: 1s
Epoch 2/5
50395/50395 [==============================] - 24s 475us/sample - loss: 0.0509 - acc: 0.9841s - loss: 0. - ETA: 16s - lo - ETA: 14s - lo - ETA: 13s - loss: 0.0538 - a - ETA: 12s - loss: 0.0531 - acc: - ETA: 9s - loss: 0.0527 -  - ETA: 3s - loss: 0.0517 - acc: 0.984 - ETA: 3s - loss: 0.0516 - acc:  - ETA: 3s - loss: 0.0512 - acc: 0 - ETA: 2s - loss: 0.0515 - acc: 0.983 - ETA: 2s -  - ETA: 1s - loss: 0.0508 - acc:  - ETA: 0s - loss: 0.05
Epoch 3/5
50395/50395 [==============================] - 24s 480us/sample - loss: 0.0343 - acc: 0.9891s - loss: 0.0352 - a - ETA: 11s - loss: 0.0353 - ETA: 10s - loss: 0. - ETA: 10s - - ETA: 9s - loss: 0.0349 - a - ETA: 8s - loss:  - ETA:

50402/50402 [==============================] - 25s 502us/sample - loss: 0.0363 - acc: 0.9885 - ETA: 21s - loss: 0.0384 - acc: 0. - ETA: 21s - loss: 0.03 - ETA: 13s - loss: 0.0370 - acc: 0. - ETA: 13s - loss:  - ETA: 13s - loss: 0.0367 - ETA: 12 - ETA: 11s - loss: 0.0373 - acc: 0.98 - ETA: 11s - loss: 0.03 - ETA: 11s - loss: 0.0374 - acc: 0.98 - ETA: 10s - loss:  - ETA: 10s - loss: 0.0370 - - ETA: 9s - loss: 0.0366  - ETA: 9s - loss: 0.0371 - acc: 0.988 - ETA: 9s - l - ETA: 7s - loss: 0.0363 - - ETA: 6s - loss: 0.0 - ETA: 5s - loss: 0.0367 - acc: 0 - ETA: 5s - loss: 0.0368 - acc: 0.98 - ETA: 5s - loss:  - ETA: 3s - loss: 0.0367 - acc: 0 - ETA: 3s - loss: 0.0365
Epoch 4/5
50402/50402 [==============================] - 25s 489us/sample - loss: 0.0270 - acc: 0.9914s - loss: 0.0274 - ETA: 10s - loss: 0.0279 - acc:  - ETA: 9s - loss: 0.0281 - acc: 0.991 - ETA: 9s - loss: 0.0280 - acc: - ETA: 9s - los - ETA: 7s - lo - ETA: 6s - loss: 0.0278 - - ETA: - ETA: 1s - loss: 0.0271 - acc: 0 - ETA: 0s

50398/50398 [==============================] - 26s 523us/sample - loss: 0.0252 - acc: 0.9925 ETA: 13s - loss: 0.0248 - ETA: 11s - loss: 0.0255 - acc - ETA: 11s - loss - ETA: 8s - loss: 0.0261 -  - ETA: 8s - loss: 0.0259 - acc:  - ETA: 7s - loss: 0.0257 - acc: 0.99 - ETA: 7s - l - ETA: 5s - loss: 0.0255 - - ETA: 5s - loss: 0.0256 - acc:  - ETA: 4s - loss: 0.0253 - ac - ETA: 4s - loss: 0.025 - ETA: 3s - loss: 0.0 - ETA: 1s - loss: 0.0255 - acc: 0.99 - ETA: 1s - loss: 0.025 - ETA: 0s - loss: 0.0255 - acc: 0 - ETA: 0s - loss: 0.0253 - acc
Epoch 5/5
50398/50398 [==============================] - 26s 514us/sample - loss: 0.0202 - acc: 0.9937s - loss: 0. - ETA: 9s - loss: 0.0193 - acc: 0. - ETA: 9s - loss: - ETA: 8s - loss: 0.0189 - a - ETA: 7s - loss: 0.0196 - acc: 0.993 - ETA: 7s - los - ETA: 5s - loss: 0.0198 - acc: 0.99 - ETA: 5s - loss: 0 - ETA: 4s - loss: 0.0200 - acc: 0.99 - ETA: 4s - l - ETA: 2s - loss: 0.0203 - ETA: 
2019-12-21 23:25:18.225212 End of fit
acc: 98.96%
2019-12-21 23:25:

50403/50403 [==============================] - 26s 512us/sample - loss: 0.0209 - acc: 0.9934s - loss: 0.0171 - - ETA: 24s - loss: 0. - ETA: 24s - loss: 0.0143 - ETA: 24s - loss: 0. - ETA: 20s - loss: 0.01 - ETA: 20s - loss: 0.0184 - a - ETA: 20s -  - ETA - ETA: 10s - loss: 0.0192  - ETA: 9s - loss: 0.019 - ETA: 8s - loss: 0.0187 - acc: 0.9 - ETA: 8s - loss: 0.0187 - a - ETA: 7s -  - ETA: 3s - loss: 0.0198  - ETA: 2s - loss: 0.0204 - acc: - ETA: 2s - loss: 0.0204 - acc: 0.9 - ETA: 2s - loss: 0.0204 - acc - ETA: 1s - l
2019-12-21 23:39:37.739610 End of fit
acc: 98.77%
2019-12-21 23:39:42.958140 Start of fit
Epoch 1/5
50403/50403 [==============================] - 27s 541us/sample - loss: 0.1832 - acc: 0.94201:15 - loss: 1.4139 - acc - ETA: 57s  - ETA: 41s - loss: 0.8354 - acc: 0.73 - ETA: 41s - loss: 0.8222 - acc - ETA:  - ETA: 28s - loss: 0.5212 - a - ETA: 21s  - ETA: 18s - loss: 0.3236 - acc - ETA - ETA: 11s - loss: 0. - ETA: 10s - loss: - ETA: 9s - loss: 0.2334 - acc: - ETA: 8s - loss

50403/50403 [==============================] - 29s 569us/sample - loss: 0.1980 - acc: 0.9360s -  - ETA: 21s - loss:  - ETA: 18s - lo - ETA: 15s - loss - ETA: 14s - loss:  - ETA: 13s - lo - ETA: 12s - loss: 0.2864 - a - ETA: 12s - loss: 0.2829 - - ETA: 11s - loss: 0.2786  - ETA: 7s - loss: 0.2387 - a - ETA: 6s - loss: 0.2337  - ETA: 5s - loss - ETA: 3s - loss: 0.2183 - acc: 0.92 - ETA: 3s - loss: 0.2173 - a - ETA: 2s - loss: 0.2127  - ETA: 1s - loss: 0.2080 -  - ETA: 1s - loss: 0.2
Epoch 2/5
50403/50403 [==============================] - 26s 524us/sample - loss: 0.0520 - acc: 0.9835s - loss: 0.0545 - acc: 0.98 - ETA: 11s - loss - ET - ETA: 7s - loss: 0 - ETA: 5s - loss: 0.0538  - ETA: 5s - loss: 0.0547 -  - ETA: - ETA: 2s - los - ETA: 0s - loss: 0.0522 - 
Epoch 3/5
50403/50403 [==============================] - 26s 523us/sample - loss: 0.0355 - acc: 0.9889s - loss: 0.0249 - acc: 0.99 - ETA: 25s - loss - ETA: 24s - loss: 0.0251 - ETA: 24s -  - E - ETA: 8s - - ETA:  - ETA: 4s - loss: 0 - 

In [22]:
Config.NUM_KERAS_TRAIN_IMAGES=60_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=10_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=verbatim_from_book_CNN()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-22 00:06:43.136336 Start of verbatim_from_book_CNN
2019-12-22 00:06:44.778915 Start of fit
Epoch 1/5
62995/62995 [==============================] - 36s 572us/sample - loss: 0.1674 - acc: 0.9463 - loss: 0 - ETA: 4s - loss:  - ETA: 3s - loss: 0.1794 - acc:  - ETA: 3s - loss: 0.1775 - acc: 0.9 - ETA: 2s - loss: 0.1766 - ac - ETA: 2s - loss: 0.1741  - ETA: 1s - loss: 0.1
Epoch 2/5
62995/62995 [==============================] - 32s 514us/sample - loss: 0.0441 - acc: 0.9866s - loss: 0.0465 - a - ETA: 9s - l - ETA: 8s - loss: 0.045 - ETA: 7s - loss: 0.0457 - acc: 0.9 - ETA: 6s - loss: 0.0455 - acc: 0.986 - ETA: 6s - loss: 0 - ETA: 5s - loss: 0.0455 -  - ETA: 2s - loss: - ETA: 0s - loss: 0.0443 - acc: - ETA: 0s - loss: 0.0443 - acc
Epoch 3/5
62995/62995 [==============================] - 33s 528us/sample - loss: 0.0311 - acc: 0.9904 - loss: 0.0315 - acc: 0. - ETA: 5s - lo - ETA: 3s - loss: 0.0303 - acc: 0 - ETA: 3s - loss: 0.0302 - acc: 0.990 - ETA: 3s - loss: 0.0303 - acc: - ETA: 2s -

62996/62996 [==============================] - 34s 544us/sample - loss: 0.0180 - acc: 0.9945
2019-12-22 00:43:28.242864 End of fit
acc: 98.96%
2019-12-22 00:43:34.619153 Start of fit
Epoch 1/5
62996/62996 [==============================] - 35s 559us/sample - loss: 0.1670 - acc: 0.9489 - loss: 0.1742 - acc: 0 - ETA: 1s - loss: 0.1730 - acc: 0. - ETA: 1s - loss: 0 - ETA: 0s - loss: 0.1680 - acc: 0
Epoch 2/5
62996/62996 [==============================] - 33s 520us/sample - loss: 0.0433 - acc: 0.9864 - - ETA: - ETA: 0s - loss: 0.0435 - ac
Epoch 3/5
62996/62996 [==============================] - 32s 514us/sample - loss: 0.0301 - acc: 0.9908s  - - ETA:  - ETA: 7s - loss: 0.0303 - acc - ETA: 7s - loss: 0.0303 - acc: 0 - ETA: 6s - loss: 0.0301 - acc: 0.99 - ETA: 6s - loss: - ETA: 5s - loss: 0.0301 - a - ETA: 4s - loss: 0.03 - ETA: 3s - loss: 0.0303 - a - ETA: 2s - loss: 0.0301 - ac -
Epoch 4/5
62996/62996 [==============================] - 33s 519us/sample - loss: 0.0227 - acc: 0.9930 - loss: 

63004/63004 [==============================] - 35s 563us/sample - loss: 0.0448 - acc: 0.9865 - E - ETA: 11s - loss: 0.0468 - - ETA: 11s - loss: 0.0468 - acc: 0. - ETA: 11s - lo - ETA: 10s - loss: 0.0462 - acc:  - ETA: 10s - lo - ETA: 5s - loss: 0.0465 - acc: 0.98 - ETA: 5s - loss: 0.0464  - ETA: 4s - loss: - ETA: 3s - ETA: 1s - loss: 0.0451 - acc: 0. - ETA: 0s - loss: 0.0450 - acc - ETA: 0s - loss: 0.0448 - acc: 0
Epoch 3/5
63004/63004 [==============================] - 35s 554us/sample - loss: 0.0319 - acc: 0.9904 - loss: 0.0319 - acc: 0. - ETA: 6s - loss: 0.03 - ETA: 5s - los - ETA: 4s - loss:
Epoch 4/5
63004/63004 [==============================] - 34s 541us/sample - loss: 0.0246 - acc: 0.9925
Epoch 5/5
63004/63004 [==============================] - 34s 545us/sample - loss: 0.0194 - acc: 0.9936s - loss: 0.0197 - acc: 0. - ETA: 10s - loss: 0.0196 - acc - ETA: 8s - loss: 0.0198 -  - ETA: 7s - loss: 0.0 -  - ETA: 4s - loss
2019-12-22 01:37:50.827171 End of fit
acc: 98.87%
2019-12-22 01

63003/63003 [==============================] - 38s 601us/sample - loss: 0.1711 - acc: 0.9466
Epoch 2/5
63003/63003 [==============================] - 35s 556us/sample - loss: 0.0464 - acc: 0.98550s - loss: 0.0462 - 
Epoch 3/5
63003/63003 [==============================] - 35s 560us/sample - loss: 0.0317 - acc: 0.9902
Epoch 4/5
63003/63003 [==============================] - 35s 560us/sample - loss: 0.0244 - acc: 0.9923
Epoch 5/5
63003/63003 [==============================] - 35s 555us/sample - loss: 0.0197 - acc: 0.9940 ETA: 2 - ETA: 0s - loss: 0.0197 - acc: 
2019-12-22 02:07:24.543334 End of fit
acc: 98.43%
2019-12-22 02:07:31.833315 Start of fit
Epoch 1/5
63004/63004 [==============================] - 38s 604us/sample - loss: 0.1694 - acc: 0.9469s -  - ETA: 14s - loss: 0.2262 - - ETA: 8
Epoch 2/5
63004/63004 [==============================] - 37s 587us/sample - loss: 0.0460 - acc: 0.9857
Epoch 3/5
63004/63004 [==============================] - 37s 590us/sample - loss: 0.0307 - acc: 0.

# final results of accuracy of verbatim_from_book_CNN

12_000
97.78% (+/- 0.54%)
all_cvscores: [[97.72241711616516, 98.21810126304626, 97.50534296035767, 96.86386585235596, 96.50499224662781, 96.28306031227112, 97.99714088439941, 98.35243821144104, 97.49283790588379, 98.20788502693176], [97.2953736782074, 97.71917462348938, 98.00427556037903, 98.28938245773315, 98.28816056251526, 97.71265387535095, 98.06867241859436, 98.35243821144104, 98.0659008026123, 97.6344108581543], [97.50889539718628, 96.86386585235596, 98.71703386306763, 98.21810126304626, 97.2895860671997, 97.85560965538025, 98.78397583961487, 97.20630645751953, 98.20916652679443, 96.55913710594177], [97.72241711616516, 98.00427556037903, 98.21810126304626, 98.21810126304626, 97.218257188797, 97.71265387535095, 97.99714088439941, 97.85100221633911, 97.06303477287292, 98.1361985206604], [97.2241997718811, 98.36065769195557, 97.71917462348938, 97.93300032615662, 98.21683168411255, 98.14152717590332, 98.06867241859436, 97.63610363006592, 97.77936935424805, 97.92114496231079]]

24_000
98.55% (+/- 0.25%)
all_cvscores: [[98.7522304058075, 98.68045449256897, 98.67951273918152, 98.42969179153442, 98.75088930130005, 98.6066460609436, 98.46264123916626, 98.35538268089294, 98.46208691596985, 98.46208691596985], [98.71657490730286, 98.0741798877716, 98.71520400047302, 98.25124740600586, 98.96502494812012, 98.0350136756897, 98.64140152931213, 98.17661643028259, 98.46208691596985, 98.74821305274963], [98.71657490730286, 98.46647381782532, 98.67951273918152, 98.46538305282593, 98.96502494812012, 98.46373796463013, 98.28387498855591, 98.60565066337585, 98.5336184501648, 98.1044352054596], [98.53832721710205, 98.89443516731262, 98.46538305282593, 98.3583152294159, 98.3940064907074, 98.46373796463013, 98.10511469841003, 98.74866008758545, 98.3905553817749, 98.49785566329956], [98.85917901992798, 98.60913157463074, 98.3583152294159, 98.71520400047302, 98.78658056259155, 99.32118654251099, 98.35538268089294, 98.85591864585876, 98.35479259490967, 98.5336184501648]]

36_000
98.78% (+/- 0.26%)
all_cvscores: [[99.09588098526001, 98.64382743835449, 98.57211112976074, 98.61970543861389, 99.02427196502686, 98.9047646522522, 98.7377941608429, 98.76131415367126, 99.09459352493286, 98.78397583961487], [99.04829859733582, 98.12039136886597, 99.04807209968567, 98.90528321266174, 98.76249432563782, 98.92857074737549, 98.66635203361511, 98.78513813018799, 98.83249998092651, 98.71244430541992], [98.59623908996582, 98.50106835365295, 99.16706085205078, 98.78629446029663, 98.57211112976074, 98.95238280296326, 99.11884069442749, 99.09480810165405, 98.59423637390137, 99.02241230010986], [98.97692203521729, 98.50106835365295, 98.50071668624878, 98.9766776561737, 98.71490001678467, 99.02380704879761, 98.71398210525513, 98.35636019706726, 98.92780780792236, 98.97472858428955], [99.14346933364868, 98.73899817466736, 98.90528321266174, 98.47691655158997, 98.83388876914978, 97.85714149475098, 99.071204662323, 98.4516441822052, 98.71336817741394, 98.92703890800476]]

48_000
98.92% (+/- 0.16%)
all_cvscores: [[98.53702187538147, 98.84011149406433, 98.85755181312561, 98.8752007484436, 98.91071319580078, 98.67833256721497, 98.87459874153137, 98.7671971321106, 99.0887999534607, 98.8922655582428], [98.8046407699585, 98.91149401664734, 98.96465539932251, 99.03588891029358, 99.23214316368103, 99.12484288215637, 98.76741766929626, 99.07093048095703, 98.92799854278564, 98.8207995891571], [98.9295244216919, 98.94717931747437, 98.73259663581848, 98.67880940437317, 99.19642806053162, 99.03554320335388, 99.16041493415833, 98.92799854278564, 99.05306696891785, 98.42773079872131], [99.12577867507935, 98.69735836982727, 98.96465539932251, 98.91090989112854, 98.92857074737549, 98.9105224609375, 99.03537034988403, 98.85653257369995, 98.7671971321106, 98.8922655582428], [98.87600541114807, 98.96502494812012, 99.07175898551941, 98.89305233955383, 99.01785850524902, 99.10698533058167, 99.01750683784485, 98.80293011665344, 99.16026592254639, 99.01732802391052]]

60_000
98.98% (+/- 0.20%)
all_cvscores: [[99.14346933364868, 99.10051226615906, 98.8292396068573, 98.84302020072937, 99.08571243286133, 99.07129406929016, 99.0282952785492, 99.07103180885315, 98.9708423614502, 99.07089471817017], [98.74375462532043, 98.95774126052856, 99.07196164131165, 99.05727505683899, 98.92857074737549, 99.14273619651794, 98.92826676368713, 99.05673861503601, 98.95654916763306, 98.7707257270813], [99.10064339637756, 99.11479353904724, 98.94345998764038, 99.07156229019165, 99.15714263916016, 99.04271960258484, 99.08545017242432, 99.04244542121887, 98.87078404426575, 99.15665984153748], [98.80085587501526, 99.15762543678284, 98.94345998764038, 99.24296736717224, 99.15714263916016, 98.54264855384827, 99.01400208473206, 98.42789769172668, 99.01372194290161, 99.19954538345337], [98.8865077495575, 99.11479353904724, 98.71501922607422, 98.21454286575317, 99.00000095367432, 99.22845959663391, 99.15690422058105, 98.9852786064148, 98.82790446281433, 98.99942874908447]]


# summary of accuracy of verbatim_from_book_CNN

12_000 97.78% (+/- 0.54%)

24_000 98.55% (+/- 0.25%)

36_000 98.78% (+/- 0.26%)

48_000 98.92% (+/- 0.16%)

60_000 98.98% (+/- 0.20%)


# Get baseline (1) where a percentage of the train and test is shrunk and in the upper left corner of picture

In [23]:
Config.NUM_KERAS_TRAIN_IMAGES=12_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=2_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

Config.TRANSFORM_SHRINK=True
all_cvscores=[]
for _ in range(5):
    cvscores=verbatim_from_book_CNN()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
Config.TRANSFORM_SHRINK=False
print('all_cvscores:',all_cvscores)

2019-12-22 02:48:02.400445 Start of verbatim_from_book_CNN
2019-12-22 02:48:05.054612 ('we have transformed ', 999, ' training images')
2019-12-22 02:48:06.144978 ('we have transformed ', 1999, ' training images')
2019-12-22 02:48:07.281237 ('we have transformed ', 2999, ' training images')
2019-12-22 02:48:08.415235 ('we have transformed ', 3999, ' training images')
2019-12-22 02:48:09.564909 ('we have transformed ', 4999, ' training images')
2019-12-22 02:48:10.665760 ('we have transformed ', 5999, ' training images')
2019-12-22 02:48:11.798947 ('we have transformed ', 6999, ' training images')
2019-12-22 02:48:12.948693 ('we have transformed ', 7999, ' training images')
2019-12-22 02:48:13.996600 ('we have transformed ', 8999, ' training images')
2019-12-22 02:48:15.068565 ('we have transformed ', 9999, ' training images')
2019-12-22 02:48:16.168680 ('we have transformed ', 10999, ' training images')
2019-12-22 02:48:17.304150 ('we have transformed ', 11999, ' training images')
2019

2019-12-22 02:58:07.747940 ('we have transformed ', 999, ' training images')
2019-12-22 02:58:08.843504 ('we have transformed ', 1999, ' training images')
2019-12-22 02:58:09.988251 ('we have transformed ', 2999, ' training images')
2019-12-22 02:58:11.081668 ('we have transformed ', 3999, ' training images')
2019-12-22 02:58:12.130335 ('we have transformed ', 4999, ' training images')
2019-12-22 02:58:13.292501 ('we have transformed ', 5999, ' training images')
2019-12-22 02:58:14.386431 ('we have transformed ', 6999, ' training images')
2019-12-22 02:58:15.421578 ('we have transformed ', 7999, ' training images')
2019-12-22 02:58:16.531731 ('we have transformed ', 8999, ' training images')
2019-12-22 02:58:17.616686 ('we have transformed ', 9999, ' training images')
2019-12-22 02:58:18.711161 ('we have transformed ', 10999, ' training images')
2019-12-22 02:58:19.773935 ('we have transformed ', 11999, ' training images')
2019-12-22 02:58:20.900922 ('we have transformed ', 999, ' test

2019-12-22 03:08:33.412365 ('we have transformed ', 4999, ' training images')
2019-12-22 03:08:34.547930 ('we have transformed ', 5999, ' training images')
2019-12-22 03:08:35.720234 ('we have transformed ', 6999, ' training images')
2019-12-22 03:08:36.899869 ('we have transformed ', 7999, ' training images')
2019-12-22 03:08:38.025702 ('we have transformed ', 8999, ' training images')
2019-12-22 03:08:39.234992 ('we have transformed ', 9999, ' training images')
2019-12-22 03:08:40.360491 ('we have transformed ', 10999, ' training images')
2019-12-22 03:08:41.476546 ('we have transformed ', 11999, ' training images')
2019-12-22 03:08:42.685797 ('we have transformed ', 999, ' testing images')
2019-12-22 03:08:43.889715 ('we have transformed ', 1999, ' testing images')
2019-12-22 03:08:44.097786 Start of fit
Epoch 1/5
12595/12595 [==============================] - 11s 895us/sample - loss: 0.8894 - acc: 0.7101
Epoch 2/5
12595/12595 [==============================] - 7s 586us/sample - los

2019-12-22 03:19:16.677929 ('we have transformed ', 11999, ' training images')
2019-12-22 03:19:17.881439 ('we have transformed ', 999, ' testing images')
2019-12-22 03:19:18.933083 ('we have transformed ', 1999, ' testing images')
2019-12-22 03:19:19.136164 Start of fit
Epoch 1/5
12595/12595 [==============================] - 11s 883us/sample - loss: 0.8332 - acc: 0.7327 - l
Epoch 2/5
12595/12595 [==============================] - 8s 600us/sample - loss: 0.2443 - acc: 0.9280
Epoch 3/5
12595/12595 [==============================] - 8s 597us/sample - loss: 0.1560 - acc: 0.9532
Epoch 4/5
12595/12595 [==============================] - 8s 602us/sample - loss: 0.1089 - acc: 0.9653
Epoch 5/5
12595/12595 [==============================] - 8s 597us/sample - loss: 0.0819 - acc: 0.9736
2019-12-22 03:20:15.208749 End of fit
acc: 95.09%
2019-12-22 03:20:21.782090 Start of fit
Epoch 1/5
12597/12597 [==============================] - 11s 878us/sample - loss: 0.8326 - acc: 0.7250 - loss: 1.0 - ETA: 1

2019-12-22 03:30:10.925578 ('we have transformed ', 999, ' testing images')
2019-12-22 03:30:12.037269 ('we have transformed ', 1999, ' testing images')
2019-12-22 03:30:12.337342 Start of fit
Epoch 1/5
12595/12595 [==============================] - 12s 947us/sample - loss: 0.8035 - acc: 0.7389
Epoch 2/5
12595/12595 [==============================] - 8s 604us/sample - loss: 0.2330 - acc: 0.9278
Epoch 3/5
12595/12595 [==============================] - 8s 602us/sample - loss: 0.1428 - acc: 0.9536s - loss: 0.1380 - acc: 0.9
Epoch 4/5
12595/12595 [==============================] - 8s 599us/sample - loss: 0.1055 - acc: 0.9680s - loss: 0.1034 - acc: 0 - ETA: 3s - loss: 0.1037  - 
Epoch 5/5
12595/12595 [==============================] - 8s 604us/sample - loss: 0.0809 - acc: 0.9759
2019-12-22 03:31:10.913069 End of fit
acc: 96.30%
2019-12-22 03:31:17.577473 Start of fit
Epoch 1/5
12597/12597 [==============================] - 11s 891us/sample - loss: 0.8431 - acc: 0.7264
Epoch 2/5
12597/12597 

# Baseline (2): Get StratifiedKFold accuracy of similarity_vector of rand #s

In [30]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-1

Config.NUM_KERAS_TRAIN_IMAGES=12_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=2_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-23 00:41:58.980773 ('Start of CNN model with similarity_vector TYPE=', -1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(12000, 28, 28) (12000,)
image_label,image_no_label 12000 12000
The range (min num reps of a digit, max num reps of a digit):
1048 1351
2019-12-23 00:42:02.446234 Start get_similarity_vectors() processing
2019-12-23 00:42:05.963873 ('we have processed ', 999, ' images')
2019-12-23 00:42:08.982512 ('we have processed ', 1999, ' images')
2019-12-23 00:42:11.102246 ('we have processed ', 2999, ' images')
2019-12-23 00:42:12.834394 ('we have processed ', 3999, ' images')
2019-12-23 00:42:14.752567 ('we have processed ', 4999, ' images')
2019-12-23 00:42:17.032616 ('we have processed ', 5999, ' images')
2019-12-23 00:42:19.490541 ('we have processed ', 6999, ' images')
2019-12-23 00:42:22.365750 ('we have processed ', 7999, ' images')
2019-12-23 00:42:26.202665 ('we have processed ', 8999, ' images')
2019-12-23 00:42:31.076135 ('we have processed ', 9999, ' imag

12602/12602 [==============================] - 13s 1ms/sample - loss: 0.0554 - acc: 0.98216s  - ETA: 1s
Epoch 5/5
12602/12602 [==============================] - 13s 1ms/sample - loss: 0.0424 - acc: 0.9868
2019-12-23 01:29:50.846824 End of fit
acc: 98.71%
2019-12-23 01:30:12.250735 Start of fit
Epoch 1/5
12604/12604 [==============================] - 27s 2ms/sample - loss: 0.4477 - acc: 0.8599
Epoch 2/5
12604/12604 [==============================] - 14s 1ms/sample - loss: 0.1266 - acc: 0.96187s - loss: 0.1437 - ETA: 6s - loss: - ET - ETA: 3s - loss: 0.1335 - ETA: 2s - loss: 0.1334 - a - ETA: 1s - loss
Epoch 3/5
12604/12604 [==============================] - 14s 1ms/sample - loss: 0.0803 - acc: 0.97321s - loss: 0.0815 - acc: 0.972 - ETA: 0s - loss: 0.0815 
Epoch 4/5
12604/12604 [==============================] - 14s 1ms/sample - loss: 0.0562 - acc: 0.9822
Epoch 5/5
12604/12604 [==============================] - 13s 1ms/sample - loss: 0.0423 - acc: 0.98611s - loss: 0.
2019-12-23 01:32:23.

12601/12601 [==============================] - 33s 3ms/sample - loss: 0.4484 - acc: 0.8561
Epoch 2/5
12601/12601 [==============================] - 18s 1ms/sample - loss: 0.1204 - acc: 0.96283s - loss: 0.1 - ETA: 1s - 
Epoch 3/5
12601/12601 [==============================] - 17s 1ms/sample - loss: 0.0737 - acc: 0.97791s - loss: 0.0724 - acc: - ETA: 1s - loss: 0.0749
Epoch 4/5
12601/12601 [==============================] - 16s 1ms/sample - loss: 0.0526 - acc: 0.9820
Epoch 5/5
12601/12601 [==============================] - 15s 1ms/sample - loss: 0.0388 - acc: 0.9883
2019-12-23 03:12:10.070570 End of fit
acc: 97.71%
2019-12-23 03:12:47.068233 Start of fit
Epoch 1/5
12602/12602 [==============================] - 35s 3ms/sample - loss: 0.4637 - acc: 0.8548
Epoch 2/5
12602/12602 [==============================] - 15s 1ms/sample - loss: 0.1187 - acc: 0.96473s - loss: 0.1280 - ac
Epoch 3/5
12602/12602 [==============================] - 16s 1ms/sample - loss: 0.0763 - acc: 0.9776
Epoch 4/5
1260

In [31]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-1

Config.NUM_KERAS_TRAIN_IMAGES=24_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=4_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-23 03:56:55.304023 ('Start of CNN model with similarity_vector TYPE=', -1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(24000, 28, 28) (24000,)
image_label,image_no_label 24000 24000
The range (min num reps of a digit, max num reps of a digit):
2144 2732
2019-12-23 03:56:58.977136 Start get_similarity_vectors() processing
2019-12-23 03:57:02.937899 ('we have processed ', 999, ' images')
2019-12-23 03:57:06.496418 ('we have processed ', 1999, ' images')
2019-12-23 03:57:09.320813 ('we have processed ', 2999, ' images')
2019-12-23 03:57:11.219234 ('we have processed ', 3999, ' images')
2019-12-23 03:57:13.354198 ('we have processed ', 4999, ' images')
2019-12-23 03:57:15.816494 ('we have processed ', 5999, ' images')
2019-12-23 03:57:18.440197 ('we have processed ', 6999, ' images')
2019-12-23 03:57:21.401413 ('we have processed ', 7999, ' images')
2019-12-23 03:57:25.241943 ('we have processed ', 8999, ' images')
2019-12-23 03:57:30.579722 ('we have processed ', 9999, ' imag

25203/25203 [==============================] - 28s 1ms/sample - loss: 0.0748 - acc: 0.9760
Epoch 3/5
25203/25203 [==============================] - 28s 1ms/sample - loss: 0.0461 - acc: 0.9852
Epoch 4/5
25203/25203 [==============================] - 28s 1ms/sample - loss: 0.0328 - acc: 0.98921s - loss:
Epoch 5/5
25203/25203 [==============================] - 29s 1ms/sample - loss: 0.0263 - acc: 0.99151s - los
2019-12-23 05:22:54.792923 End of fit
acc: 98.50%
2019-12-23 05:23:22.193756 Start of fit
Epoch 1/5
25204/25204 [==============================] - 44s 2ms/sample - loss: 0.2861 - acc: 0.9089
Epoch 2/5
25204/25204 [==============================] - 31s 1ms/sample - loss: 0.0706 - acc: 0.9777
Epoch 3/5
25204/25204 [==============================] - 30s 1ms/sample - loss: 0.0450 - acc: 0.9857
Epoch 4/5
25204/25204 [==============================] - 31s 1ms/sample - loss: 0.0327 - acc: 0.9892
Epoch 5/5
25204/25204 [==============================] - 30s 1ms/sample - loss: 0.0240 - acc: 

25198/25198 [==============================] - 30s 1ms/sample - loss: 0.0319 - acc: 0.9888
Epoch 5/5
25198/25198 [==============================] - 30s 1ms/sample - loss: 0.0229 - acc: 0.9925
2019-12-23 06:47:34.059101 End of fit
acc: 98.89%
2019-12-23 06:48:01.748988 Start of fit
Epoch 1/5
25201/25201 [==============================] - 47s 2ms/sample - loss: 0.2902 - acc: 0.9070
Epoch 2/5
25201/25201 [==============================] - 32s 1ms/sample - loss: 0.0768 - acc: 0.9759
Epoch 3/5
25201/25201 [==============================] - 30s 1ms/sample - loss: 0.0484 - acc: 0.9844
Epoch 4/5
25201/25201 [==============================] - 30s 1ms/sample - loss: 0.0350 - acc: 0.9887
Epoch 5/5
25201/25201 [==============================] - 30s 1ms/sample - loss: 0.0260 - acc: 0.9920
2019-12-23 06:52:03.207165 End of fit
acc: 98.79%
2019-12-23 06:52:31.699291 Start of fit
Epoch 1/5
25203/25203 [==============================] - 47s 2ms/sample - loss: 0.2729 - acc: 0.9151
Epoch 2/5
25203/25203 

In [32]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-1

Config.NUM_KERAS_TRAIN_IMAGES=36_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=6_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-23 08:00:53.355307 ('Start of CNN model with similarity_vector TYPE=', -1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(36000, 28, 28) (36000,)
image_label,image_no_label 36000 36000
The range (min num reps of a digit, max num reps of a digit):
3230 4113
2019-12-23 08:00:57.781232 Start get_similarity_vectors() processing
2019-12-23 08:01:02.241152 ('we have processed ', 999, ' images')
2019-12-23 08:01:05.948008 ('we have processed ', 1999, ' images')
2019-12-23 08:01:08.995948 ('we have processed ', 2999, ' images')
2019-12-23 08:01:10.652376 ('we have processed ', 3999, ' images')
2019-12-23 08:01:12.926978 ('we have processed ', 4999, ' images')
2019-12-23 08:01:15.396563 ('we have processed ', 5999, ' images')
2019-12-23 08:01:18.142848 ('we have processed ', 6999, ' images')
2019-12-23 08:01:20.975302 ('we have processed ', 7999, ' images')
2019-12-23 08:01:24.903124 ('we have processed ', 8999, ' images')
2019-12-23 08:01:30.114514 ('we have processed ', 9999, ' imag

37798/37798 [==============================] - 47s 1ms/sample - loss: 0.0267 - acc: 0.9910
Epoch 5/5
37798/37798 [==============================] - 48s 1ms/sample - loss: 0.0208 - acc: 0.9932
2019-12-23 09:53:53.247783 End of fit
acc: 98.74%
2019-12-23 09:54:28.607499 Start of fit
Epoch 1/5
37800/37800 [==============================] - 66s 2ms/sample - loss: 0.2136 - acc: 0.9318
Epoch 2/5
37800/37800 [==============================] - 49s 1ms/sample - loss: 0.0553 - acc: 0.9826
Epoch 3/5
37800/37800 [==============================] - 48s 1ms/sample - loss: 0.0365 - acc: 0.9889
Epoch 4/5
37800/37800 [==============================] - 47s 1ms/sample - loss: 0.0264 - acc: 0.9915
Epoch 5/5
37800/37800 [==============================] - 46s 1ms/sample - loss: 0.0203 - acc: 0.9938
2019-12-23 09:59:59.606947 End of fit
acc: 98.79%
2019-12-23 10:00:32.670467 Start of fit
Epoch 1/5
37801/37801 [==============================] - 66s 2ms/sample - loss: 0.2227 - acc: 0.92952s - lo
Epoch 2/5
37801

2019-12-23 11:48:59.562452 ('we have processed ', 2999, ' images')
2019-12-23 11:49:01.219402 ('we have processed ', 3999, ' images')
2019-12-23 11:49:03.184153 ('we have processed ', 4999, ' images')
2019-12-23 11:49:05.463854 ('we have processed ', 5999, ' images')
2019-12-23 11:49:05.479507 End get_similarity_vectors() processing
The testing accuracy is  0.09616666666666666
type(test_similarity_vectors= <class 'pandas.core.frame.DataFrame'>
(36000, 784) (6000,)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
train_images,train_labels:  (36000, 32, 28, 1) (36000,)
2019-12-23 11:50:09.233208 Start of fit
Epoch 1/5
37797/37797 [==============================] - 66s 2ms/sample - loss: 0.2106 - acc: 0.9344
Epoch 2/5
37797/37797 [==============================] - 47s 1ms/sample - loss: 0.0540 - acc: 0.9827
Epoch 3/5
37797/37797 [==============================] - 46s 1ms/sample - loss: 0.0368 - acc: 0.9880
Epoch 4/5
37797/37797 [==============================] - 47s 1ms/sample - loss: 0.02

2019-12-23 12:54:27.216756 ('we have processed ', 17999, ' images')
2019-12-23 12:54:36.019627 ('we have processed ', 18999, ' images')
2019-12-23 12:54:45.284995 ('we have processed ', 19999, ' images')
2019-12-23 12:54:54.639822 ('we have processed ', 20999, ' images')
2019-12-23 12:55:04.149090 ('we have processed ', 21999, ' images')
2019-12-23 12:55:13.965872 ('we have processed ', 22999, ' images')
2019-12-23 12:55:23.913122 ('we have processed ', 23999, ' images')
2019-12-23 12:55:34.049396 ('we have processed ', 24999, ' images')
2019-12-23 12:55:44.498944 ('we have processed ', 25999, ' images')
2019-12-23 12:55:55.316536 ('we have processed ', 26999, ' images')
2019-12-23 12:56:06.310639 ('we have processed ', 27999, ' images')
2019-12-23 12:56:17.346138 ('we have processed ', 28999, ' images')
2019-12-23 12:56:28.555956 ('we have processed ', 29999, ' images')
2019-12-23 12:56:39.944607 ('we have processed ', 30999, ' images')
2019-12-23 12:56:51.638630 ('we have processed '

In [33]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-1

Config.NUM_KERAS_TRAIN_IMAGES=48_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=8_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-23 14:09:04.361179 ('Start of CNN model with similarity_vector TYPE=', -1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(48000, 28, 28) (48000,)
image_label,image_no_label 48000 48000
The range (min num reps of a digit, max num reps of a digit):
4324 5470
2019-12-23 14:09:10.856198 Start get_similarity_vectors() processing
2019-12-23 14:09:18.289326 ('we have processed ', 999, ' images')
2019-12-23 14:09:22.976240 ('we have processed ', 1999, ' images')
2019-12-23 14:09:26.641329 ('we have processed ', 2999, ' images')
2019-12-23 14:09:28.401241 ('we have processed ', 3999, ' images')
2019-12-23 14:09:30.479591 ('we have processed ', 4999, ' images')
2019-12-23 14:09:32.939343 ('we have processed ', 5999, ' images')
2019-12-23 14:09:35.671421 ('we have processed ', 6999, ' images')
2019-12-23 14:09:38.839323 ('we have processed ', 7999, ' images')
2019-12-23 14:09:43.034941 ('we have processed ', 8999, ' images')
2019-12-23 14:09:48.750285 ('we have processed ', 9999, ' imag

50396/50396 [==============================] - 90s 2ms/sample - loss: 0.1751 - acc: 0.9445
Epoch 2/5
50396/50396 [==============================] - 67s 1ms/sample - loss: 0.0490 - acc: 0.98421s - loss: 
Epoch 3/5
50396/50396 [==============================] - 68s 1ms/sample - loss: 0.0331 - acc: 0.98964s - loss: 0.
Epoch 4/5
50396/50396 [==============================] - 69s 1ms/sample - loss: 0.0249 - acc: 0.9921
Epoch 5/5
50396/50396 [==============================] - 69s 1ms/sample - loss: 0.0199 - acc: 0.9939
2019-12-23 16:26:38.194754 End of fit
acc: 98.89%
2019-12-23 16:27:20.816849 Start of fit
Epoch 1/5
50398/50398 [==============================] - 93s 2ms/sample - loss: 0.1831 - acc: 0.94222s - l
Epoch 2/5
50398/50398 [==============================] - 65s 1ms/sample - loss: 0.0481 - acc: 0.9852
Epoch 3/5
50398/50398 [==============================] - 68s 1ms/sample - loss: 0.0331 - acc: 0.9899
Epoch 4/5
50398/50398 [==============================] - 64s 1ms/sample - loss: 0.

2019-12-23 17:43:30.323735 ('we have processed ', 34999, ' images')
2019-12-23 17:43:43.280613 ('we have processed ', 35999, ' images')
2019-12-23 17:43:56.201406 ('we have processed ', 36999, ' images')
2019-12-23 17:44:09.311438 ('we have processed ', 37999, ' images')
2019-12-23 17:44:22.590368 ('we have processed ', 38999, ' images')
2019-12-23 17:44:36.004154 ('we have processed ', 39999, ' images')
2019-12-23 17:44:49.705577 ('we have processed ', 40999, ' images')
2019-12-23 17:45:03.587409 ('we have processed ', 41999, ' images')
2019-12-23 17:45:17.695743 ('we have processed ', 42999, ' images')
2019-12-23 17:45:31.877653 ('we have processed ', 43999, ' images')
2019-12-23 17:45:46.335361 ('we have processed ', 44999, ' images')
2019-12-23 17:46:01.397502 ('we have processed ', 45999, ' images')
2019-12-23 17:46:16.456676 ('we have processed ', 46999, ' images')
2019-12-23 17:46:31.606843 ('we have processed ', 47999, ' images')
2019-12-23 17:46:31.622465 End get_similarity_ve

50401/50401 [==============================] - 91s 2ms/sample - loss: 0.1767 - acc: 0.9443
Epoch 2/5
50401/50401 [==============================] - 69s 1ms/sample - loss: 0.0470 - acc: 0.9851
Epoch 3/5
50401/50401 [==============================] - 67s 1ms/sample - loss: 0.0305 - acc: 0.9900: 0s - loss: 0.0304 - ac
Epoch 4/5
50401/50401 [==============================] - 67s 1ms/sample - loss: 0.0231 - acc: 0.9927
Epoch 5/5
50401/50401 [==============================] - 69s 1ms/sample - loss: 0.0179 - acc: 0.9941
2019-12-23 22:31:35.530694 End of fit
acc: 99.02%
2019-12-23 22:32:22.282749 Start of fit
Epoch 1/5
50402/50402 [==============================] - 91s 2ms/sample - loss: 0.1806 - acc: 0.9423
Epoch 2/5
50402/50402 [==============================] - 70s 1ms/sample - loss: 0.0498 - acc: 0.9849
Epoch 3/5
50402/50402 [==============================] - 69s 1ms/sample - loss: 0.0333 - acc: 0.9890
Epoch 4/5
50402/50402 [==============================] - 69s 1ms/sample - loss: 0.0244 -

In [34]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-1

Config.NUM_KERAS_TRAIN_IMAGES=60_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=10_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-24 00:52:06.464044 ('Start of CNN model with similarity_vector TYPE=', -1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(60000, 28, 28) (60000,)
image_label,image_no_label 60000 60000
The range (min num reps of a digit, max num reps of a digit):
5421 6742
2019-12-24 00:52:18.943251 Start get_similarity_vectors() processing
2019-12-24 00:52:28.396641 ('we have processed ', 999, ' images')
2019-12-24 00:52:32.193345 ('we have processed ', 1999, ' images')
2019-12-24 00:52:34.979632 ('we have processed ', 2999, ' images')
2019-12-24 00:52:37.060046 ('we have processed ', 3999, ' images')
2019-12-24 00:52:39.805894 ('we have processed ', 4999, ' images')
2019-12-24 00:52:42.366008 ('we have processed ', 5999, ' images')
2019-12-24 00:52:45.108425 ('we have processed ', 6999, ' images')
2019-12-24 00:52:48.353007 ('we have processed ', 7999, ' images')
2019-12-24 00:52:52.570617 ('we have processed ', 8999, ' images')
2019-12-24 00:52:58.475556 ('we have processed ', 9999, ' imag

2019-12-24 03:17:30.043379 ('we have processed ', 58999, ' images')
2019-12-24 03:17:47.825675 ('we have processed ', 59999, ' images')
2019-12-24 03:17:47.841329 End get_similarity_vectors() processing
The training accuracy is:  0.2909
The range (min num reps of a digit, max num reps of a digit):
892 1135
2019-12-24 03:31:35.783016 Start get_similarity_vectors() processing
2019-12-24 03:31:39.107864 ('we have processed ', 999, ' images')
2019-12-24 03:31:42.898433 ('we have processed ', 1999, ' images')
2019-12-24 03:31:45.148063 ('we have processed ', 2999, ' images')
2019-12-24 03:31:46.822900 ('we have processed ', 3999, ' images')
2019-12-24 03:31:48.909230 ('we have processed ', 4999, ' images')
2019-12-24 03:31:51.304454 ('we have processed ', 5999, ' images')
2019-12-24 03:31:53.968293 ('we have processed ', 6999, ' images')
2019-12-24 03:31:57.025069 ('we have processed ', 7999, ' images')
2019-12-24 03:32:01.137119 ('we have processed ', 8999, ' images')
2019-12-24 03:32:06.8

5421 6742
2019-12-24 05:29:36.491244 Start get_similarity_vectors() processing
2019-12-24 05:29:46.683653 ('we have processed ', 999, ' images')
2019-12-24 05:29:52.993975 ('we have processed ', 1999, ' images')
2019-12-24 05:29:58.404140 ('we have processed ', 2999, ' images')
2019-12-24 05:30:00.832059 ('we have processed ', 3999, ' images')
2019-12-24 05:30:04.202737 ('we have processed ', 4999, ' images')
2019-12-24 05:30:07.047635 ('we have processed ', 5999, ' images')
2019-12-24 05:30:10.264512 ('we have processed ', 6999, ' images')
2019-12-24 05:30:14.367558 ('we have processed ', 7999, ' images')
2019-12-24 05:30:19.658097 ('we have processed ', 8999, ' images')
2019-12-24 05:30:25.985137 ('we have processed ', 9999, ' images')
2019-12-24 05:30:32.672845 ('we have processed ', 10999, ' images')
2019-12-24 05:30:39.822863 ('we have processed ', 11999, ' images')
2019-12-24 05:30:46.938828 ('we have processed ', 12999, ' images')
2019-12-24 05:30:54.302970 ('we have processed '

2019-12-24 08:09:01.897169 ('we have processed ', 59999, ' images')
2019-12-24 08:09:01.912753 End get_similarity_vectors() processing
The training accuracy is:  0.28958333333333336
The range (min num reps of a digit, max num reps of a digit):
892 1135
2019-12-24 08:22:56.210176 Start get_similarity_vectors() processing
2019-12-24 08:22:59.606218 ('we have processed ', 999, ' images')
2019-12-24 08:23:03.833597 ('we have processed ', 1999, ' images')
2019-12-24 08:23:07.284605 ('we have processed ', 2999, ' images')
2019-12-24 08:23:08.972034 ('we have processed ', 3999, ' images')
2019-12-24 08:23:11.000308 ('we have processed ', 4999, ' images')
2019-12-24 08:23:13.424785 ('we have processed ', 5999, ' images')
2019-12-24 08:23:16.091890 ('we have processed ', 6999, ' images')
2019-12-24 08:23:19.183332 ('we have processed ', 7999, ' images')
2019-12-24 08:23:23.392050 ('we have processed ', 8999, ' images')
2019-12-24 08:23:29.140863 ('we have processed ', 9999, ' images')
2019-12-2

62999/62999 [==============================] - 122s 2ms/sample - loss: 0.1526 - acc: 0.9519s - l
Epoch 2/5
62999/62999 [==============================] - 92s 1ms/sample - loss: 0.0428 - acc: 0.9868
Epoch 3/5
62999/62999 [==============================] - 91s 1ms/sample - loss: 0.0298 - acc: 0.99070s - loss: 0.0298 - acc:
Epoch 4/5
62999/62999 [==============================] - 93s 1ms/sample - loss: 0.0225 - acc: 0.9935
Epoch 5/5
62999/62999 [==============================] - 92s 1ms/sample - loss: 0.0172 - acc: 0.9946
2019-12-24 12:09:11.188902 End of fit
acc: 98.96%
2019-12-24 12:10:22.826367 Start of fit
Epoch 1/5
63000/63000 [==============================] - 124s 2ms/sample - loss: 0.1520 - acc: 0.9515
Epoch 2/5
63000/63000 [==============================] - 92s 1ms/sample - loss: 0.0418 - acc: 0.98671
Epoch 3/5
63000/63000 [==============================] - 89s 1ms/sample - loss: 0.0290 - acc: 0.9910
Epoch 4/5
63000/63000 [==============================] - 91s 1ms/sample - loss: 

# Use both random numbers and similarity_vector together

In [ ]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-3

Config.NUM_KERAS_TRAIN_IMAGES=12_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=2_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-24 13:30:25.985691 ('Start of CNN model with similarity_vector TYPE=', -3)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(12000, 28, 28) (12000,)
image_label,image_no_label 12000 12000
The range (min num reps of a digit, max num reps of a digit):
1048 1351
2019-12-24 13:30:39.439491 Start get_similarity_vectors() processing
2019-12-24 13:31:10.746328 ('we have processed ', 999, ' images')
2019-12-24 13:31:24.564513 ('we have processed ', 1999, ' images')
2019-12-24 13:31:37.299122 ('we have processed ', 2999, ' images')
2019-12-24 13:31:51.127485 ('we have processed ', 3999, ' images')
2019-12-24 13:32:05.240339 ('we have processed ', 4999, ' images')
2019-12-24 13:32:19.622733 ('we have processed ', 5999, ' images')
2019-12-24 13:32:34.654153 ('we have processed ', 6999, ' images')
2019-12-24 13:32:51.274903 ('we have processed ', 7999, ' images')
2019-12-24 13:33:10.570327 ('we have processed ', 8999, ' images')
2019-12-24 13:33:30.649727 ('we have processed ', 9999, ' imag

12604/12604 [==============================] - 19s 2ms/sample - loss: 0.0763 - acc: 0.9748
Epoch 4/5
12604/12604 [==============================] - 21s 2ms/sample - loss: 0.0537 - acc: 0.98378s - loss: 0.0534 - acc: 0.98 - ETA: - ETA: 1s - loss: 0.05
Epoch 5/5
12604/12604 [==============================] - 19s 2ms/sample - loss: 0.0410 - acc: 0.9871
2019-12-24 15:19:30.986005 End of fit
acc: 98.35%
2019-12-24 15:20:34.507859 Start of fit
Epoch 1/5
12604/12604 [==============================] - 49s 4ms/sample - loss: 0.4439 - acc: 0.86077s -
Epoch 2/5
12604/12604 [==============================] - 19s 1ms/sample - loss: 0.1211 - acc: 0.9633
Epoch 3/5
12604/12604 [==============================] - 18s 1ms/sample - loss: 0.0730 - acc: 0.97762s - lo
Epoch 4/5
12604/12604 [==============================] - 18s 1ms/sample - loss: 0.0520 - acc: 0.9834
Epoch 5/5
12604/12604 [==============================] - 18s 1ms/sample - loss: 0.0361 - acc: 0.98911s - loss: 0.0373 - ac - ETA: 0s - loss: 0.

12601/12601 [==============================] - 21s 2ms/sample - loss: 0.0355 - acc: 0.9892
2019-12-24 17:19:53.455633 End of fit
acc: 98.43%
2019-12-24 17:21:34.203434 Start of fit
Epoch 1/5
12602/12602 [==============================] - 70s 6ms/sample - loss: 0.4459 - acc: 0.8619
Epoch 2/5
12602/12602 [==============================] - 22s 2ms/sample - loss: 0.1225 - acc: 0.9621
Epoch 3/5
12602/12602 [==============================] - 22s 2ms/sample - loss: 0.0718 - acc: 0.9779
Epoch 4/5
12602/12602 [==============================] - 19s 2ms/sample - loss: 0.0520 - acc: 0.9835
Epoch 5/5
12602/12602 [==============================] - 19s 2ms/sample - loss: 0.0399 - acc: 0.98753s - loss: 0.0369 - acc: 0.9 - ETA: 2s - loss: 0.0378 - ETA: 1s - loss: 0.0
2019-12-24 17:27:33.804144 End of fit
acc: 98.78%
2019-12-24 17:28:41.714242 Start of fit
Epoch 1/5
12604/12604 [==============================] - 55s 4ms/sample - loss: 0.4482 - acc: 0.86097s - l
Epoch 2/5
12604/12604 [===================

In [ ]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-3

Config.NUM_KERAS_TRAIN_IMAGES=24_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=4_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-24 19:08:57.462061 ('Start of CNN model with similarity_vector TYPE=', -3)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(24000, 28, 28) (24000,)
image_label,image_no_label 24000 24000
The range (min num reps of a digit, max num reps of a digit):
2144 2732
2019-12-24 19:09:19.125594 Start get_similarity_vectors() processing
2019-12-24 19:09:53.176356 ('we have processed ', 999, ' images')
2019-12-24 19:10:07.929063 ('we have processed ', 1999, ' images')
2019-12-24 19:10:21.662618 ('we have processed ', 2999, ' images')
2019-12-24 19:10:35.859815 ('we have processed ', 3999, ' images')
2019-12-24 19:10:50.321930 ('we have processed ', 4999, ' images')
2019-12-24 19:11:05.114049 ('we have processed ', 5999, ' images')
2019-12-24 19:11:20.959685 ('we have processed ', 6999, ' images')
2019-12-24 19:11:37.582971 ('we have processed ', 7999, ' images')
2019-12-24 19:11:56.764502 ('we have processed ', 8999, ' images')
2019-12-24 19:12:17.011549 ('we have processed ', 9999, ' imag

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-24 19:14:51.533722 ('we have processed ', 16999, ' images')
2019-12-24 19:15:15.975735 ('we have processed ', 17999, ' images')
2019-12-24 19:15:41.085845 ('we have processed ', 18999, ' images')
2019-12-24 19:16:07.156235 ('we have processed ', 19999, ' images')
2019-12-24 19:16:33.112034 ('we have processed ', 20999, ' images')
2019-12-24 19:16:59.162318 ('we have processed ', 21999, ' images')
2019-12-24 19:17:26.154787 ('we have processed ', 22999, ' images')
2019-12-24 19:17:53.729300 ('we have processed ', 23999, ' images')
2019-12-24 19:17:53.776158 End get_similarity_vectors() processing
The training accuracy is:  0.485125
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-24 19:20:20.595620 Start get_similarity_vectors() processing
2019-12-24 19:20:34.448485 ('we have processed ', 999, ' images')
2019-12-24 19:20:46.048697 ('we have processed ', 1999, ' images')
2019-12-24 19:20:57.287757 ('we have processed ', 2999, ' images')
2019-12-24 19:

2019-12-24 22:59:40.324194 ('we have processed ', 2999, ' images')
2019-12-24 22:59:54.819544 ('we have processed ', 3999, ' images')
2019-12-24 23:00:09.453979 ('we have processed ', 4999, ' images')
2019-12-24 23:00:24.260785 ('we have processed ', 5999, ' images')
2019-12-24 23:00:39.503626 ('we have processed ', 6999, ' images')
2019-12-24 23:00:56.667261 ('we have processed ', 7999, ' images')
2019-12-24 23:01:15.991839 ('we have processed ', 8999, ' images')
2019-12-24 23:01:36.250591 ('we have processed ', 9999, ' images')
2019-12-24 23:01:56.773397 ('we have processed ', 10999, ' images')
2019-12-24 23:02:17.338041 ('we have processed ', 11999, ' images')
2019-12-24 23:02:38.962591 ('we have processed ', 12999, ' images')
2019-12-24 23:03:00.921748 ('we have processed ', 13999, ' images')
2019-12-24 23:03:23.420033 ('we have processed ', 14999, ' images')
2019-12-24 23:03:46.394959 ('we have processed ', 15999, ' images')


C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


...

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-25 02:51:05.023345 ('we have processed ', 16999, ' images')
2019-12-25 02:51:28.886215 ('we have processed ', 17999, ' images')
2019-12-25 02:51:53.707637 ('we have processed ', 18999, ' images')
2019-12-25 02:52:19.533458 ('we have processed ', 19999, ' images')
2019-12-25 02:52:46.347712 ('we have processed ', 20999, ' images')
2019-12-25 02:53:13.579176 ('we have processed ', 21999, ' images')
2019-12-25 02:53:42.249199 ('we have processed ', 22999, ' images')
2019-12-25 02:54:10.253609 ('we have processed ', 23999, ' images')
2019-12-25 02:54:10.300441 End get_similarity_vectors() processing
The training accuracy is:  0.479625
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-25 02:56:38.503476 Start get_similarity_vectors() processing
2019-12-25 02:56:52.599052 ('we have processed ', 999, ' images')
2019-12-25 02:57:03.731598 ('we have processed ', 1999, ' images')
2019-12-25 02:57:14.846459 ('we have processed ', 2999, ' images')
2019-12-25 02:

2019-12-25 04:55:12.308679 ('we have processed ', 5999, ' images')
2019-12-25 04:55:27.613352 ('we have processed ', 6999, ' images')
2019-12-25 04:55:44.514025 ('we have processed ', 7999, ' images')
2019-12-25 04:56:04.248895 ('we have processed ', 8999, ' images')
2019-12-25 04:56:24.436183 ('we have processed ', 9999, ' images')
2019-12-25 04:56:45.021458 ('we have processed ', 10999, ' images')
2019-12-25 04:57:05.616293 ('we have processed ', 11999, ' images')
2019-12-25 04:57:27.669998 ('we have processed ', 12999, ' images')
2019-12-25 04:57:50.056150 ('we have processed ', 13999, ' images')
2019-12-25 04:58:12.349023 ('we have processed ', 14999, ' images')
2019-12-25 04:58:34.942017 ('we have processed ', 15999, ' images')


C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-25 04:58:58.476171 ('we have processed ', 16999, ' images')
2019-12-25 04:59:22.761135 ('we have processed ', 17999, ' images')
2019-12-25 04:59:47.691286 ('we have processed ', 18999, ' images')
2019-12-25 05:00:13.601263 ('we have processed ', 19999, ' images')
2019-12-25 05:00:39.688367 ('we have processed ', 20999, ' images')
2019-12-25 05:01:06.392811 ('we have processed ', 21999, ' images')
2019-12-25 05:01:33.981320 ('we have processed ', 22999, ' images')
2019-12-25 05:02:01.893815 ('we have processed ', 23999, ' images')
2019-12-25 05:02:01.925094 End get_similarity_vectors() processing
The training accuracy is:  0.48333333333333334
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-25 05:04:26.878349 Start get_similarity_vectors() processing
2019-12-25 05:04:40.623271 ('we have processed ', 999, ' images')
2019-12-25 05:04:52.069186 ('we have processed ', 1999, ' images')
2019-12-25 05:05:03.360194 ('we have processed ', 2999, ' images')
201

In [ ]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-3

Config.NUM_KERAS_TRAIN_IMAGES=36_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=6_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-25 07:12:26.732602 ('Start of CNN model with similarity_vector TYPE=', -3)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(36000, 28, 28) (36000,)
image_label,image_no_label 36000 36000
The range (min num reps of a digit, max num reps of a digit):
3230 4113
2019-12-25 07:12:48.842108 Start get_similarity_vectors() processing
2019-12-25 07:13:25.159745 ('we have processed ', 999, ' images')
2019-12-25 07:13:59.547317 ('we have processed ', 1999, ' images')
2019-12-25 07:14:18.037388 ('we have processed ', 2999, ' images')
2019-12-25 07:14:45.138215 ('we have processed ', 3999, ' images')
2019-12-25 07:15:06.020771 ('we have processed ', 4999, ' images')
2019-12-25 07:15:22.574077 ('we have processed ', 5999, ' images')
2019-12-25 07:15:39.351178 ('we have processed ', 6999, ' images')
2019-12-25 07:15:58.716439 ('we have processed ', 7999, ' images')
2019-12-25 07:16:25.939878 ('we have processed ', 8999, ' images')
2019-12-25 07:16:51.270978 ('we have processed ', 9999, ' imag

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-25 07:19:40.450788 ('we have processed ', 16999, ' images')
2019-12-25 07:20:09.592203 ('we have processed ', 17999, ' images')
2019-12-25 07:20:47.097288 ('we have processed ', 18999, ' images')
2019-12-25 07:21:16.218617 ('we have processed ', 19999, ' images')
2019-12-25 07:21:46.014107 ('we have processed ', 20999, ' images')
2019-12-25 07:22:14.343774 ('we have processed ', 21999, ' images')
2019-12-25 07:22:42.728645 ('we have processed ', 22999, ' images')
2019-12-25 07:23:11.139149 ('we have processed ', 23999, ' images')
2019-12-25 07:23:40.150849 ('we have processed ', 24999, ' images')
2019-12-25 07:24:10.711063 ('we have processed ', 25999, ' images')
2019-12-25 07:24:43.857344 ('we have processed ', 26999, ' images')
2019-12-25 07:25:16.208488 ('we have processed ', 27999, ' images')
2019-12-25 07:25:49.348582 ('we have processed ', 28999, ' images')
2019-12-25 07:26:22.221252 ('we have processed ', 29999, ' images')
2019-12-25 07:26:55.538023 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-25 13:15:57.415045 ('we have processed ', 16999, ' images')
2019-12-25 13:16:21.933026 ('we have processed ', 17999, ' images')
2019-12-25 13:16:46.726563 ('we have processed ', 18999, ' images')
2019-12-25 13:17:12.728568 ('we have processed ', 19999, ' images')
2019-12-25 13:17:39.061546 ('we have processed ', 20999, ' images')
2019-12-25 13:18:05.435274 ('we have processed ', 21999, ' images')
2019-12-25 13:18:32.790997 ('we have processed ', 22999, ' images')
2019-12-25 13:19:00.621276 ('we have processed ', 23999, ' images')
2019-12-25 13:19:28.574926 ('we have processed ', 24999, ' images')
2019-12-25 13:19:58.435085 ('we have processed ', 25999, ' images')
2019-12-25 13:20:28.369893 ('we have processed ', 26999, ' images')
2019-12-25 13:20:59.102984 ('we have processed ', 27999, ' images')
2019-12-25 13:21:31.486487 ('we have processed ', 28999, ' images')
2019-12-25 13:22:03.950481 ('we have processed ', 29999, ' images')
2019-12-25 13:22:36.682459 ('we have processed '

...

2019-12-25 16:09:29.153434 ('we have processed ', 16999, ' images')
2019-12-25 16:09:54.195736 ('we have processed ', 17999, ' images')
2019-12-25 16:10:19.548058 ('we have processed ', 18999, ' images')
2019-12-25 16:10:45.863831 ('we have processed ', 19999, ' images')
2019-12-25 16:11:12.610935 ('we have processed ', 20999, ' images')
2019-12-25 16:11:39.534967 ('we have processed ', 21999, ' images')
2019-12-25 16:12:07.366323 ('we have processed ', 22999, ' images')
2019-12-25 16:12:35.446860 ('we have processed ', 23999, ' images')
2019-12-25 16:13:03.983315 ('we have processed ', 24999, ' images')
2019-12-25 16:13:34.017676 ('we have processed ', 25999, ' images')
2019-12-25 16:14:04.776491 ('we have processed ', 26999, ' images')
2019-12-25 16:14:35.835964 ('we have processed ', 27999, ' images')
2019-12-25 16:15:08.175691 ('we have processed ', 28999, ' images')
2019-12-25 16:15:41.283984 ('we have processed ', 29999, ' images')
2019-12-25 16:16:14.521618 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-25 19:04:05.865217 ('we have processed ', 16999, ' images')
2019-12-25 19:04:30.997670 ('we have processed ', 17999, ' images')
2019-12-25 19:04:57.133482 ('we have processed ', 18999, ' images')
2019-12-25 19:05:24.152310 ('we have processed ', 19999, ' images')
2019-12-25 19:05:51.443634 ('we have processed ', 20999, ' images')
2019-12-25 19:06:19.186817 ('we have processed ', 21999, ' images')
2019-12-25 19:06:47.826979 ('we have processed ', 22999, ' images')
2019-12-25 19:07:16.019517 ('we have processed ', 23999, ' images')
2019-12-25 19:07:45.015085 ('we have processed ', 24999, ' images')
2019-12-25 19:08:15.120148 ('we have processed ', 25999, ' images')
2019-12-25 19:08:46.023677 ('we have processed ', 26999, ' images')
2019-12-25 19:09:18.813926 ('we have processed ', 27999, ' images')
2019-12-25 19:09:51.803443 ('we have processed ', 28999, ' images')
2019-12-25 19:10:25.137702 ('we have processed ', 29999, ' images')
2019-12-25 19:10:58.897519 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-25 22:14:06.247857 ('we have processed ', 16999, ' images')
2019-12-25 22:14:30.755874 ('we have processed ', 17999, ' images')
2019-12-25 22:14:55.622544 ('we have processed ', 18999, ' images')
2019-12-25 22:15:21.583859 ('we have processed ', 19999, ' images')
2019-12-25 22:15:48.045640 ('we have processed ', 20999, ' images')
2019-12-25 22:16:14.700626 ('we have processed ', 21999, ' images')
2019-12-25 22:16:42.678221 ('we have processed ', 22999, ' images')
2019-12-25 22:17:10.759889 ('we have processed ', 23999, ' images')
2019-12-25 22:17:39.897097 ('we have processed ', 24999, ' images')
2019-12-25 22:18:10.640996 ('we have processed ', 25999, ' images')
2019-12-25 22:18:41.561427 ('we have processed ', 26999, ' images')
2019-12-25 22:19:13.070587 ('we have processed ', 27999, ' images')
2019-12-25 22:19:45.935696 ('we have processed ', 28999, ' images')
2019-12-25 22:20:20.966794 ('we have processed ', 29999, ' images')
2019-12-25 22:21:11.094183 ('we have processed '

In [16]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-3

Config.NUM_KERAS_TRAIN_IMAGES=48_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=8_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-26 01:56:08.136016 ('Start of CNN model with similarity_vector TYPE=', -3)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(48000, 28, 28) (48000,)
image_label,image_no_label 48000 48000
The range (min num reps of a digit, max num reps of a digit):
4324 5470
2019-12-26 01:56:08.759866 Start get_similarity_vectors() processing
2019-12-26 01:56:19.166765 ('we have processed ', 999, ' images')
2019-12-26 01:56:29.795589 ('we have processed ', 1999, ' images')
2019-12-26 01:56:41.042904 ('we have processed ', 2999, ' images')
2019-12-26 01:56:52.460642 ('we have processed ', 3999, ' images')
2019-12-26 01:57:04.578426 ('we have processed ', 4999, ' images')
2019-12-26 01:57:17.293813 ('we have processed ', 5999, ' images')
2019-12-26 01:57:30.328726 ('we have processed ', 6999, ' images')
2019-12-26 01:57:43.477264 ('we have processed ', 7999, ' images')
2019-12-26 01:57:57.622679 ('we have processed ', 8999, ' images')
2019-12-26 01:58:11.865350 ('we have processed ', 9999, ' imag

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-26 02:00:05.641819 ('we have processed ', 16999, ' images')
2019-12-26 02:00:23.602554 ('we have processed ', 17999, ' images')
2019-12-26 02:00:42.018030 ('we have processed ', 18999, ' images')
2019-12-26 02:01:01.500581 ('we have processed ', 19999, ' images')
2019-12-26 02:01:20.962299 ('we have processed ', 20999, ' images')
2019-12-26 02:01:41.073920 ('we have processed ', 21999, ' images')
2019-12-26 02:02:01.666865 ('we have processed ', 22999, ' images')
2019-12-26 02:02:22.558555 ('we have processed ', 23999, ' images')
2019-12-26 02:02:43.665445 ('we have processed ', 24999, ' images')
2019-12-26 02:03:05.265435 ('we have processed ', 25999, ' images')
2019-12-26 02:03:27.227318 ('we have processed ', 26999, ' images')
2019-12-26 02:03:49.515568 ('we have processed ', 27999, ' images')
2019-12-26 02:04:12.569474 ('we have processed ', 28999, ' images')
2019-12-26 02:04:35.486262 ('we have processed ', 29999, ' images')
2019-12-26 02:04:58.980725 ('we have processed '

50398/50398 [==============================] - 20s 400us/sample - loss: 0.1727 - acc: 0.9448s - loss: 0.4359 - acc - ETA: 15s - loss - ETA: 14s - loss: 0.3838 - acc: 0.87 - ETA: 14s - loss: 0.3807 - acc: 0.87 - ETA: 14s - loss: 0.37 - ETA: 11s - loss: 0.2889 - acc - ETA: 9s - loss: 0.2600  - ETA: 8s - loss: 0.2465 - ETA: 0s - loss: 0.1764 - a
Epoch 2/5
50398/50398 [==============================] - 20s 392us/sample - loss: 0.0485 - acc: 0.985018s - loss - E - ETA: 14s - loss: 0. - ETA: 13s - loss: 0.0545 - acc: 0. - E - ETA:  - ETA: 10s - loss: 0.0540 - acc: 0. - ETA: - ETA: 8s - loss: 0.0535 - acc:  - ETA: 7s - loss: 0.0523 - acc: 0.9 - ETA: 7s - loss: 0.0521 - ac - ETA: 6s - loss: 0.0521 - acc - ETA: 6s - loss: 0.0515 - acc: 0.98 - E - ETA: 4s - loss: 0.0499  - ETA: 3s - los - ETA: 1
Epoch 3/5
50398/50398 [==============================] - 20s 391us/sample - loss: 0.0324 - acc: 0.9896s - loss: 0.03 - ETA: 13s  - ETA: 11s - loss: 0.0310 - acc: 0. - ETA: 11s - loss: 0.0308 - a - ETA: 1

50402/50402 [==============================] - 21s 426us/sample - loss: 0.1751 - acc: 0.9451s - loss: 0.3095 - ETA: 12s - loss: 0.3002 - acc: 0. - ETA: 12s - l - ETA: 9s - loss: 0.2591 - ac - ETA: 9s - loss: 0.2506 - acc: 0.92 - ETA: 9s - loss: 0.2 - ETA: 8s - loss: 0.2360 - acc: 0.9 - ETA: 7s - loss: 0.2339 - acc: 0 - ETA: - ETA: 5s - loss: 0.2131  - ETA: 4s - loss: 0.2067 - acc: 0.935 - ETA: 4s - loss: 0.2063 - acc: 0.935 - ETA: 4s - loss: 0.2058 - acc: 0.93 - ETA: 4s - loss: 0.2047 - acc: 0.93 - ETA: 4s - loss: - ETA: 3s - loss: 0.1942 - acc: 0. - ETA: 2s - loss: 0.1926 - acc - - ETA: 0s - loss: 0.1761 - acc: 0.
Epoch 2/5
50402/50402 [==============================] - 21s 413us/sample - loss: 0.0464 - acc: 0.9854TA: 11s  - ETA: 7s - loss: - ETA: 6s - loss:  - ETA: 4s - loss: - ETA: 3s - loss: 0.  - ETA: 0s - loss: 0.0464 - acc: 0.98
Epoch 3/5
50402/50402 [==============================] - 21s 418us/sample - loss: 0.0313 - acc: 0.9903 - loss: 0.0308 -  - ETA: 4s - loss: 0.0310 - acc 

...

50403/50403 [==============================] - 21s 412us/sample - loss: 0.0196 - acc: 0.9939s - loss: 0.0186 - - ETA: 18s - loss: 0. - ETA: 14s - loss: 0.0198 - ETA: 14s - loss: 0.01 - ETA: 13s - loss: 0.0186 - a - ETA: 13s - loss: 0.01 - ETA: 8s - loss: 0.0176 - acc: 0.9 - ETA: 8s - loss: 0.0178 - - ETA: 7s - loss: 0.0177 - acc:  - 
2019-12-26 05:03:04.838473 End of fit
acc: 99.05%
2019-12-26 05:03:06.428510 Start of fit
Epoch 1/5
50403/50403 [==============================] - 21s 419us/sample - loss: 0.1755 - acc: 0.9442s - loss:  - ETA: 13s - loss: 0. - ETA: 11s - loss: 0.2832 - a - ETA: - ETA: 9s - loss: 0.2604 -  - ETA: 9s - loss: 0.2501 - acc: 0.92 - ETA: 8s - loss: 0.2482 -  - ETA: 8s - loss: 0.2405 - acc: 0. - ETA: 7s - loss: 0.2371 - - ETA: 7s - loss: 0.2296 - acc: 0.927 - ETA: 7s - loss: 0.2288 - acc: 0.927 - ETA: 7s - loss: 0.2281 - - ETA: 6s - loss: 0.2206 - ETA: 5s - - ETA: 1s - loss: 0.18 - ETA: 0s - loss: 0.1766 - acc: 0.9
Epoch 2/5
50403/50403 [=========================

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-26 05:08:53.822879 ('we have processed ', 16999, ' images')
2019-12-26 05:09:11.691239 ('we have processed ', 17999, ' images')
2019-12-26 05:09:29.992544 ('we have processed ', 18999, ' images')
2019-12-26 05:09:49.383086 ('we have processed ', 19999, ' images')
2019-12-26 05:10:08.907911 ('we have processed ', 20999, ' images')
2019-12-26 05:10:29.094744 ('we have processed ', 21999, ' images')
2019-12-26 05:10:50.167676 ('we have processed ', 22999, ' images')
2019-12-26 05:11:11.429046 ('we have processed ', 23999, ' images')
2019-12-26 05:11:33.003684 ('we have processed ', 24999, ' images')
2019-12-26 05:11:54.927822 ('we have processed ', 25999, ' images')
2019-12-26 05:12:17.205832 ('we have processed ', 26999, ' images')
2019-12-26 05:12:40.158871 ('we have processed ', 27999, ' images')
2019-12-26 05:13:03.938208 ('we have processed ', 28999, ' images')
2019-12-26 05:13:27.560815 ('we have processed ', 29999, ' images')
2019-12-26 05:13:51.456885 ('we have processed '

50398/50398 [==============================] - 21s 417us/sample - loss: 0.0340 - acc: 0.9893s - loss: 0.03 - ETA: 17s -  - ETA: 16s - loss: 0.0326 - acc:  - ETA: 16s - lo - ETA: 10s - loss: 0.0332 - a - E - ETA: 9s - loss: 0.033 - ETA: 0s - loss: 0.0342 
Epoch 4/5
50398/50398 [==============================] - 21s 412us/sample - loss: 0.0260 - acc: 0.9920s - loss: 0.0175 - acc: 0.99 - ETA: 17s - loss: 0.0174 - - ETA: 17s  - ETA: 15s - loss: 0.0226 - acc: 0.99 - ETA: 15s  - ETA: 10s - loss: 0. - ETA:  - ETA: 8s - loss:  - ETA: 5s - - ETA: 3s - loss
Epoch 5/5
50398/50398 [==============================] - 21s 416us/sample - loss: 0.0196 - acc: 0.9934s - loss: 0.0155 - acc: 0. - ETA: 16s - loss: 0.0162 - acc: 0.99 - ETA: 16s  - ETA: 10s - loss: 0.01 - ETA: 9s - loss - ETA: 3s - loss: 0.0192
2019-12-26 05:40:20.712271 End of fit
acc: 99.14%
2019-12-26 05:40:22.396917 Start of fit
Epoch 1/5
50399/50399 [==============================] - 21s 426us/sample - loss: 0.1752 - acc: 0.9434 - ETA: 7

50403/50403 [==============================] - 22s 429us/sample - loss: 0.1840 - acc: 0.9421s - loss: 0.3225 - acc: 0. - - ETA: 11s - loss: 0.2949 - acc: 0. - ETA: 11s - loss: 0.2928 - ETA: 10s - loss: 0.2844 - ETA: 10s - loss: 0.2776 - acc: 0. - ETA: 10s - loss: 0.2764 - - ETA: 9s - loss: 0.2677 - acc - ETA: 6s - lo - ETA: 4s - loss: 0.2174 - acc - ETA: 4s - loss: 0.2129 - ETA: 1s - loss:
Epoch 2/5
50403/50403 [==============================] - 21s 426us/sample - loss: 0.0465 - acc: 0.9853s - loss: 0.0584 - acc: 0.98 - E - ETA: 7s - loss: 0.0486 - acc: 0 - ETA: 7s - loss: 0.0486 - acc: 0.984 - ETA:  - ETA: 0s - loss: 0.0467 - acc: 0. - ETA: 0s - loss: 0.0466 - acc: 0.9
Epoch 3/5
50403/50403 [==============================] - 22s 428us/sample - loss: 0.0322 - acc: 0.989719s - loss: 0.0322 - acc: 0. - ETA: 19s - loss: 0. - ETA: 18s - loss: 0.03 - E - ETA: 15s - loss: 0. - ETA: 12s - loss: 0.0302 - acc: 0.99 - ETA: 12s - loss: 0.0305 - acc:  - ETA: 12s - loss: 0.0302 - acc: 0. - ETA: 12s

In [17]:
#     elif Config.SIMILARITY_VECTOR_TYPE==-1:
#         return an array of random numbers
Config.SIMILARITY_VECTOR_TYPE=-3

Config.NUM_KERAS_TRAIN_IMAGES=60_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=10_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-26 09:05:17.377114 ('Start of CNN model with similarity_vector TYPE=', -3)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(60000, 28, 28) (60000,)
image_label,image_no_label 60000 60000
The range (min num reps of a digit, max num reps of a digit):
5421 6742
2019-12-26 09:05:17.707430 Start get_similarity_vectors() processing
2019-12-26 09:05:27.909872 ('we have processed ', 999, ' images')
2019-12-26 09:05:38.658703 ('we have processed ', 1999, ' images')
2019-12-26 09:05:49.858991 ('we have processed ', 2999, ' images')
2019-12-26 09:06:01.310327 ('we have processed ', 3999, ' images')
2019-12-26 09:06:13.562871 ('we have processed ', 4999, ' images')
2019-12-26 09:06:26.380033 ('we have processed ', 5999, ' images')
2019-12-26 09:06:39.604969 ('we have processed ', 6999, ' images')
2019-12-26 09:06:53.157433 ('we have processed ', 7999, ' images')
2019-12-26 09:07:07.543560 ('we have processed ', 8999, ' images')
2019-12-26 09:07:22.193259 ('we have processed ', 9999, ' imag

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-26 09:09:18.228799 ('we have processed ', 16999, ' images')
2019-12-26 09:09:36.278804 ('we have processed ', 17999, ' images')
2019-12-26 09:09:54.776048 ('we have processed ', 18999, ' images')
2019-12-26 09:10:14.267338 ('we have processed ', 19999, ' images')
2019-12-26 09:10:34.024367 ('we have processed ', 20999, ' images')
2019-12-26 09:10:54.459602 ('we have processed ', 21999, ' images')
2019-12-26 09:11:15.506341 ('we have processed ', 22999, ' images')
2019-12-26 09:11:36.909143 ('we have processed ', 23999, ' images')
2019-12-26 09:11:58.662744 ('we have processed ', 24999, ' images')
2019-12-26 09:12:20.889648 ('we have processed ', 25999, ' images')
2019-12-26 09:12:43.340164 ('we have processed ', 26999, ' images')
2019-12-26 09:13:06.490157 ('we have processed ', 27999, ' images')
2019-12-26 09:13:30.341029 ('we have processed ', 28999, ' images')
2019-12-26 09:13:54.173196 ('we have processed ', 29999, ' images')
2019-12-26 09:14:18.394012 ('we have processed '

62996/62996 [==============================] - 27s 423us/sample - loss: 0.0222 - acc: 0.9931 - los - ETA: 3s - loss: 0.0223 - acc: - ETA: 3s - loss: - ETA: 0s - loss: 0.0222 - acc: 0.993
Epoch 5/5
62996/62996 [==============================] - 27s 422us/sample - loss: 0.0173 - acc: 0.9947 - loss: 0.0174 - acc: 0 - ETA: 5s - loss -
2019-12-26 09:55:47.335810 End of fit
acc: 98.86%
2019-12-26 09:55:49.489394 Start of fit
Epoch 1/5
62999/62999 [==============================] - 29s 457us/sample - loss: 0.1600 - acc: 0.9495s - loss: 0.5939 - ETA: 24s - loss: 0.4782 - - ETA: 22s  - ETA: 16s - loss: 0.2735 - - - ETA: 14s - loss: 0.2532 - acc:  - ETA: 14s - loss: 0. - ETA - ETA: 12s - loss: 0.2352 - acc: 0.92 - ETA: 12s - loss: 0.2347 - ETA: 12s - loss: 0.23 - ETA: 11s - loss: 0.2257 - a - ETA: 11s - loss: 0.2234 - acc: 0.92 - ETA - ETA: 10s - loss: 0.2135 - a - ETA: 9s - los - ETA: 8s - loss: 0.2 - ETA: 7s - loss: 0.1 - ETA: 6s - loss: 0.1876 - acc: 0.940 - ETA: 6s - loss: 0.1873 - acc: 0.94

63004/63004 [==============================] - 27s 429us/sample - loss: 0.0284 - acc: 0.9913s - loss:  - ETA: 9s - loss: - ETA: 1s - loss: 0.0288  - ETA: 1s - loss: 0.0287 - ETA: 0s - loss: 0.0282 - acc: 0.
Epoch 4/5
63004/63004 [==============================] - 27s 425us/sample - loss: 0.0209 - acc: 0.9935
Epoch 5/5
63004/63004 [==============================] - 27s 427us/sample - loss: 0.0168 - acc: 0.9950 - loss: 0.015 - ETA: 8s - loss: 0.0155 -  - ETA: 8s - loss: 0.0154 - acc: 0. - ETA: 7s - loss: 0.0155 - acc: 0.99 - ETA: 7s - loss: - ETA: 6s - loss: 0.0160 - - ETA: 3s - loss: 0.0163 - acc:  - ETA - ETA: 0s - loss: 0.0
2019-12-26 10:10:00.281209 End of fit
acc: 98.99%
2019-12-26 10:10:02.526270 Start of fit
Epoch 1/5
63004/63004 [==============================] - 28s 437us/sample - loss: 0.1604 - acc: 0.9505s - loss: 0.3492 - a - ETA: 19s - loss - ETA: 16s - loss: 0.2769 - ETA: 15s - loss: 0.2703 - acc:  - ETA: 14 - ETA: 12s -  - ETA: 11s - loss: 0.2243 - -  -  - ETA: 7s - loss: 

...

62996/62996 [==============================] - 30s 473us/sample - loss: 0.0408 - acc: 0.9868s - loss: 0.04 - - ETA: 25s - loss: 0.04 - ETA: 25s - loss: 0.0430 - ETA: 25s - loss:  - ETA: 23s - loss: 0.0456 - acc: 0.98 - ETA: 22 - - ETA: 14s - loss: 0.0417 - ETA: 12s - loss: 0.0414 - acc - ETA: 12s - loss: 0.04 - ETA: 11s - loss: 0.0421 - acc:  - ETA: 10s - loss: 0.0420 - ETA: 10s - loss: 0.0417 - ETA: 9s - loss: 0.0416 - acc: 0. - E - ETA: 2s - loss: 0.0407 - ac - ETA: 2s - loss:  - ETA: 0s - loss: 0.0407 
Epoch 3/5
62996/62996 [==============================] - 29s 465us/sample - loss: 0.0286 - acc: 0.9909s - loss: 0.0328 - acc: 0.99 - - - ETA: 20s - loss: 0.0270 - - ETA: 20s - loss - ETA: 19s - loss: 0.0269 - a - ETA: 19s - loss - ETA:  - ETA: 16s - loss: 0.0277 - acc: 0. - ETA: 12 - ETA: 11s - loss: 0.02 - ETA: 11s - loss: 0.0 - ETA: 9s - ETA: 7s - loss: 0.0287 - acc - ETA: 6s - loss: 0.0292 - - ETA: 6s - loss: 0.0292 - ac - ETA: 5s - loss: 0.0291 - acc: 0.9 - ETA: 5s - loss: 0.0290 

62999/62999 [==============================] - 31s 485us/sample - loss: 0.0179 - acc: 0.994628s - loss: 0.0185 - acc - ETA: 27s - loss: 0.0192 - acc: 0.99 - ETA: 27s - lo - ETA - ETA: 24s - loss: 0.0200 - - E - ETA: 23s - loss: 0.0181 - acc: 0. - ETA: 23s - loss: 0.0185 - acc: 0. - ETA: 22s - loss: 0.0183 - - E - ETA: 21s - loss: 0.0189 - acc: 0. - ETA: 21s - loss:  - ETA: 20s - loss: 0.0184 - a - ETA: 16s - loss:  - ETA: 15s - loss: 0.0184 - acc: 0. - ETA: 15s - loss: 0.0183 - acc: 0. - ETA: 15s - loss: 0.0186 - - ETA: 15s  - ETA: 14s - loss: 0.0182 - ETA: 13s - loss: 0.0181 - - ETA: 13s - loss: 0.0184 - a - ETA: 13s - loss: 0.0181 - - ETA: 12s - loss: 0.0179 - acc:  - ETA: 12s  - ETA: 11s - loss: 0.0178 - - ETA: 11s - loss: 0.0177 - a - ETA: 11s - loss: 0.0179 - a - ETA: 10s - loss: 0.0179 - acc:  - ETA: 10s - loss: 0.0178 - a - ETA: 10s - loss: 0 - ETA: 9s - loss: 0.0180 - acc:  - ETA: 8s - loss: 0.0181 - ETA: 7s - lo - ETA: 6s - loss: 0. - ETA: 5s - loss: 0.0179 - a - ETA: 4s - los

63001/63001 [==============================] - 31s 494us/sample - loss: 0.0217 - acc: 0.9935s - loss:  - ETA: 27s - loss: 0.0172 - a - ETA: 27s - loss: 0.0167 - acc: 0. - ETA: 27s - loss:  - ETA: 26s - loss: 0.0157 - acc: 0.99 - ETA: 26s - loss - ETA: 25s - loss: 0.0173 - acc - ETA: 25s - loss: 0.0176 - acc: 0.99 - ETA: 25 - ETA:  - ETA: 17s - loss: 0.01 - ETA: 17s - loss: 0.0198 - ETA: 16s - loss:  - ETA: 16s - loss: 0.0202 - acc: 0.99 - ETA: 16s - loss: 0.0202 - acc: 0.99 - ETA: 15s - loss: 0.0201 - acc: 0.99 - ETA: 15s - loss: 0.0201 - acc:  - ETA: 15s - loss: 0. - ETA: 15s - loss: 0.0203 - - ETA: 14s - loss: 0. - ETA: 14s - loss: 0.0207 - acc:  - ETA: 13s - loss: 0.0207 - - ETA: 13s - loss: 0.0207 - acc:  - ETA: 13s - loss: 0.0206 - - ETA: 13s  - ETA: 10s - loss: 0.0214 - ETA: 10s - loss: 0.0 - ETA: 9s - loss: 0.0212 - acc: 0.99 - ETA: 9s - loss: 0.0211 - acc: 0.99 - ETA: 9s - loss: 0.0211 - acc: 0.993 - ETA: 9s - ETA: 7s - loss: 0.0216 - acc: 0.99 - ETA: 7s - loss: 0.0216 - acc: 0

63003/63003 [==============================] - 31s 494us/sample - loss: 0.1534 - acc: 0.9520s - loss: 1.0652 - acc - ETA:  - ETA: 31s - loss: 0.5634 - acc - ETA: 31s - loss: 0.5428 - acc: 0.82 - ETA: 30s - loss: 0.5374 - acc - ETA: 30s - loss: 0.5220 - acc:  - ETA: 29s - loss: 0. - ETA: 28s - loss: 0.4695 - acc: 0.85 - ETA: 28s - loss: 0.4662 - acc: 0.85 - ETA: 28s - loss: 0.4631 - acc: 0.85 - ETA: 28s -  - ETA: 27s - loss: 0.4191 - - ETA: 26s - lo - ETA: 20s - loss: 0.2914 - acc: 0.90 - ETA: 20s - loss:  - ETA: 19s - loss: 0.2809 - acc: 0.91 - ETA: 18s - loss: 0.2606 - acc - ETA: 17s - loss: 0.2576 - acc: 0. - ETA: 17s - loss: 0.2558 - acc:  - ETA: 17s - loss: 0.2539 - acc:  - ETA: 17s - loss: 0.2516 - ETA: 16s - loss: 0.2461 - acc - ETA: 16s -  - ETA: 15s - loss: 0.23 - ETA: 13s - loss: 0.2175 - acc: 0. - ETA: 13s - loss: 0.21 - ETA: 12s - loss:  - ETA: 12s - loss: 0.2087 - - ETA: 11s - loss: 0.2054 - acc:  - ETA: 11s - loss:  - ETA: 11s - loss: 0. - ETA: 10s - loss: 0.1967 - acc: 0.

63004/63004 [==============================] - 31s 495us/sample - loss: 0.1567 - acc: 0.95121:15 - loss: 1.6582 - acc: 0. - ETA: 58s - loss: 1.3865 - ac - ETA: 50s - lo - - ETA: 31s - loss: 0.5708 - acc - ETA: 30s - loss: 0.5524 - acc: 0.82 - ETA: 30s - loss:  - ETA: 23s -  - ETA: 22s - loss: 0.3299 - acc: 0. - ETA: 22s - loss: 0.3268 - a - ETA: 21s -  - ETA:  - ETA: 18s - loss - ETA: 14s - loss: 0.2326 - a - ETA: 14s - loss: 0.2296 - a - ETA - ETA: 12s - loss: 0.2180 - - ETA: 12s - lo - ETA: 10s - loss: 0.2008 - ac - ETA: 9s - loss: 0.1988 - acc: 0. - ETA: 9s - loss: 0.1972 - acc: 0.938 - ETA: 9s - loss: 0.1967 - a - ET - ETA: 6s - l - ETA: 5s - loss: 0.1740 - acc:  - ETA: 4s - loss: 0.1723 - acc: 0.946 - ETA: 4s - loss: 0.1720 - ac - ETA: 4s - loss:  - ETA: 2s - loss: 0.1650 - a - ETA: 2s - loss: 0.1629 - acc: 0 - ETA: 1s - loss: 0.1616 - acc:  - ETA: 1s - loss: 0.16 - ETA: 0s - loss: 0.1575 - acc: 0
Epoch 2/5
63004/63004 [==============================] - 31s 488us/sample - loss: 0.

# Compare: Get StratifiedKFold accuracy of similarity_vector TYPE=1

In [18]:
#     if Config.SIMILARITY_VECTOR_TYPE==0:
#         return np.hstack((num_sector_sphere_pixels_np, similarity_vector))
#     elif Config.SIMILARITY_VECTOR_TYPE==1:
#         return similarity_vector
#     elif Config.SIMILARITY_VECTOR_TYPE==2:
#         return num_sector_sphere_pixels_np
#     elif Config.SIMILARITY_VECTOR_TYPE==3:
#         return np.hstack((one_image.to_numpy(), num_sector_sphere_pixels_np, \
#                           similarity_vector))
#     elif Config.SIMILARITY_VECTOR_TYPE==4:
#         return np.hstack((one_image.to_numpy(), similarity_vector))
#     else:# Config.SIMILARITY_VECTOR_TYPE==5:
#         return one_image.to_numpy()

# summary of accuracy of test_CNN_with_similarity_vector


In [24]:
Config.NUM_KERAS_TRAIN_IMAGES=12_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=2_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-22 03:41:15.073009 ('Start of CNN model with similarity_vector TYPE=', 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(12000, 28, 28) (12000,)
image_label,image_no_label 12000 12000
The range (min num reps of a digit, max num reps of a digit):
1048 1351
2019-12-22 03:41:17.162091 Start get_similarity_vectors() processing
2019-12-22 03:41:32.164586 ('we have processed ', 999, ' images')
2019-12-22 03:41:46.702721 ('we have processed ', 1999, ' images')
2019-12-22 03:41:59.704529 ('we have processed ', 2999, ' images')
2019-12-22 03:42:11.314892 ('we have processed ', 3999, ' images')
2019-12-22 03:42:23.461350 ('we have processed ', 4999, ' images')
2019-12-22 03:42:36.005472 ('we have processed ', 5999, ' images')
2019-12-22 03:42:48.522285 ('we have processed ', 6999, ' images')
2019-12-22 03:43:00.266451 ('we have processed ', 7999, ' images')
2019-12-22 03:43:13.142665 ('we have processed ', 8999, ' images')
2019-12-22 03:43:25.786774 ('we have processed ', 9999, ' image

12604/12604 [==============================] - 8s 645us/sample - loss: 0.1305 - acc: 0.9601
Epoch 3/5
12604/12604 [==============================] - 8s 647us/sample - loss: 0.0813 - acc: 0.9749s - loss: 0.0840 - ac - ETA: 0s - loss: 0.0826 -
Epoch 4/5
12604/12604 [==============================] - 8s 648us/sample - loss: 0.0537 - acc: 0.9828s - - ETA - ETA
Epoch 5/5
12604/12604 [==============================] - 8s 645us/sample - loss: 0.0428 - acc: 0.9864
2019-12-22 04:07:52.000248 End of fit
acc: 97.85%
2019-12-22 04:07:59.534663 Start of fit
Epoch 1/5
12604/12604 [==============================] - 12s 970us/sample - loss: 0.4854 - acc: 0.8477
Epoch 2/5
12604/12604 [==============================] - 8s 665us/sample - loss: 0.1276 - acc: 0.9614s - loss: 0.1 - ETA: 3s - loss: 0. - ETA: 
Epoch 3/5
12604/12604 [==============================] - 8s 659us/sample - loss: 0.0801 - acc: 0.9745s - loss:
Epoch 4/5
12604/12604 [==============================] - 8s 663us/sample - loss: 0.0561 - a

2019-12-22 04:34:14.045996 End of fit
acc: 98.22%
2019-12-22 04:34:21.877810 Start of fit
Epoch 1/5
12601/12601 [==============================] - 14s 1ms/sample - loss: 0.4649 - acc: 0.8526
Epoch 2/5
12601/12601 [==============================] - 9s 691us/sample - loss: 0.1254 - acc: 0.9614
Epoch 3/5
12601/12601 [==============================] - 9s 694us/sample - loss: 0.0810 - acc: 0.9729s - loss: 0.0625 - acc: 0.97 - ET - ETA: 2s - loss:  - ETA: 1s - loss: 0.0
Epoch 4/5
12601/12601 [==============================] - 9s 697us/sample - loss: 0.0568 - acc: 0.9826
Epoch 5/5
12601/12601 [==============================] - 9s 686us/sample - loss: 0.0416 - acc: 0.9871
2019-12-22 04:35:28.497357 End of fit
acc: 97.64%
2019-12-22 04:35:36.466317 Start of fit
Epoch 1/5
12602/12602 [==============================] - 13s 994us/sample - loss: 0.4781 - acc: 0.8495
Epoch 2/5
12602/12602 [==============================] - 8s 660us/sample - loss: 0.1333 - acc: 0.9589s - loss: 0.1473 - ac - ETA: 1s -

In [25]:
Config.NUM_KERAS_TRAIN_IMAGES=24_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=4_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-22 04:55:42.698987 ('Start of CNN model with similarity_vector TYPE=', 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(24000, 28, 28) (24000,)
image_label,image_no_label 24000 24000
The range (min num reps of a digit, max num reps of a digit):
2144 2732
2019-12-22 04:55:44.655965 Start get_similarity_vectors() processing
2019-12-22 04:55:54.954521 ('we have processed ', 999, ' images')
2019-12-22 04:56:06.923098 ('we have processed ', 1999, ' images')
2019-12-22 04:56:18.994698 ('we have processed ', 2999, ' images')
2019-12-22 04:56:30.679513 ('we have processed ', 3999, ' images')
2019-12-22 04:56:42.805563 ('we have processed ', 4999, ' images')
2019-12-22 04:56:55.487867 ('we have processed ', 5999, ' images')
2019-12-22 04:57:08.444567 ('we have processed ', 6999, ' images')
2019-12-22 04:57:20.768713 ('we have processed ', 7999, ' images')
2019-12-22 04:57:33.737903 ('we have processed ', 8999, ' images')
2019-12-22 04:57:47.226309 ('we have processed ', 9999, ' image

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 04:59:17.945775 ('we have processed ', 16999, ' images')
2019-12-22 04:59:30.005024 ('we have processed ', 17999, ' images')
2019-12-22 04:59:42.594884 ('we have processed ', 18999, ' images')
2019-12-22 04:59:55.522213 ('we have processed ', 19999, ' images')
2019-12-22 05:00:08.140555 ('we have processed ', 20999, ' images')
2019-12-22 05:00:20.861202 ('we have processed ', 21999, ' images')
2019-12-22 05:00:33.644738 ('we have processed ', 22999, ' images')
2019-12-22 05:00:46.284626 ('we have processed ', 23999, ' images')
2019-12-22 05:00:46.300245 End get_similarity_vectors() processing
The training accuracy is:  0.6163333333333333
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-22 05:00:48.094399 Start get_similarity_vectors() processing
2019-12-22 05:00:57.909022 ('we have processed ', 999, ' images')
2019-12-22 05:01:09.145458 ('we have processed ', 1999, ' images')
2019-12-22 05:01:20.783061 ('we have processed ', 2999, ' images')
2019

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 05:25:44.023712 ('we have processed ', 16999, ' images')
2019-12-22 05:25:56.098684 ('we have processed ', 17999, ' images')
2019-12-22 05:26:08.617411 ('we have processed ', 18999, ' images')
2019-12-22 05:26:21.540784 ('we have processed ', 19999, ' images')
2019-12-22 05:26:34.144937 ('we have processed ', 20999, ' images')
2019-12-22 05:26:46.948164 ('we have processed ', 21999, ' images')
2019-12-22 05:26:59.848940 ('we have processed ', 22999, ' images')
2019-12-22 05:27:12.774897 ('we have processed ', 23999, ' images')
2019-12-22 05:27:12.790550 End get_similarity_vectors() processing
The training accuracy is:  0.6163333333333333
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-22 05:27:14.604190 Start get_similarity_vectors() processing
2019-12-22 05:27:24.738612 ('we have processed ', 999, ' images')
2019-12-22 05:27:35.978463 ('we have processed ', 1999, ' images')
2019-12-22 05:27:47.734372 ('we have processed ', 2999, ' images')
2019

...

2144 2732
2019-12-22 06:17:37.901026 Start get_similarity_vectors() processing
2019-12-22 06:17:49.302111 ('we have processed ', 999, ' images')
2019-12-22 06:18:03.437544 ('we have processed ', 1999, ' images')
2019-12-22 06:18:17.003117 ('we have processed ', 2999, ' images')
2019-12-22 06:18:29.209166 ('we have processed ', 3999, ' images')
2019-12-22 06:18:42.150804 ('we have processed ', 4999, ' images')
2019-12-22 06:18:54.974265 ('we have processed ', 5999, ' images')
2019-12-22 06:19:07.538341 ('we have processed ', 6999, ' images')
2019-12-22 06:19:20.069333 ('we have processed ', 7999, ' images')
2019-12-22 06:19:33.216533 ('we have processed ', 8999, ' images')
2019-12-22 06:19:46.657728 ('we have processed ', 9999, ' images')
2019-12-22 06:20:00.685623 ('we have processed ', 10999, ' images')
2019-12-22 06:20:14.299711 ('we have processed ', 11999, ' images')
2019-12-22 06:20:28.699388 ('we have processed ', 12999, ' images')
2019-12-22 06:20:42.471207 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 06:21:21.963019 ('we have processed ', 16999, ' images')
2019-12-22 06:21:35.859979 ('we have processed ', 17999, ' images')
2019-12-22 06:21:48.441550 ('we have processed ', 18999, ' images')
2019-12-22 06:22:01.873060 ('we have processed ', 19999, ' images')
2019-12-22 06:22:14.867191 ('we have processed ', 20999, ' images')
2019-12-22 06:22:28.259225 ('we have processed ', 21999, ' images')
2019-12-22 06:22:41.728667 ('we have processed ', 22999, ' images')
2019-12-22 06:22:54.914668 ('we have processed ', 23999, ' images')
2019-12-22 06:22:54.930289 End get_similarity_vectors() processing
The training accuracy is:  0.6163333333333333
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-22 06:22:56.953053 Start get_similarity_vectors() processing
2019-12-22 06:23:07.025222 ('we have processed ', 999, ' images')
2019-12-22 06:23:18.081780 ('we have processed ', 1999, ' images')
2019-12-22 06:23:29.777678 ('we have processed ', 2999, ' images')
2019

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 06:49:45.032984 ('we have processed ', 16999, ' images')
2019-12-22 06:49:57.983161 ('we have processed ', 17999, ' images')
2019-12-22 06:50:10.388573 ('we have processed ', 18999, ' images')
2019-12-22 06:50:23.200598 ('we have processed ', 19999, ' images')
2019-12-22 06:50:35.913334 ('we have processed ', 20999, ' images')
2019-12-22 06:50:48.771434 ('we have processed ', 21999, ' images')
2019-12-22 06:51:01.707519 ('we have processed ', 22999, ' images')
2019-12-22 06:51:14.492565 ('we have processed ', 23999, ' images')
2019-12-22 06:51:14.508188 End get_similarity_vectors() processing
The training accuracy is:  0.6163333333333333
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-22 06:51:16.345912 Start get_similarity_vectors() processing
2019-12-22 06:51:26.282572 ('we have processed ', 999, ' images')
2019-12-22 06:51:37.555720 ('we have processed ', 1999, ' images')
2019-12-22 06:51:49.022146 ('we have processed ', 2999, ' images')
2019

In [26]:
Config.NUM_KERAS_TRAIN_IMAGES=36_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=6_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-22 07:13:53.278327 ('Start of CNN model with similarity_vector TYPE=', 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(36000, 28, 28) (36000,)
image_label,image_no_label 36000 36000
The range (min num reps of a digit, max num reps of a digit):
3230 4113
2019-12-22 07:13:55.364918 Start get_similarity_vectors() processing
2019-12-22 07:14:05.878072 ('we have processed ', 999, ' images')
2019-12-22 07:14:17.932800 ('we have processed ', 1999, ' images')
2019-12-22 07:14:30.389998 ('we have processed ', 2999, ' images')
2019-12-22 07:14:42.647667 ('we have processed ', 3999, ' images')
2019-12-22 07:14:55.300078 ('we have processed ', 4999, ' images')
2019-12-22 07:15:08.111795 ('we have processed ', 5999, ' images')
2019-12-22 07:15:20.710766 ('we have processed ', 6999, ' images')
2019-12-22 07:15:33.294473 ('we have processed ', 7999, ' images')
2019-12-22 07:15:46.391643 ('we have processed ', 8999, ' images')
2019-12-22 07:15:59.784572 ('we have processed ', 9999, ' image

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 07:17:31.116874 ('we have processed ', 16999, ' images')
2019-12-22 07:17:43.676574 ('we have processed ', 17999, ' images')
2019-12-22 07:17:56.080692 ('we have processed ', 18999, ' images')
2019-12-22 07:18:08.979923 ('we have processed ', 19999, ' images')
2019-12-22 07:18:21.579612 ('we have processed ', 20999, ' images')
2019-12-22 07:18:34.443678 ('we have processed ', 21999, ' images')
2019-12-22 07:18:47.441697 ('we have processed ', 22999, ' images')
2019-12-22 07:19:00.368015 ('we have processed ', 23999, ' images')
2019-12-22 07:19:13.090581 ('we have processed ', 24999, ' images')
2019-12-22 07:19:26.101734 ('we have processed ', 25999, ' images')
2019-12-22 07:19:38.974691 ('we have processed ', 26999, ' images')
2019-12-22 07:19:52.105047 ('we have processed ', 27999, ' images')
2019-12-22 07:20:05.548493 ('we have processed ', 28999, ' images')
2019-12-22 07:20:18.451048 ('we have processed ', 29999, ' images')
2019-12-22 07:20:31.350746 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 07:56:25.350553 ('we have processed ', 16999, ' images')
2019-12-22 07:56:38.025059 ('we have processed ', 17999, ' images')
2019-12-22 07:56:50.461377 ('we have processed ', 18999, ' images')
2019-12-22 07:57:03.607145 ('we have processed ', 19999, ' images')
2019-12-22 07:57:16.411117 ('we have processed ', 20999, ' images')
2019-12-22 07:57:29.210462 ('we have processed ', 21999, ' images')
2019-12-22 07:57:42.106458 ('we have processed ', 22999, ' images')
2019-12-22 07:57:54.913456 ('we have processed ', 23999, ' images')
2019-12-22 07:58:07.696799 ('we have processed ', 24999, ' images')
2019-12-22 07:58:20.727780 ('we have processed ', 25999, ' images')
2019-12-22 07:58:33.446764 ('we have processed ', 26999, ' images')
2019-12-22 07:58:46.562690 ('we have processed ', 27999, ' images')
2019-12-22 07:58:59.987490 ('we have processed ', 28999, ' images')
2019-12-22 07:59:12.886260 ('we have processed ', 29999, ' images')
2019-12-22 07:59:25.707526 ('we have processed '

...

2019-12-22 09:03:37.640226 Start of fit
Epoch 1/5
37802/37802 [==============================] - 35s 926us/sample - loss: 0.2417 - acc: 0.9234 - l - E - ETA: 1s - loss: 0.2464
Epoch 2/5
37802/37802 [==============================] - 28s 746us/sample - loss: 0.0604 - acc: 0.9807
Epoch 3/5
37802/37802 [==============================] - 28s 748us/sample - loss: 0.0403 - acc: 0.9866 - loss: 0.0405 - acc: 0. - ETA: 8s - loss: 0.0409 - ac - ETA: 8s  - ETA: 6s - loss: 0.0401 - ac - ETA: 5s - loss: 0.0401 - acc: - ETA: 5s - loss: 0.0398 - acc: 0.9 - ETA: 4s - loss: 0.0396 - acc:  - ETA:  - ETA: 2s - loss: 0.0403 - ac - ETA: 1s - loss: 0 - ETA: 0s - loss: 0.0402 - acc: 
Epoch 4/5
37802/37802 [==============================] - 28s 753us/sample - loss: 0.0301 - acc: 0.9902
Epoch 5/5
37802/37802 [==============================] - 28s 753us/sample - loss: 0.0229 - acc: 0.9924 - loss: 0.0232  - ETA: 
2019-12-22 09:07:14.023636 End of fit
acc: 98.79%
2019-12-22 09:07:26.361802 Start of fit
Epoch 1/5


C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 09:17:28.855465 ('we have processed ', 16999, ' images')
2019-12-22 09:17:41.649572 ('we have processed ', 17999, ' images')
2019-12-22 09:17:54.150331 ('we have processed ', 18999, ' images')
2019-12-22 09:18:07.215201 ('we have processed ', 19999, ' images')
2019-12-22 09:18:20.014603 ('we have processed ', 20999, ' images')
2019-12-22 09:18:32.780600 ('we have processed ', 21999, ' images')
2019-12-22 09:18:45.734273 ('we have processed ', 22999, ' images')
2019-12-22 09:18:58.827939 ('we have processed ', 23999, ' images')
2019-12-22 09:19:11.775948 ('we have processed ', 24999, ' images')
2019-12-22 09:19:24.926755 ('we have processed ', 25999, ' images')
2019-12-22 09:19:37.869363 ('we have processed ', 26999, ' images')
2019-12-22 09:19:51.113191 ('we have processed ', 27999, ' images')
2019-12-22 09:20:04.782993 ('we have processed ', 28999, ' images')
2019-12-22 09:20:17.853713 ('we have processed ', 29999, ' images')
2019-12-22 09:20:30.904771 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 09:58:56.817768 ('we have processed ', 16999, ' images')
2019-12-22 09:59:09.319656 ('we have processed ', 17999, ' images')
2019-12-22 09:59:21.899332 ('we have processed ', 18999, ' images')
2019-12-22 09:59:35.086638 ('we have processed ', 19999, ' images')
2019-12-22 09:59:47.920809 ('we have processed ', 20999, ' images')
2019-12-22 10:00:00.831622 ('we have processed ', 21999, ' images')
2019-12-22 10:00:13.888129 ('we have processed ', 22999, ' images')
2019-12-22 10:00:27.096769 ('we have processed ', 23999, ' images')
2019-12-22 10:00:40.124151 ('we have processed ', 24999, ' images')
2019-12-22 10:00:53.308909 ('we have processed ', 25999, ' images')
2019-12-22 10:01:06.348950 ('we have processed ', 26999, ' images')
2019-12-22 10:01:19.587428 ('we have processed ', 27999, ' images')
2019-12-22 10:01:33.383480 ('we have processed ', 28999, ' images')
2019-12-22 10:01:46.488623 ('we have processed ', 29999, ' images')
2019-12-22 10:01:59.573841 ('we have processed '

In [27]:
Config.NUM_KERAS_TRAIN_IMAGES=48_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=8_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-22 10:37:14.374216 ('Start of CNN model with similarity_vector TYPE=', 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(48000, 28, 28) (48000,)
image_label,image_no_label 48000 48000
The range (min num reps of a digit, max num reps of a digit):
4324 5470
2019-12-22 10:37:16.782612 Start get_similarity_vectors() processing
2019-12-22 10:37:27.158822 ('we have processed ', 999, ' images')
2019-12-22 10:37:39.344558 ('we have processed ', 1999, ' images')
2019-12-22 10:37:51.834325 ('we have processed ', 2999, ' images')
2019-12-22 10:38:04.285724 ('we have processed ', 3999, ' images')
2019-12-22 10:38:17.047833 ('we have processed ', 4999, ' images')
2019-12-22 10:38:30.280425 ('we have processed ', 5999, ' images')
2019-12-22 10:38:43.169150 ('we have processed ', 6999, ' images')
2019-12-22 10:38:55.542692 ('we have processed ', 7999, ' images')
2019-12-22 10:39:08.716128 ('we have processed ', 8999, ' images')
2019-12-22 10:39:22.458097 ('we have processed ', 9999, ' image

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 10:40:58.901566 ('we have processed ', 16999, ' images')
2019-12-22 10:41:11.641297 ('we have processed ', 17999, ' images')
2019-12-22 10:41:24.389716 ('we have processed ', 18999, ' images')
2019-12-22 10:41:37.465632 ('we have processed ', 19999, ' images')
2019-12-22 10:41:50.489363 ('we have processed ', 20999, ' images')
2019-12-22 10:42:03.486992 ('we have processed ', 21999, ' images')
2019-12-22 10:42:16.718226 ('we have processed ', 22999, ' images')
2019-12-22 10:42:29.746848 ('we have processed ', 23999, ' images')
2019-12-22 10:42:42.700843 ('we have processed ', 24999, ' images')
2019-12-22 10:42:55.926980 ('we have processed ', 25999, ' images')
2019-12-22 10:43:08.899274 ('we have processed ', 26999, ' images')
2019-12-22 10:43:22.256394 ('we have processed ', 27999, ' images')
2019-12-22 10:43:35.854517 ('we have processed ', 28999, ' images')
2019-12-22 10:43:49.001569 ('we have processed ', 29999, ' images')
2019-12-22 10:44:02.178038 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 11:37:27.241502 ('we have processed ', 16999, ' images')
2019-12-22 11:37:40.683038 ('we have processed ', 17999, ' images')
2019-12-22 11:37:53.938029 ('we have processed ', 18999, ' images')
2019-12-22 11:38:07.973761 ('we have processed ', 19999, ' images')
2019-12-22 11:38:21.290919 ('we have processed ', 20999, ' images')
2019-12-22 11:38:34.648515 ('we have processed ', 21999, ' images')
2019-12-22 11:38:47.937505 ('we have processed ', 22999, ' images')
2019-12-22 11:39:01.178319 ('we have processed ', 23999, ' images')
2019-12-22 11:39:14.328722 ('we have processed ', 24999, ' images')
2019-12-22 11:39:27.747264 ('we have processed ', 25999, ' images')
2019-12-22 11:39:41.532420 ('we have processed ', 26999, ' images')
2019-12-22 11:39:54.860583 ('we have processed ', 27999, ' images')
2019-12-22 11:40:08.918276 ('we have processed ', 28999, ' images')
2019-12-22 11:40:22.318705 ('we have processed ', 29999, ' images')
2019-12-22 11:40:35.630896 ('we have processed '

...

2019-12-22 12:49:49.364784 ('we have processed ', 16999, ' images')
2019-12-22 12:50:02.867864 ('we have processed ', 17999, ' images')
2019-12-22 12:50:15.952065 ('we have processed ', 18999, ' images')
2019-12-22 12:50:29.709280 ('we have processed ', 19999, ' images')
2019-12-22 12:50:43.298219 ('we have processed ', 20999, ' images')
2019-12-22 12:50:56.753922 ('we have processed ', 21999, ' images')
2019-12-22 12:51:10.495268 ('we have processed ', 22999, ' images')
2019-12-22 12:51:24.212141 ('we have processed ', 23999, ' images')
2019-12-22 12:51:37.814561 ('we have processed ', 24999, ' images')
2019-12-22 12:51:51.897937 ('we have processed ', 25999, ' images')
2019-12-22 12:52:05.786187 ('we have processed ', 26999, ' images')
2019-12-22 12:52:19.588183 ('we have processed ', 27999, ' images')
2019-12-22 12:52:33.743624 ('we have processed ', 28999, ' images')
2019-12-22 12:52:47.801791 ('we have processed ', 29999, ' images')
2019-12-22 12:53:01.474666 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 13:51:00.326391 ('we have processed ', 16999, ' images')
2019-12-22 13:51:13.644137 ('we have processed ', 17999, ' images')
2019-12-22 13:51:26.428894 ('we have processed ', 18999, ' images')
2019-12-22 13:51:39.957666 ('we have processed ', 19999, ' images')
2019-12-22 13:51:53.410421 ('we have processed ', 20999, ' images')
2019-12-22 13:52:06.686632 ('we have processed ', 21999, ' images')
2019-12-22 13:52:20.129142 ('we have processed ', 22999, ' images')
2019-12-22 13:52:33.342477 ('we have processed ', 23999, ' images')
2019-12-22 13:52:46.513762 ('we have processed ', 24999, ' images')
2019-12-22 13:52:59.713877 ('we have processed ', 25999, ' images')
2019-12-22 13:53:12.921210 ('we have processed ', 26999, ' images')
2019-12-22 13:53:26.293211 ('we have processed ', 27999, ' images')
2019-12-22 13:53:39.953278 ('we have processed ', 28999, ' images')
2019-12-22 13:53:53.433993 ('we have processed ', 29999, ' images')
2019-12-22 13:54:07.070862 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 14:51:10.109110 ('we have processed ', 16999, ' images')
2019-12-22 14:51:23.193820 ('we have processed ', 17999, ' images')
2019-12-22 14:51:36.108184 ('we have processed ', 18999, ' images')
2019-12-22 14:51:49.663076 ('we have processed ', 19999, ' images')
2019-12-22 14:52:02.854711 ('we have processed ', 20999, ' images')
2019-12-22 14:52:16.393707 ('we have processed ', 21999, ' images')
2019-12-22 14:52:29.899022 ('we have processed ', 22999, ' images')
2019-12-22 14:52:43.223743 ('we have processed ', 23999, ' images')
2019-12-22 14:52:56.721021 ('we have processed ', 24999, ' images')
2019-12-22 14:53:10.196372 ('we have processed ', 25999, ' images')
2019-12-22 14:53:23.555547 ('we have processed ', 26999, ' images')
2019-12-22 14:53:37.077867 ('we have processed ', 27999, ' images')
2019-12-22 14:53:50.934797 ('we have processed ', 28999, ' images')
2019-12-22 14:54:04.768714 ('we have processed ', 29999, ' images')
2019-12-22 14:54:18.162723 ('we have processed '

In [28]:
Config.NUM_KERAS_TRAIN_IMAGES=60_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=10_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

2019-12-22 15:48:51.447869 ('Start of CNN model with similarity_vector TYPE=', 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(60000, 28, 28) (60000,)
image_label,image_no_label 60000 60000
The range (min num reps of a digit, max num reps of a digit):
5421 6742
2019-12-22 15:48:54.378020 Start get_similarity_vectors() processing
2019-12-22 15:49:05.110389 ('we have processed ', 999, ' images')
2019-12-22 15:49:18.314078 ('we have processed ', 1999, ' images')
2019-12-22 15:49:31.561639 ('we have processed ', 2999, ' images')
2019-12-22 15:49:44.520124 ('we have processed ', 3999, ' images')
2019-12-22 15:49:58.093389 ('we have processed ', 4999, ' images')
2019-12-22 15:50:11.551742 ('we have processed ', 5999, ' images')
2019-12-22 15:50:25.013498 ('we have processed ', 6999, ' images')
2019-12-22 15:50:38.026883 ('we have processed ', 7999, ' images')
2019-12-22 15:50:52.287402 ('we have processed ', 8999, ' images')
2019-12-22 15:51:07.534161 ('we have processed ', 9999, ' image

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 15:52:47.307923 ('we have processed ', 16999, ' images')
2019-12-22 15:53:00.452445 ('we have processed ', 17999, ' images')
2019-12-22 15:53:13.682830 ('we have processed ', 18999, ' images')
2019-12-22 15:53:27.384115 ('we have processed ', 19999, ' images')
2019-12-22 15:53:40.898033 ('we have processed ', 20999, ' images')
2019-12-22 15:53:54.303410 ('we have processed ', 21999, ' images')
2019-12-22 15:54:08.170822 ('we have processed ', 22999, ' images')
2019-12-22 15:54:21.647023 ('we have processed ', 23999, ' images')
2019-12-22 15:54:34.987837 ('we have processed ', 24999, ' images')
2019-12-22 15:54:48.501112 ('we have processed ', 25999, ' images')
2019-12-22 15:55:01.876234 ('we have processed ', 26999, ' images')
2019-12-22 15:55:15.645796 ('we have processed ', 27999, ' images')
2019-12-22 15:55:29.616405 ('we have processed ', 28999, ' images')
2019-12-22 15:55:43.189609 ('we have processed ', 29999, ' images')
2019-12-22 15:55:56.554203 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 17:10:25.598321 ('we have processed ', 16999, ' images')
2019-12-22 17:10:38.737532 ('we have processed ', 17999, ' images')
2019-12-22 17:10:52.229779 ('we have processed ', 18999, ' images')
2019-12-22 17:11:06.400010 ('we have processed ', 19999, ' images')
2019-12-22 17:11:20.236534 ('we have processed ', 20999, ' images')
2019-12-22 17:11:34.040698 ('we have processed ', 21999, ' images')
2019-12-22 17:11:47.861497 ('we have processed ', 22999, ' images')
2019-12-22 17:12:01.712506 ('we have processed ', 23999, ' images')
2019-12-22 17:12:15.297953 ('we have processed ', 24999, ' images')
2019-12-22 17:12:29.034323 ('we have processed ', 25999, ' images')
2019-12-22 17:12:42.760408 ('we have processed ', 26999, ' images')
2019-12-22 17:12:56.978018 ('we have processed ', 27999, ' images')
2019-12-22 17:13:11.253289 ('we have processed ', 28999, ' images')
2019-12-22 17:13:25.247288 ('we have processed ', 29999, ' images')
2019-12-22 17:13:39.177401 ('we have processed '

...

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 19:49:53.283573 ('we have processed ', 16999, ' images')
2019-12-22 19:50:06.360378 ('we have processed ', 17999, ' images')
2019-12-22 19:50:19.480345 ('we have processed ', 18999, ' images')
2019-12-22 19:50:33.244842 ('we have processed ', 19999, ' images')
2019-12-22 19:50:46.982877 ('we have processed ', 20999, ' images')
2019-12-22 19:51:00.558452 ('we have processed ', 21999, ' images')
2019-12-22 19:51:14.343045 ('we have processed ', 22999, ' images')
2019-12-22 19:51:28.008344 ('we have processed ', 23999, ' images')
2019-12-22 19:51:41.879414 ('we have processed ', 24999, ' images')
2019-12-22 19:51:55.768061 ('we have processed ', 25999, ' images')
2019-12-22 19:52:09.383075 ('we have processed ', 26999, ' images')
2019-12-22 19:52:23.240254 ('we have processed ', 27999, ' images')
2019-12-22 19:52:37.495801 ('we have processed ', 28999, ' images')
2019-12-22 19:52:51.416088 ('we have processed ', 29999, ' images')
2019-12-22 19:53:05.121180 ('we have processed '

2019-12-22 21:09:07.558892 ('we have processed ', 3999, ' images')
2019-12-22 21:09:21.457665 ('we have processed ', 4999, ' images')
2019-12-22 21:09:35.632589 ('we have processed ', 5999, ' images')
2019-12-22 21:09:49.462747 ('we have processed ', 6999, ' images')
2019-12-22 21:10:02.930985 ('we have processed ', 7999, ' images')
2019-12-22 21:10:17.463782 ('we have processed ', 8999, ' images')
2019-12-22 21:10:32.515756 ('we have processed ', 9999, ' images')
2019-12-22 21:10:48.573866 ('we have processed ', 10999, ' images')
2019-12-22 21:11:03.526850 ('we have processed ', 11999, ' images')
2019-12-22 21:11:18.662250 ('we have processed ', 12999, ' images')
2019-12-22 21:11:33.619928 ('we have processed ', 13999, ' images')
2019-12-22 21:11:47.927950 ('we have processed ', 14999, ' images')
2019-12-22 21:12:01.328182 ('we have processed ', 15999, ' images')


C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-22 21:12:14.440605 ('we have processed ', 16999, ' images')
2019-12-22 21:12:27.750170 ('we have processed ', 17999, ' images')
2019-12-22 21:12:41.086691 ('we have processed ', 18999, ' images')
2019-12-22 21:12:54.899325 ('we have processed ', 19999, ' images')
2019-12-22 21:13:08.455187 ('we have processed ', 20999, ' images')
2019-12-22 21:13:22.206367 ('we have processed ', 21999, ' images')
2019-12-22 21:13:35.976324 ('we have processed ', 22999, ' images')
2019-12-22 21:13:49.691664 ('we have processed ', 23999, ' images')
2019-12-22 21:14:03.477814 ('we have processed ', 24999, ' images')
2019-12-22 21:14:17.261612 ('we have processed ', 25999, ' images')
2019-12-22 21:14:31.008429 ('we have processed ', 26999, ' images')
2019-12-22 21:14:45.059910 ('we have processed ', 27999, ' images')
2019-12-22 21:14:59.510777 ('we have processed ', 28999, ' images')
2019-12-22 21:15:13.272359 ('we have processed ', 29999, ' images')
2019-12-22 21:15:27.022725 ('we have processed '

# comparison summary of accuracy of 5 reps of 10-fold cross-validation


In [18]:
Config.SIMILARITY_VECTOR_TYPE=-4

Config.NUM_KERAS_TRAIN_IMAGES=12_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=2_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-26 17:35:58.128383 ('Start of CNN model with similarity_vector TYPE=', -4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(12000, 28, 28) (12000,)
image_label,image_no_label 12000 12000
The range (min num reps of a digit, max num reps of a digit):
1048 1351
2019-12-26 17:35:58.675821 Start get_similarity_vectors() processing
2019-12-26 17:35:59.602476 ('we have processed ', 999, ' images')
2019-12-26 17:36:00.795589 ('we have processed ', 1999, ' images')
2019-12-26 17:36:02.357555 ('we have processed ', 2999, ' images')
2019-12-26 17:36:04.379046 ('we have processed ', 3999, ' images')
2019-12-26 17:36:06.834076 ('we have processed ', 4999, ' images')
2019-12-26 17:36:09.768392 ('we have processed ', 5999, ' images')
2019-12-26 17:36:13.277675 ('we have processed ', 6999, ' images')
2019-12-26 17:36:17.189745 ('we have processed ', 7999, ' images')
2019-12-26 17:36:21.718747 ('we have processed ', 8999, ' images')
2019-12-26 17:36:26.782911 ('we have processed ', 9999, ' imag

12598/12598 [==============================] - 7s 559us/sample - loss: 0.1233 - acc: 0.9620s - loss: 0.17 - E
Epoch 3/5
12598/12598 [==============================] - 7s 552us/sample - loss: 0.0765 - acc: 0.9762s - loss: 0.0760 - acc:  - ETA: 5s - loss: 0. - ETA: 4 - ETA: 2s  - ETA: 0s - loss: 0.0769 - a - ETA: 0s - loss: 0.0772 - acc: 0.
Epoch 4/5
12598/12598 [==============================] - 7s 538us/sample - loss: 0.0541 - acc: 0.9829s - loss: 0.0562 - a - ETA: 5s - loss: 0.0570 - - ETA: 5s - loss: 0.0554 - acc: 0. - ETA: 4s - loss: 0.0523 - acc: - ETA: 4s - loss: 0.0538 - acc: 0.9 - ETA: 4s - loss: 0.0549 - acc - ETA: 3s - loss: 0.0585 - acc - ETA: 3s - loss: 0.0584 - ac - ETA: 2s - loss - ETA: 1s - loss: 0.0555 - acc: 0 - ETA: 0s - loss: 0.0547 - acc: 0.98 - ETA: 0s - loss: 0.0543 - ac
Epoch 5/5
12598/12598 [==============================] - 7s 526us/sample - loss: 0.0402 - acc: 0.9877s - loss: 0.0428 - acc: 0.98 - ETA: 5s - loss: 0.0443 - a - ETA: 5s - loss: 0.0397 - acc - ETA: 

12605/12605 [==============================] - 7s 540us/sample - loss: 0.0359 - acc: 0.9893s - loss: 0.0313 - acc - ETA: 5s - loss: 0.02 - ETA: 1s - loss:  - ETA: 0s - loss: 0.0362 - acc: 0.98
2019-12-26 17:44:42.346817 End of fit
acc: 98.49%
97.92% (+/- 1.00%)
2019-12-26 17:44:44.556709 End of CNN model with similarity_vector
2019-12-26 17:44:44.572384 ('Start of CNN model with similarity_vector TYPE=', -4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(12000, 28, 28) (12000,)
image_label,image_no_label 12000 12000
The range (min num reps of a digit, max num reps of a digit):
1048 1351
2019-12-26 17:44:45.209304 Start get_similarity_vectors() processing
2019-12-26 17:44:46.151422 ('we have processed ', 999, ' images')
2019-12-26 17:44:47.375385 ('we have processed ', 1999, ' images')
2019-12-26 17:44:48.969202 ('we have processed ', 2999, ' images')
2019-12-26 17:44:50.999675 ('we have processed ', 3999, ' images')
2019-12-26 17:44:53.487101 ('we have processed ', 4999, ' images')
2

12597/12597 [==============================] - 7s 543us/sample - loss: 0.0549 - acc: 0.9836s - loss: 0.0484 - acc: 0. - ETA: 5s - loss - ETA: 4s - loss: 0.0587 - acc:  - ETA: 4s - loss: 0.0574 - acc: 0 - ETA: 3s - loss: 0.0555 - acc:  - ETA: 3s - loss: 0.0524 - acc: 0.984 - ETA: 3s - loss: 0.0525 - acc: 0.984 - ETA: 3s - loss: 0.0518 - acc: 0.984 - ETA: 3s - loss: 0.0512 - acc:  - ETA: 0s - loss: 0.0550 - acc: 0.983
Epoch 5/5
12597/12597 [==============================] - 7s 537us/sample - loss: 0.0399 - acc: 0.9878s - loss: 0.0347 - acc:  - ETA: 5s - loss: 0.0371 - a - ETA: 5s - loss: 0.0399 - ac - E
2019-12-26 17:49:21.858195 End of fit
acc: 98.29%
2019-12-26 17:49:24.207454 Start of fit
Epoch 1/5
12598/12598 [==============================] - 8s 617us/sample - loss: 0.4884 - acc: 0.84703s - loss: 1 - ETA: 8s - loss: 1.1319 - acc:  - ETA: 7s - loss: 1.0142 - acc: 0.67 - ETA: 6s - loss: 0.9705 - acc: - ETA: 5s - loss: 0.8682 - acc:  - ETA: 5s - loss: 0.8015 - acc: 0.74 - ETA: 4s - los

12604/12604 [==============================] - 7s 551us/sample - loss: 0.0512 - acc: 0.9830s - loss: 0.0576 - acc: 0 - ETA: 6s - loss: - ETA: 2s - l - ETA: 0s - loss: 0.0527 - acc:
Epoch 5/5
12604/12604 [==============================] - 7s 545us/sample - loss: 0.0377 - acc: 0.9880 3s - loss: 0. - ETA: 2s - loss: 0.0362 - ETA: 1s - loss: 0.0369 - acc: 0. - ETA: 1s - loss: 0.
2019-12-26 17:52:56.382607 End of fit
acc: 97.06%
2019-12-26 17:52:58.817101 Start of fit
Epoch 1/5
12605/12605 [==============================] - 8s 627us/sample - loss: 0.4334 - acc: 0.8632s - loss: 1.2041 - acc:  - ETA: 8s - loss: 1.0286 - acc: 0. - ETA: 7s - loss: 0.9372 - a - ETA: 5s - loss: - ETA: 3s - loss: 0.6087 - acc: 0
Epoch 2/5
12605/12605 [==============================] - 7s 543us/sample - loss: 0.1099 - acc: 0.9645s - loss: 0.1338 - acc: 0.955 - ETA: 6s - loss: 0. - ETA: 5s - loss: 0.1356 - ac - ETA: 4s - loss: 0.1298 - ac - ETA: 3s - loss: 0.1277 - acc: 0.95 - ETA: 3s - loss: 0.1260 -  - ETA: 2s - l

12597/12597 [==============================] - 7s 553us/sample - loss: 0.0537 - acc: 0.9835s - loss: 0.0665 - acc: 0.9 - ETA: 6s - loss: 0.0563 - acc: 0. - ETA: 5s - l - ETA: 4s - loss: 0.0584 - acc - ETA: 4s - loss: 0.0581 - acc: 0.9 - ETA: 3s -  - ETA: 2s - loss: 0.0527 - acc: 0.98 - ETA: 2s - loss: 0. - ETA: 0s - loss: 0.0559 - acc: 0.98 - ETA: 0s - loss: 0.0557 - 
Epoch 5/5
12597/12597 [==============================] - 7s 547us/sample - loss: 0.0378 - acc: 0.9873s - loss: 0.0277 - - ETA: 5s - loss: 0.03 - ETA: 4s - loss: 0.0314 - acc: 0. - ETA: 4s - loss: 0.0317 - acc - ETA: 3s - loss: 0.0318 - acc:  - ETA: 3s - loss:  - ETA: 1s - loss: 0.0357 - ac - ETA: 1s - loss: 
2019-12-26 17:57:41.099733 End of fit
acc: 97.58%
2019-12-26 17:57:43.625252 Start of fit
Epoch 1/5
12597/12597 [==============================] - 8s 637us/sample - loss: 0.4373 - acc: 0.86202s - loss: 1.9463 - acc: 0.36 - ET - ETA: 7s - loss: 0.9350 - - ETA: 6s - loss: 0.7656 - acc - ETA: 5s - loss: 0 -
Epoch 2/5
125

12604/12604 [==============================] - 7s 564us/sample - loss: 0.1182 - acc: 0.9643s - loss: 0.1622 -  - ETA: 5s - loss:  - ETA: 4s - ETA: 2s - loss: 0.1253 - acc - ETA: 1s - loss: - ETA: 0s - loss: 0.1207 - ac
Epoch 3/5
12604/12604 [==============================] - 7s 550us/sample - loss: 0.0727 - acc: 0.9778s - loss: 0.0655 - acc: -  - ETA: 2s - lo - ETA: 1s - loss: 0.0719 -  - ETA: 0s - loss: 0.0736 - acc: 
Epoch 4/5
12604/12604 [==============================] - 7s 554us/sample - loss: 0.0524 - acc: 0.9836s - loss: 0.0470 - a - ETA: 4s - loss: 0.0470 - acc: - ETA: 3s - loss: 0.05 - ETA: 2s - loss: 0.0497 - acc: 0.9 - ETA: 2s - loss: 0.0482 - acc - ETA: 2s - loss: 0.0481 - acc: 0. - ETA: 1s - loss: 0.0491 - a - ETA: 1s - loss: 0.0522 - acc: 0. - ETA: 0s - loss: 0.0533
Epoch 5/5
12604/12604 [==============================] - 7s 557us/sample - loss: 0.0403 - acc: 0.9874s - lo - ETA: 0s - loss: 0.0412 
2019-12-26 18:01:22.140961 End of fit
acc: 98.42%
2019-12-26 18:01:24.85651

12597/12597 [==============================] - 9s 706us/sample - loss: 0.0773 - acc: 0.9767s - loss: 0.0780 - acc: 0.
Epoch 4/5
12597/12597 [==============================] - 7s 544us/sample - loss: 0.0544 - acc: 0.9832s - loss: 0.0541 - a
Epoch 5/5
12597/12597 [==============================] - 7s 550us/sample - loss: 0.0388 - acc: 0.9879
2019-12-26 18:06:53.499599 End of fit
acc: 97.72%
2019-12-26 18:06:56.244656 Start of fit
Epoch 1/5
12597/12597 [==============================] - 8s 634us/sample - loss: 0.4448 - acc: 0.8562s - loss: 0.6209 - acc: 0.7 - ETA: 3s - loss:  - ETA: 1s - loss: 0.5
Epoch 2/5
12597/12597 [==============================] - 7s 535us/sample - loss: 0.1183 - acc: 0.9640s - loss: 0.1327 - - ETA: 2s - loss: 0.1243 - acc:  - ETA: 2s - loss: 0.1216 -  - ETA: 1s - loss: 0.1214 - ac - ETA: 0s - loss: 0.1195 - 
Epoch 3/5
12597/12597 [==============================] - 7s 532us/sample - loss: 0.0719 - acc: 0.9781s - loss: 0.0753 - acc: 0.976 - ETA: 3s - loss: 0.0743 -  

2019-12-26 18:12:12.567453 ('we have processed ', 2999, ' images')
2019-12-26 18:12:14.618120 ('we have processed ', 3999, ' images')
2019-12-26 18:12:17.103477 ('we have processed ', 4999, ' images')
2019-12-26 18:12:20.073250 ('we have processed ', 5999, ' images')
2019-12-26 18:12:23.540167 ('we have processed ', 6999, ' images')
2019-12-26 18:12:27.480372 ('we have processed ', 7999, ' images')
2019-12-26 18:12:32.084124 ('we have processed ', 8999, ' images')
2019-12-26 18:12:37.336321 ('we have processed ', 9999, ' images')
2019-12-26 18:12:43.023280 ('we have processed ', 10999, ' images')
2019-12-26 18:12:49.197365 ('we have processed ', 11999, ' images')
2019-12-26 18:12:49.212985 End get_similarity_vectors() processing
The training accuracy is:  0.28458333333333335
The range (min num reps of a digit, max num reps of a digit):
175 234
2019-12-26 18:13:46.198254 Start get_similarity_vectors() processing
2019-12-26 18:13:47.328159 ('we have processed ', 999, ' images')
2019-12-2

12604/12604 [==============================] - 8s 661us/sample - loss: 0.4824 - acc: 0.8483TA: 7s - loss: 0.9338  - ETA: 1s - loss: 0.5358 - acc: 0.831 - ETA: 1s - loss: 0.5312 - acc: 0 - ETA: 0s - loss: 0.5104 - acc
Epoch 2/5
12604/12604 [==============================] - 7s 541us/sample - loss: 0.1265 - acc: 0.9612s - loss: 0.1430 - acc: 0 - ETA:  - ETA: 1
Epoch 3/5
12604/12604 [==============================] - 7s 548us/sample - loss: 0.0772 - acc: 0.9763s - loss: 0.0808 - acc: 0.9 - ETA: 5s - loss: 0.07 - ETA: 4s - loss: 0.0794 - ETA: 0s - loss: 0.0791 - acc: 0
Epoch 4/5
12604/12604 [==============================] - 7s 549us/sample - loss: 0.0558 - acc: 0.9825s - loss: 0.0 - ETA: 4s - los 
Epoch 5/5
12604/12604 [==============================] - 7s 550us/sample - loss: 0.0398 - acc: 0.9871
2019-12-26 18:19:59.649819 End of fit
acc: 98.21%
2019-12-26 18:20:02.754386 Start of fit
Epoch 1/5
12604/12604 [==============================] - 8s 669us/sample - loss: 0.4861 - acc: 0.8443
Ep

In [19]:
Config.SIMILARITY_VECTOR_TYPE=-4

Config.NUM_KERAS_TRAIN_IMAGES=24_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=4_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-26 18:21:35.253476 ('Start of CNN model with similarity_vector TYPE=', -4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(24000, 28, 28) (24000,)
image_label,image_no_label 24000 24000
The range (min num reps of a digit, max num reps of a digit):
2144 2732
2019-12-26 18:21:36.030868 Start get_similarity_vectors() processing
2019-12-26 18:21:37.178315 ('we have processed ', 999, ' images')
2019-12-26 18:21:38.345382 ('we have processed ', 1999, ' images')
2019-12-26 18:21:39.934659 ('we have processed ', 2999, ' images')
2019-12-26 18:21:42.012152 ('we have processed ', 3999, ' images')
2019-12-26 18:21:44.511761 ('we have processed ', 4999, ' images')
2019-12-26 18:21:47.534680 ('we have processed ', 5999, ' images')
2019-12-26 18:21:51.023115 ('we have processed ', 6999, ' images')
2019-12-26 18:21:55.021860 ('we have processed ', 7999, ' images')
2019-12-26 18:21:59.707708 ('we have processed ', 8999, ' images')
2019-12-26 18:22:04.998804 ('we have processed ', 9999, ' imag

25198/25198 [==============================] - 15s 597us/sample - loss: 0.0210 - acc: 0.9932s - loss: 0.0181 - ETA: 10s - loss: 0.01 - ETA: 9s - loss: 0.0193 - acc: 0.99 - ETA: 9s - loss: 0.0193 - ac - ETA: 9s - loss: 0.0208 - acc: - ETA: 8s - loss: 0.0213 - acc: 0. - ETA: 8s - loss: 0.0212 - - ETA: 7s - loss: 0.0213 - acc: - ETA: 7s - loss: 0.0216 - a - ETA: 3s - loss: 0.0208 - acc: 0 - ETA: 3s - loss: 0.0211 -  - ETA: 2s - loss: 0.0206  - ETA: 1s - loss: 0.0207  - ETA: 0s - loss: 0.0204 -
2019-12-26 18:34:12.374773 End of fit
acc: 98.82%
2019-12-26 18:34:16.185005 Start of fit
Epoch 1/5
25198/25198 [==============================] - 16s 640us/sample - loss: 0.2832 - acc: 0.9082 - loss: 0.2862 - acc: 0.
Epoch 2/5
25198/25198 [==============================] - 15s 609us/sample - loss: 0.0693 - acc: 0.9779 - loss: 0.0727 - acc: 0. - ETA: 5s - loss: 0.0724 - ac - ETA
Epoch 3/5
25198/25198 [==============================] - 15s 579us/sample - loss: 0.0434 - acc: 0.9860 - loss: 0.0419 - ac

25204/25204 [==============================] - 15s 595us/sample - loss: 0.0684 - acc: 0.9781 - loss: 0.0730 - acc: 0 - ETA: 4s - loss: 0.0721 -  - ETA: 4s - loss: 0.0714 - acc:  - ETA: 3s - loss: 0.0710 - acc: 0.977 - ETA: 3s - loss: 0.0714 - ac - ETA: 2s  - ETA: 0s - loss: 0.0692 - acc: 0. - ETA: 0s - loss: 0.0690 - acc: 0. - ETA: 0s - loss: 0.0693 - acc: 0
Epoch 3/5
25204/25204 [==============================] - 15s 582us/sample - loss: 0.0441 - acc: 0.9866s - loss:  - ETA: 8s - loss: 0.0420 - ETA: 7s - loss: 0.0411 - ac - ETA: 6s - loss: 0.0412 - - ETA: 5s - loss: 0.0427 - acc:  - - ET - ETA: 0s - loss: 0.0443 - acc: 
Epoch 4/5
25204/25204 [==============================] - 15s 581us/sample - loss: 0.0317 - acc: 0.9898s -  - ETA: 11s - loss - ETA: 9s - loss: 0.0314 - acc: 0.9 - ETA: - ETA: 4s - loss: 0.0296 - acc: - ETA: 4s - loss: 0.0300 - acc:  - ETA: 3s - loss: 0.0297 - acc: 0.990 - ETA: 3s - loss: 0.0296 - acc: 0 - ETA: 3s - loss: 0.0304  - ETA: 2s - loss: 0.0317 - a - ETA: 1s -

25198/25198 [==============================] - 17s 665us/sample - loss: 0.2719 - acc: 0.91491:14 - loss: 1.9206 - ac - ETA: 24s -  - ETA: 12s - loss: 0.5165 - ETA: 11s - loss: 0.4883 - acc - ETA: 11s - loss: 0.4750 - a - ETA: 10s - - ETA: 8s - loss: 0.4095  - ETA: 7s - loss: 0.3851 - acc: 0 - ETA: 7s  - ETA: 1s - loss: 0.2909 - acc: 0.90 - ETA: 1s - los
Epoch 2/5
25198/25198 [==============================] - 16s 620us/sample - loss: 0.0722 - acc: 0.9772s - loss - ETA: 10s - - ETA: 8s - loss: 0.0786 - acc: - ETA: 5s - loss: 0.0764 - a - ETA: 4s - loss: 0.0758 - acc: 0.97 - ETA: - ETA: 2s - loss: 0.0735 - acc: - ETA: 1s - l
Epoch 3/5
25198/25198 [==============================] - 15s 602us/sample - loss: 0.0457 - acc: 0.9852 - loss: 0.0457 - acc - ETA: 9s - loss: 0.0462 - acc - ETA: 8s - los - ETA: 7s - loss: 0.0453 - acc: - ETA: 7s  - ETA: 5s - loss: 0.04 - ETA: 4s - loss: 0.0456 -  - ETA: 3s - loss: 0 - ETA: 2s - loss: 0. - ETA: 1s - loss: 0.
Epoch 4/5
25198/25198 [===================

25203/25203 [==============================] - 17s 693us/sample - loss: 0.2723 - acc: 0.9143 ETA:  - ETA: 15s - loss: 0.5930 - - ETA: 14s  - ETA: 10s - loss - ETA: 8s -
Epoch 2/5
25203/25203 [==============================] - 15s 613us/sample - loss: 0.0666 - acc: 0.9787 - loss: 0.0668 - acc: 0.97
Epoch 3/5
25203/25203 [==============================] - 16s 619us/sample - loss: 0.0432 - acc: 0.9862s - l - ETA: 9s  - ETA: 7s - loss: 0.0456 - acc: 0.98 - ETA: 2s - loss: 0.0448 - acc: 0.9 - 
Epoch 4/5
25203/25203 [==============================] - 15s 607us/sample - loss: 0.0329 - acc: 0.9892s - loss: 0.0251 - a - ETA: 14s - loss: 0.0282 - a - - ETA: 11s - loss: 0. - ETA: 9s - loss: 0.0323 - - ETA: 8s - loss: 0.0313 - acc: 0.989 - ETA: 8s - loss: 0.0314 - ac - ETA: 7s - loss: 0 -  - ETA: 3s - loss: 0.0319 - acc: 0 - ETA: 3s - loss: 0.0316 - a - ETA: 3s - loss: 0.0316 - acc: 0.989 - ETA: 2s - loss: 0.0318 - a - ETA: 2s - loss: 0.0311 - a - ETA: 1s - l
Epoch 5/5
25203/25203 [===============

25195/25195 [==============================] - 15s 607us/sample - loss: 0.0726 - acc: 0.9779 - loss: 0.0
Epoch 3/5
25195/25195 [==============================] - 15s 601us/sample - loss: 0.0448 - acc: 0.9851s - loss - ETA - ETA: 11s - loss: 0. - ETA: 10s - loss: 0.0478 - acc:  - ETA: 10s - loss: 0.0477 - - ETA: 9s - loss: 0.0476 -  - ETA: 8s - loss: 0.0476 -  - ETA: 8s - loss: 0.0480 - acc: - ETA: 7s - loss: 0.0487 - acc: 0.983 - ETA: 7s - loss: 0.0485 - acc: 0.983 - ETA: 7s - loss: 0. - ETA: 6s - loss - ETA: 4s - loss: 0 - ETA: 3s - lo - ETA: 1s - los
Epoch 4/5
25195/25195 [==============================] - 15s 596us/sample - loss: 0.0327 - acc: 0.9896 
Epoch 5/5
25195/25195 [==============================] - 15s 595us/sample - loss: 0.0230 - acc: 0.9925 - loss: 0.0194 - acc: 0.9 - ETA: 9s - loss: 0.0192 - acc: 0.993 - ETA: 9s - loss - ETA: 7s 
2019-12-26 19:13:45.153574 End of fit
acc: 98.97%
2019-12-26 19:13:49.241667 Start of fit
Epoch 1/5
25196/25196 [=============================

25204/25204 [==============================] - 15s 613us/sample - loss: 0.0437 - acc: 0.9857 - loss: 0.0455 - acc: 0.984 - ETA: 4s  
Epoch 4/5
25204/25204 [==============================] - 15s 601us/sample - loss: 0.0315 - acc: 0.9903 - loss: 0.0318 - acc: 0. - ETA: 0s - loss: 0.0321
Epoch 5/5
25204/25204 [==============================] - 15s 601us/sample - loss: 0.0230 - acc: 0.9925s - loss: - ETA: 5s - los - ETA: 1s - loss: 0.0
2019-12-26 19:27:18.543388 End of fit
acc: 98.57%
98.57% (+/- 0.31%)
2019-12-26 19:27:22.766241 End of CNN model with similarity_vector
2019-12-26 19:27:22.813061 ('Start of CNN model with similarity_vector TYPE=', -4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(24000, 28, 28) (24000,)
image_label,image_no_label 24000 24000
The range (min num reps of a digit, max num reps of a digit):
2144 2732
2019-12-26 19:27:23.787073 Start get_similarity_vectors() processing
2019-12-26 19:27:25.096580 ('we have processed ', 999, ' images')
2019-12-26 19:27:26.367801

25203/25203 [==============================] - 16s 625us/sample - loss: 0.0482 - acc: 0.9852TA:  - ET - ETA: 1s -
Epoch 4/5
25203/25203 [==============================] - 15s 610us/sample - loss: 0.0344 - acc: 0.9887s - loss: 0.036 - ETA
Epoch 5/5
25203/25203 [==============================] - 15s 612us/sample - loss: 0.0261 - acc: 0.9923 - loss: 0.0258 - ET - ETA: 0s - loss: 0.0260 - acc:  - ETA: 0s - loss: 0.0258 - acc: 0.9
2019-12-26 19:45:18.763825 End of fit
acc: 98.57%
2019-12-26 19:45:23.316922 Start of fit
Epoch 1/5
25203/25203 [==============================] - 18s 695us/sample - loss: 0.2991 - acc: 0.9055
Epoch 2/5
25203/25203 [==============================] - 15s 604us/sample - loss: 0.0679 - acc: 0.9785 - loss: 0.0734  - ETA: 5s - loss: 0.0714 - acc: 0.97 - E
Epoch 3/5
25203/25203 [==============================] - 15s 609us/sample - loss: 0.0442 - acc: 0.9862
Epoch 4/5
25203/25203 [==============================] - 16s 620us/sample - loss: 0.0312 - acc: 0.9892 - loss: 0.0

25198/25198 [==============================] - 16s 615us/sample - loss: 0.0713 - acc: 0.9784
Epoch 3/5
25198/25198 [==============================] - 15s 614us/sample - loss: 0.0458 - acc: 0.9852s -  - ETA: 1 - - ETA: 6s - loss: 0.0433 - ETA
Epoch 4/5
25198/25198 [==============================] - 15s 613us/sample - loss: 0.0320 - acc: 0.9895 - loss: 0.0321 - ac - ETA: 0s - loss: 0.0321 - acc:  - ETA: 0s - loss: 0.0322 - acc: 0.98
Epoch 5/5
25198/25198 [==============================] - 15s 609us/sample - loss: 0.0260 - acc: 0.9915 - loss: 0.0268 - a - ET
2019-12-26 20:03:29.196351 End of fit
acc: 98.82%
2019-12-26 20:03:33.938653 Start of fit
Epoch 1/5
25198/25198 [==============================] - 18s 708us/sample - loss: 0.2912 - acc: 0.9074
Epoch 2/5
25198/25198 [==============================] - 16s 617us/sample - loss: 0.0713 - acc: 0.9763 - lo - ETA: 5s - loss: 0.0 - ET - ETA: 2s - loss: 0.0725 - acc: 0.976 - ETA: 1s - los - ETA: 0s - loss: 0.0716 - acc: 0.
Epoch 3/5
25198/25198

In [20]:
Config.SIMILARITY_VECTOR_TYPE=-4

Config.NUM_KERAS_TRAIN_IMAGES=36_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=6_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-26 20:23:35.413567 ('Start of CNN model with similarity_vector TYPE=', -4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(36000, 28, 28) (36000,)
image_label,image_no_label 36000 36000
The range (min num reps of a digit, max num reps of a digit):
3230 4113
2019-12-26 20:23:36.559578 Start get_similarity_vectors() processing
2019-12-26 20:23:37.999980 ('we have processed ', 999, ' images')
2019-12-26 20:23:39.360588 ('we have processed ', 1999, ' images')
2019-12-26 20:23:40.982496 ('we have processed ', 2999, ' images')
2019-12-26 20:23:43.160686 ('we have processed ', 3999, ' images')
2019-12-26 20:23:45.900960 ('we have processed ', 4999, ' images')
2019-12-26 20:23:49.103437 ('we have processed ', 5999, ' images')
2019-12-26 20:23:52.711161 ('we have processed ', 6999, ' images')
2019-12-26 20:23:56.884874 ('we have processed ', 7999, ' images')
2019-12-26 20:24:01.827242 ('we have processed ', 8999, ' images')
2019-12-26 20:24:07.457109 ('we have processed ', 9999, ' imag

37798/37798 [==============================] - 26s 679us/sample - loss: 0.2176 - acc: 0.9319s - lo - ETA: - ETA: 9s - loss: 0.2861  - ETA: 7s - loss: 0.2756 - acc: 0 - ETA: 7s - - ETA: 5s - loss:  - ETA: 3s - loss - ETA: 1s - loss:
Epoch 2/5
37798/37798 [==============================] - 23s 616us/sample - loss: 0.0539 - acc: 0.9829s - loss: 0.05 - ETA:   - ETA: 4s - - ETA: 2s - loss - ETA: 0s - loss: 0.0541 -
Epoch 3/5
37798/37798 [==============================] - 23s 614us/sample - loss: 0.0349 - acc: 0.9891
Epoch 4/5
37798/37798 [==============================] - 23s 621us/sample - loss: 0.0264 - acc: 0.9914 -  - ETA: 7 - ETA: 5s - loss: 0.025 - ETA: 4s - loss: 0.0258 - - ETA: 3s - loss: 0.0262 - acc: 0.991 - ET - ETA: 1s - loss: 0.0266 - acc: 0.99 - ETA: 0s - loss: 0.0266
Epoch 5/5
37798/37798 [==============================] - 23s 616us/sample - loss: 0.0193 - acc: 0.9938 - loss: 0.0180 - acc: 0. - ETA: 8s -  - E - ETA: 4s - loss: 0.0189 - acc: 0.993 - ETA: 4s - 
2019-12-26 20:49

2019-12-26 21:03:55.225842 ('we have processed ', 6999, ' images')
2019-12-26 21:03:59.462129 ('we have processed ', 7999, ' images')
2019-12-26 21:04:04.560445 ('we have processed ', 8999, ' images')
2019-12-26 21:04:10.240104 ('we have processed ', 9999, ' images')
2019-12-26 21:04:16.419795 ('we have processed ', 10999, ' images')
2019-12-26 21:04:23.040638 ('we have processed ', 11999, ' images')
2019-12-26 21:04:30.099001 ('we have processed ', 12999, ' images')
2019-12-26 21:04:37.565921 ('we have processed ', 13999, ' images')
2019-12-26 21:04:45.456848 ('we have processed ', 14999, ' images')
2019-12-26 21:04:54.053529 ('we have processed ', 15999, ' images')
2019-12-26 21:05:02.916991 ('we have processed ', 16999, ' images')
2019-12-26 21:05:12.108562 ('we have processed ', 17999, ' images')
2019-12-26 21:05:21.759823 ('we have processed ', 18999, ' images')
2019-12-26 21:05:31.969764 ('we have processed ', 19999, ' images')
2019-12-26 21:05:42.434001 ('we have processed ', 20

37798/37798 [==============================] - 25s 661us/sample - loss: 0.0285 - acc: 0.9912s  - ETA: 21s - loss: 0.0284 - acc: 0. - ETA: 21s - loss: 0.0277 - acc: 0.99 - ETA: 21s - loss: 0.0272 - - ETA: 21s - loss: 0.0274 - acc: 0. - ETA: 21s - loss: 0.0271 - a - ETA:  - ETA: 12s - loss: 0.0310 - acc: 0.99 - ETA: 12s - loss: 0.0311 - acc: 0. - ETA: 12s - loss: 0.03 - ETA: 10s - loss: 0.03 - ETA: 10s - loss: 0.0303 - acc - ETA: 10s - loss - ETA: 8s - loss: 0.0303 -  - ETA: 5s - loss: 0.0295 - - ETA: 4s - loss: 0.0300 - acc: - ETA: 4s - loss: 0. - ETA: 0s - loss: 0.0284 - acc: 0.9
Epoch 5/5
37798/37798 [==============================] - 25s 660us/sample - loss: 0.0210 - acc: 0.9932s - loss: 0.0148 - acc:  - ETA: 21 - ETA: 21s  - ETA: 19s - lo - ETA: 17s - loss: 0.0181 - - ETA: 16s - loss: 0.0186 - acc: 0. - ETA: 13s - loss:  - ETA: 11 - ETA: 10s - loss: 0.0204 - acc: 0.99 - ETA: 10s - loss: 0.0203 - a - ETA: 10s - ETA: 8s - l - ETA: 7s - loss: 0.0205 - acc:  - ETA: 6s - loss: 0.0205 - a

2019-12-26 21:46:53.182547 ('we have processed ', 20999, ' images')
2019-12-26 21:47:04.096719 ('we have processed ', 21999, ' images')
2019-12-26 21:47:15.544568 ('we have processed ', 22999, ' images')
2019-12-26 21:47:27.632899 ('we have processed ', 23999, ' images')
2019-12-26 21:47:40.118961 ('we have processed ', 24999, ' images')
2019-12-26 21:47:53.058085 ('we have processed ', 25999, ' images')
2019-12-26 21:48:06.364087 ('we have processed ', 26999, ' images')
2019-12-26 21:48:20.087222 ('we have processed ', 27999, ' images')
2019-12-26 21:48:34.205505 ('we have processed ', 28999, ' images')
2019-12-26 21:48:48.837109 ('we have processed ', 29999, ' images')
2019-12-26 21:49:03.745398 ('we have processed ', 30999, ' images')
2019-12-26 21:49:19.111644 ('we have processed ', 31999, ' images')
2019-12-26 21:49:34.867232 ('we have processed ', 32999, ' images')
2019-12-26 21:49:51.021014 ('we have processed ', 33999, ' images')
2019-12-26 21:50:07.574762 ('we have processed '

37802/37802 [==============================] - 24s 647us/sample - loss: 0.0536 - acc: 0.9836 - loss: 0.0581 - ac - ETA: 6s - los - 
Epoch 3/5
37802/37802 [==============================] - 25s 655us/sample - loss: 0.0352 - acc: 0.9888
Epoch 4/5
37802/37802 [==============================] - 25s 665us/sample - loss: 0.0260 - acc: 0.9913
Epoch 5/5
37802/37802 [==============================] - 25s 653us/sample - loss: 0.0192 - acc: 0.9940 - l - 
2019-12-26 22:20:58.541256 End of fit
acc: 99.02%
2019-12-26 22:21:04.895423 Start of fit
Epoch 1/5
37803/37803 [==============================] - 28s 734us/sample - loss: 0.2241 - acc: 0.9289
Epoch 2/5
37803/37803 [==============================] - 25s 658us/sample - loss: 0.0577 - acc: 0.9816 - loss: 0.058
Epoch 3/5
37803/37803 [==============================] - 25s 661us/sample - loss: 0.0390 - acc: 0.9878s - loss:  - ETA: 7s - loss: 0.0396 -  - ETA: 6s - loss: 0.0
Epoch 4/5
37803/37803 [==============================] - 25s 661us/sample - los

37798/37798 [==============================] - 25s 671us/sample - loss: 0.0270 - acc: 0.9922 - loss: 0.0265 - a
Epoch 5/5
37798/37798 [==============================] - 25s 664us/sample - loss: 0.0204 - acc: 0.9935 - los
2019-12-26 22:50:17.038517 End of fit
acc: 98.79%
2019-12-26 22:50:23.548612 Start of fit
Epoch 1/5
37798/37798 [==============================] - 28s 752us/sample - loss: 0.2200 - acc: 0.9310
Epoch 2/5
37798/37798 [==============================] - 25s 662us/sample - loss: 0.0545 - acc: 0.9828
Epoch 3/5
37798/37798 [==============================] - 25s 660us/sample - loss: 0.0369 - acc: 0.9875 - loss: 0.0372 - acc: 0.987 - ETA: 1s - loss: 0.037
Epoch 4/5
37798/37798 [==============================] - 25s 659us/sample - loss: 0.0266 - acc: 0.9915 - loss - ETA: 2s - loss: 0. - ETA: 1s - loss: 0.0255 - acc: 0.99 - ETA: 1s - loss: 0
Epoch 5/5
37798/37798 [==============================] - 25s 657us/sample - loss: 0.0190 - acc: 0.9934 - loss: 0.0190 - acc: 0.99
2019-12-26

2019-12-26 23:10:46.420625 ('we have processed ', 24999, ' images')
2019-12-26 23:10:59.524373 ('we have processed ', 25999, ' images')
2019-12-26 23:11:13.019773 ('we have processed ', 26999, ' images')
2019-12-26 23:11:26.973145 ('we have processed ', 27999, ' images')
2019-12-26 23:11:41.224126 ('we have processed ', 28999, ' images')
2019-12-26 23:11:55.924746 ('we have processed ', 29999, ' images')
2019-12-26 23:12:11.027761 ('we have processed ', 30999, ' images')
2019-12-26 23:12:26.550933 ('we have processed ', 31999, ' images')
2019-12-26 23:12:42.422177 ('we have processed ', 32999, ' images')
2019-12-26 23:12:58.790490 ('we have processed ', 33999, ' images')
2019-12-26 23:13:15.569130 ('we have processed ', 34999, ' images')
2019-12-26 23:13:32.678864 ('we have processed ', 35999, ' images')
2019-12-26 23:13:32.710109 End get_similarity_vectors() processing
The training accuracy is:  0.2853888888888889
The range (min num reps of a digit, max num reps of a digit):
548 686
2

37802/37802 [==============================] - 26s 687us/sample - loss: 0.0352 - acc: 0.98929s - loss: 0.0356
Epoch 4/5
37802/37802 [==============================] - 26s 688us/sample - loss: 0.0262 - acc: 0.9912 - loss: 0
Epoch 5/5
37802/37802 [==============================] - 26s 698us/sample - loss: 0.0206 - acc: 0.9933 - loss: 0.0209 - - ETA: 7s - loss: 0.0209 - acc: - ETA: 7s - loss: 0.0210 - acc: - ETA: 6s - loss: 0. - ETA
2019-12-26 23:45:16.859523 End of fit
acc: 98.79%
2019-12-26 23:45:23.947840 Start of fit
Epoch 1/5
37803/37803 [==============================] - 29s 779us/sample - loss: 0.2099 - acc: 0.9348 - loss: 0.2610 - ETA: 7s - loss: 0.2522 - acc: 0. - ETA: 6s - loss:  - ETA: 5s - loss: 0.2389 - acc: - ETA: 4s - los - ETA: 2s - loss: 0.2240 - a - ETA: 1s - los
Epoch 2/5
37803/37803 [==============================] - 26s 686us/sample - loss: 0.0511 - acc: 0.9831
Epoch 3/5
37803/37803 [==============================] - 26s 691us/sample - loss: 0.0336 - acc: 0.9896 - los

In [16]:
Config.SIMILARITY_VECTOR_TYPE=-4

Config.NUM_KERAS_TRAIN_IMAGES=48_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=8_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-27 03:24:03.780269 ('Start of CNN model with similarity_vector TYPE=', -4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(48000, 28, 28) (48000,)
image_label,image_no_label 48000 48000
The range (min num reps of a digit, max num reps of a digit):
4324 5470
2019-12-27 03:24:04.121467 Start get_similarity_vectors() processing
2019-12-27 03:24:04.734938 ('we have processed ', 999, ' images')
2019-12-27 03:24:05.780966 ('we have processed ', 1999, ' images')
2019-12-27 03:24:07.343600 ('we have processed ', 2999, ' images')
2019-12-27 03:24:09.360956 ('we have processed ', 3999, ' images')
2019-12-27 03:24:11.816543 ('we have processed ', 4999, ' images')
2019-12-27 03:24:14.686442 ('we have processed ', 5999, ' images')
2019-12-27 03:24:17.979344 ('we have processed ', 6999, ' images')
2019-12-27 03:24:21.732421 ('we have processed ', 7999, ' images')
2019-12-27 03:24:25.904869 ('we have processed ', 8999, ' images')
2019-12-27 03:24:30.479618 ('we have processed ', 9999, ' imag

50400/50400 [==============================] - 21s 423us/sample - loss: 0.0484 - acc: 0.9850
Epoch 3/5
50400/50400 [==============================] - 21s 422us/sample - loss: 0.0318 - acc: 0.9903
Epoch 4/5
50400/50400 [==============================] - 21s 420us/sample - loss: 0.0232 - acc: 0.9925
Epoch 5/5
50400/50400 [==============================] - 22s 427us/sample - loss: 0.0188 - acc: 0.9938
2019-12-27 04:00:51.787426 End of fit
acc: 99.07%
2019-12-27 04:00:52.820621 Start of fit
Epoch 1/5
50401/50401 [==============================] - 21s 425us/sample - loss: 0.1791 - acc: 0.9430
Epoch 2/5
50401/50401 [==============================] - 21s 418us/sample - loss: 0.0482 - acc: 0.9851 - loss: 0.0521 - acc: 0.9 - ETA: 8s - loss: -  - ETA - ETA: 0s - loss: 0.0485 
Epoch 3/5
50401/50401 [==============================] - 21s 421us/sample - loss: 0.0323 - acc: 0.9896 - loss: 0.0318 - acc: 0.989 - ETA: 1s - lo
Epoch 4/5
50401/50401 [==============================] - 21s 420us/sample - l

2019-12-27 04:16:57.343752 ('we have processed ', 42999, ' images')
2019-12-27 04:17:16.395674 ('we have processed ', 43999, ' images')
2019-12-27 04:17:35.783885 ('we have processed ', 44999, ' images')
2019-12-27 04:17:55.517481 ('we have processed ', 45999, ' images')
2019-12-27 04:18:15.804921 ('we have processed ', 46999, ' images')
2019-12-27 04:18:36.530790 ('we have processed ', 47999, ' images')
2019-12-27 04:18:36.546409 End get_similarity_vectors() processing
The training accuracy is:  0.28702083333333334
The range (min num reps of a digit, max num reps of a digit):
723 905
2019-12-27 04:34:21.980701 Start get_similarity_vectors() processing
2019-12-27 04:34:22.605531 ('we have processed ', 999, ' images')
2019-12-27 04:34:23.639198 ('we have processed ', 1999, ' images')
2019-12-27 04:34:25.236522 ('we have processed ', 2999, ' images')
2019-12-27 04:34:27.258333 ('we have processed ', 3999, ' images')
2019-12-27 04:34:29.759786 ('we have processed ', 4999, ' images')
2019-

50403/50403 [==============================] - 22s 429us/sample - loss: 0.0192 - acc: 0.9944
2019-12-27 04:56:31.761096 End of fit
acc: 98.82%
98.91% (+/- 0.13%)
2019-12-27 04:56:32.970679 End of CNN model with similarity_vector
2019-12-27 04:56:33.033163 ('Start of CNN model with similarity_vector TYPE=', -4)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(48000, 28, 28) (48000,)
image_label,image_no_label 48000 48000
The range (min num reps of a digit, max num reps of a digit):
4324 5470
2019-12-27 04:56:33.326484 Start get_similarity_vectors() processing
2019-12-27 04:56:33.966944 ('we have processed ', 999, ' images')
2019-12-27 04:56:35.010077 ('we have processed ', 1999, ' images')
2019-12-27 04:56:36.591021 ('we have processed ', 2999, ' images')
2019-12-27 04:56:38.703187 ('we have processed ', 3999, ' images')
2019-12-27 04:56:41.171887 ('we have processed ', 4999, ' images')
2019-12-27 04:56:44.063709 ('we have processed ', 5999, ' images')
2019-12-27 04:56:47.435047 ('we ha

50399/50399 [==============================] - 21s 420us/sample - loss: 0.0197 - acc: 0.9933 - loss: 0.0193 - acc: 0.99 - ETA: 1s - loss: 0.0195 - acc: 0.9 - ETA: 1s - lo
2019-12-27 05:31:29.566330 End of fit
acc: 98.98%
2019-12-27 05:31:30.959387 Start of fit
Epoch 1/5
50400/50400 [==============================] - 22s 428us/sample - loss: 0.1756 - acc: 0.9451 - los - ETA: 2s - loss: 0.1909 - acc: 0.94 - ETA: 2s - loss - ETA: 1s - loss: 0.1814 - acc: 0 - ETA: 0s - loss: 0.1802 - ac - ETA: 0s - loss: 0.1778 - acc: 0.944 - ETA: 0s - loss: 0.1776 - acc:
Epoch 2/5
50400/50400 [==============================] - 21s 425us/sample - loss: 0.0492 - acc: 0.9844
Epoch 3/5
50400/50400 [==============================] - 21s 422us/sample - loss: 0.0322 - acc: 0.9896 - loss: 0.032 - ETA: 4s - loss: 0.0319 - acc: 0
Epoch 4/5
50400/50400 [==============================] - 22s 427us/sample - loss: 0.0249 - acc: 0.9920
Epoch 5/5
50400/50400 [==============================] - 21s 426us/sample - loss: 0.0

2019-12-27 05:50:35.090118 ('we have processed ', 45999, ' images')
2019-12-27 05:50:55.338658 ('we have processed ', 46999, ' images')
2019-12-27 05:51:16.010938 ('we have processed ', 47999, ' images')
2019-12-27 05:51:16.026557 End get_similarity_vectors() processing
The training accuracy is:  0.284
The range (min num reps of a digit, max num reps of a digit):
723 905
2019-12-27 06:06:58.216841 Start get_similarity_vectors() processing
2019-12-27 06:06:58.881928 ('we have processed ', 999, ' images')
2019-12-27 06:06:59.950145 ('we have processed ', 1999, ' images')
2019-12-27 06:07:01.513818 ('we have processed ', 2999, ' images')
2019-12-27 06:07:03.501538 ('we have processed ', 3999, ' images')
2019-12-27 06:07:05.941892 ('we have processed ', 4999, ' images')
2019-12-27 06:07:08.828311 ('we have processed ', 5999, ' images')
2019-12-27 06:07:12.166002 ('we have processed ', 6999, ' images')
2019-12-27 06:07:15.938714 ('we have processed ', 7999, ' images')
2019-12-27 06:07:15.93

50403/50403 [==============================] - 23s 449us/sample - loss: 0.0467 - acc: 0.9850 ETA: 15s - loss:  - ETA: 15s - loss:  - ETA: 10
Epoch 3/5
50403/50403 [==============================] - 22s 441us/sample - loss: 0.0315 - acc: 0.9903 - loss: 0.0319
Epoch 4/5
50403/50403 [==============================] - 22s 437us/sample - loss: 0.0231 - acc: 0.9929s - loss: 0.0201 - - ETA: 2s - loss: 0.02 - ETA: 1s 
Epoch 5/5
50403/50403 [==============================] - 22s 441us/sample - loss: 0.0179 - acc: 0.9944 -  - ET
2019-12-27 06:28:02.269834 End of fit
acc: 99.07%
2019-12-27 06:28:03.929566 Start of fit
Epoch 1/5
50403/50403 [==============================] - 22s 445us/sample - loss: 0.1814 - acc: 0.9417 - loss: 0.1823 - acc: 0.
Epoch 2/5
50403/50403 [==============================] - 22s 443us/sample - loss: 0.0462 - acc: 0.9853 - loss: 0.0484  - ETA: 6s - los - ETA: 5s - loss: 0.0478 -  - ETA: 4s - loss: 0.04 - ETA: 0s - loss: 0.0
Epoch 3/5
50403/50403 [==========================

50396/50396 [==============================] - 24s 468us/sample - loss: 0.0185 - acc: 0.9942 ETA: 8s - loss: 0.0171 - acc: 0  - ETA: 3s - loss: - ETA: 2s - loss: 0.0180 - acc: 0 
2019-12-27 07:01:26.630352 End of fit
acc: 98.84%
2019-12-27 07:01:28.354100 Start of fit
Epoch 1/5
50398/50398 [==============================] - 23s 460us/sample - loss: 0.1787 - acc: 0.9434 - loss: 0.2202 - acc: 0.930 - ETA: 6s - loss: 0 - ETA: 2 - ETA: 1s - loss: 0
Epoch 2/5
50398/50398 [==============================] - 23s 459us/sample - loss: 0.0473 - acc: 0.9849 - loss: 0.0474 - acc: 
Epoch 3/5
50398/50398 [==============================] - 23s 464us/sample - loss: 0.0328 - acc: 0.9898 - loss: 0.0 - ETA: 1s - loss: 0 - ETA: 0s - loss: 0.0329 - acc: 0.9
Epoch 4/5
50398/50398 [==============================] - 23s 462us/sample - loss: 0.0230 - acc: 0.9928 - loss: 0.0230 - acc: 0.9 - ETA: 
Epoch 5/5
50398/50398 [==============================] - 23s 459us/sample - loss: 0.0186 - acc: 0.9944 - loss: 0. - E

In [ ]:
Config.SIMILARITY_VECTOR_TYPE=-4

Config.NUM_KERAS_TRAIN_IMAGES=60_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=10_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

# Results for improving the kNN model

accuracy   invariant vector description

 np.hstack((num_sector_sphere_pixels_np, similarity_vector))

 num_sector_sphere_pixels_np

 np.hstack((image_no_label.to_numpy(), num_sector_sphere_pixels_np, similarity_vector))

 np.hstack((image_no_label.to_numpy(), similarity_vector))

 image_no_label.to_numpy()

 similarity_vector

In [17]:
#     NUM_X_PIXELS=28
#     NUM_Y_PIXELS=28
#     PIXEL_CUTOFF=50
#     NUM_SPHERES=8
#     NUM_SECTORS_QUAD_1=3
#     NUM_SECTORS=4*NUM_SECTORS_QUAD_1
#     NUM_POWER_SUMS=4
#     KNOWN_NUM_CLUSTERS=10
#     TRAIN_NUM_ROWS=50_000#
#     TEST_NUM_ROWS=10_000#
#     NUM_NEIGHBORS=9
#     USE_AREA_INVAR=False
#     SIMILARITY_VECTOR_TYPE=1
#     NUM_EPOCHS=5
#     NUM_KERAS_TRAIN_IMAGES=60_000
#     NUM_KERAS_TRAIN_LABELS=NUM_KERAS_TRAIN_IMAGES
#     NUM_KERAS_TEST_IMAGES=10_000
#     NUM_KERAS_TEST_LABELS=NUM_KERAS_TEST_IMAGES
#     SCALE_SIMILARITY_VECTORS=True
#     USE_CENTROIDS=False
#     DO_PROFILING=False

# Does accuracy improvement help the most with fewer images?

# Some Config choice tuning: type of similarity vector

    if Config.SIMILARITY_VECTOR_TYPE==0:
        return np.hstack((num_sector_sphere_pixels_np, similarity_vector))
    elif Config.SIMILARITY_VECTOR_TYPE==1:
        return similarity_vector
    elif Config.SIMILARITY_VECTOR_TYPE==2:
        return num_sector_sphere_pixels_np
    elif Config.SIMILARITY_VECTOR_TYPE==3:
        return np.hstack((one_image.to_numpy(), num_sector_sphere_pixels_np, \
                          similarity_vector))
    elif Config.SIMILARITY_VECTOR_TYPE==4:
        return np.hstack((one_image.to_numpy(), similarity_vector))
    elif Config.SIMILARITY_VECTOR_TYPE==5:
        return one_image.to_numpy()
    elif Config.SIMILARITY_VECTOR_TYPE==-3:
        rand_vector=np.random.randint(0,high=256,\
                    size=(Config.NUM_SECTORS*Config.NUM_SPHERES))
        return np.hstack((rand_vector, similarity_vector))
    else:
        debug(0,('ERROR BAD SIMILARITY_VECTOR_TYPE'))


In [17]:
Config.SIMILARITY_VECTOR_TYPE=0

Config.NUM_KERAS_TRAIN_IMAGES=12_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=2_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-27 07:17:40.372796 ('Start of CNN model with similarity_vector TYPE=', 0)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(12000, 28, 28) (12000,)
image_label,image_no_label 12000 12000
The range (min num reps of a digit, max num reps of a digit):
1048 1351
2019-12-27 07:17:40.683522 Start get_similarity_vectors() processing
2019-12-27 07:17:52.678065 ('we have processed ', 999, ' images')
2019-12-27 07:18:03.080273 ('we have processed ', 1999, ' images')
2019-12-27 07:18:14.220706 ('we have processed ', 2999, ' images')
2019-12-27 07:18:25.515234 ('we have processed ', 3999, ' images')
2019-12-27 07:18:37.614785 ('we have processed ', 4999, ' images')
2019-12-27 07:18:50.506045 ('we have processed ', 5999, ' images')
2019-12-27 07:19:03.453093 ('we have processed ', 6999, ' images')
2019-12-27 07:19:16.562908 ('we have processed ', 7999, ' images')
2019-12-27 07:19:30.666304 ('we have processed ', 8999, ' images')
2019-12-27 07:19:45.229115 ('we have processed ', 9999, ' image

12604/12604 [==============================] - 6s 464us/sample - loss: 0.4507 - acc: 0.85561s - loss:  - ETA: 4s - l
Epoch 2/5
12604/12604 [==============================] - 5s 422us/sample - loss: 0.1115 - acc: 0.96562s - loss: 0.1206 -  - ETA: 2s - loss: 0.1171 - ETA: 1s - lo
Epoch 3/5
12604/12604 [==============================] - 5s 415us/sample - loss: 0.0711 - acc: 0.9772s - loss: 0.0692 - ETA: 1s - loss: 0.0
Epoch 4/5
12604/12604 [==============================] - 5s 413us/sample - loss: 0.0501 - acc: 0.9835s
Epoch 5/5
12604/12604 [==============================] - 5s 410us/sample - loss: 0.0376 - acc: 0.9897s - lo - ET
2019-12-27 07:25:40.668726 End of fit
acc: 98.21%
2019-12-27 07:25:41.938111 Start of fit
Epoch 1/5
12605/12605 [==============================] - 6s 460us/sample - loss: 0.4665 - acc: 0.8512
Epoch 2/5
12605/12605 [==============================] - 5s 420us/sample - loss: 0.1222 - acc: 0.9616s - los - ETA: 3s - l - ETA: 0s - loss: 0.1222 - acc: 0.961
Epoch 3/5
12

12601/12601 [==============================] - 5s 431us/sample - loss: 0.0506 - acc: 0.9844s - loss: 0.0480 - a - ETA: 3s - loss: 0.0472 - acc: 0.98 - ETA: 3s - loss: 0.0 - ETA: 
Epoch 5/5
12601/12601 [==============================] - 5s 416us/sample - loss: 0.0362 - acc: 0.9883A: 0s - loss: 0.0366 - acc: - ETA: 0s - loss: 0.0367 - acc: 0.98 - ETA: 0s - loss: 0.0362 - acc: 0.988
2019-12-27 07:32:44.181465 End of fit
acc: 98.21%
2019-12-27 07:32:45.565956 Start of fit
Epoch 1/5
12602/12602 [==============================] - 6s 469us/sample - loss: 0.4455 - acc: 0.8561s - loss: 0.4725 - acc
Epoch 2/5
12602/12602 [==============================] - 5s 422us/sample - loss: 0.1183 - acc: 0.9642s - loss: 0.1277 - acc:  - ETA: 1s - loss: 0.1246 - acc: 0. - ETA: 1s - los
Epoch 3/5
12602/12602 [==============================] - 5s 423us/sample - loss: 0.0779 - acc: 0.9760s - loss:  - ETA: 2s - lo - ETA: 1s - 
Epoch 4/5
12602/12602 [==============================] - 5s 422us/sample - loss: 0.052

12597/12597 [==============================] - 6s 439us/sample - loss: 0.0380 - acc: 0.9880s -  - ETA: 1s - loss: 0
2019-12-27 07:40:28.021010 End of fit
acc: 98.08%
2019-12-27 07:40:29.537782 Start of fit
Epoch 1/5
12598/12598 [==============================] - 6s 479us/sample - loss: 0.4633 - acc: 0.8538
Epoch 2/5
12598/12598 [==============================] - 5s 431us/sample - loss: 0.1219 - acc: 0.9610s -
Epoch 3/5
12598/12598 [==============================] - 5s 432us/sample - loss: 0.0737 - acc: 0.9772s - loss: 0.0754 -
Epoch 4/5
12598/12598 [==============================] - 5s 429us/sample - loss: 0.0519 - acc: 0.9845
Epoch 5/5
12598/12598 [==============================] - 5s 425us/sample - loss: 0.0401 - acc: 0.9872s - loss: 0.04 - ETA: 1s - loss: 0.0393 - acc: - ETA: 1s - 
2019-12-27 07:41:00.189379 End of fit
acc: 97.93%
2019-12-27 07:41:01.731767 Start of fit
Epoch 1/5
12601/12601 [==============================] - 6s 484us/sample - loss: 0.4339 - acc: 0.8629s -  - ETA: 0

Epoch 1/5
12597/12597 [==============================] - 6s 499us/sample - loss: 0.4473 - acc: 0.8596
Epoch 2/5
12597/12597 [==============================] - 6s 448us/sample - loss: 0.1156 - acc: 0.9652s - loss: 0.1460 - acc: - ETA: 3s - loss: 0. - ETA: 2s - loss: 0.1304 - acc: 0.9 - ETA: 2s - loss: 0.1277 - ac - ETA
Epoch 3/5
12597/12597 [==============================] - 6s 443us/sample - loss: 0.0738 - acc: 0.9773
Epoch 4/5
12597/12597 [==============================] - 6s 446us/sample - loss: 0.0536 - acc: 0.9825
Epoch 5/5
12597/12597 [==============================] - 6s 441us/sample - loss: 0.0381 - acc: 0.9878
2019-12-27 07:48:52.680301 End of fit
acc: 98.57%
2019-12-27 07:48:54.360358 Start of fit
Epoch 1/5
12597/12597 [==============================] - 6s 498us/sample - loss: 0.4814 - acc: 0.8488
Epoch 2/5
12597/12597 [==============================] - 6s 440us/sample - loss: 0.1276 - acc: 0.9617s
Epoch 3/5
12597/12597 [==============================] - 6s 438us/sample - loss

12595/12595 [==============================] - 6s 448us/sample - loss: 0.0666 - acc: 0.9799
Epoch 4/5
12595/12595 [==============================] - 6s 457us/sample - loss: 0.0471 - acc: 0.9848s - loss:  - ETA: 1s - loss: 0.0474 - acc: 0 - ETA: 1s - loss: 0.0465 - acc: - ETA: 1s - loss: 0.0466  - ETA: 0s - loss: 0.0469 - acc:
Epoch 5/5
12595/12595 [==============================] - 6s 453us/sample - loss: 0.0343 - acc: 0.9887
2019-12-27 07:56:53.120459 End of fit
acc: 98.01%
2019-12-27 07:56:54.968389 Start of fit
Epoch 1/5
12597/12597 [==============================] - 6s 514us/sample - loss: 0.4455 - acc: 0.8612s - l
Epoch 2/5
12597/12597 [==============================] - 6s 452us/sample - loss: 0.1183 - acc: 0.9646
Epoch 3/5
12597/12597 [==============================] - 6s 449us/sample - loss: 0.0711 - acc: 0.9767s - loss: 0.0669 - - ETA: 2s - l - ETA: 0s - loss: 0.0710 - acc: 0 - ETA: 0s - loss: 0.0708 - acc: 0.
Epoch 4/5
12597/12597 [==============================] - 6s 455us/sa

In [19]:
Config.SIMILARITY_VECTOR_TYPE=0

Config.NUM_KERAS_TRAIN_IMAGES=24_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=4_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-27 09:06:53.655032 ('Start of CNN model with similarity_vector TYPE=', 0)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(24000, 28, 28) (24000,)
image_label,image_no_label 24000 24000
The range (min num reps of a digit, max num reps of a digit):
2144 2732
2019-12-27 09:06:54.411877 Start get_similarity_vectors() processing
2019-12-27 09:07:05.038844 ('we have processed ', 999, ' images')
2019-12-27 09:07:15.714894 ('we have processed ', 1999, ' images')
2019-12-27 09:07:26.999759 ('we have processed ', 2999, ' images')
2019-12-27 09:07:38.463287 ('we have processed ', 3999, ' images')
2019-12-27 09:07:50.752402 ('we have processed ', 4999, ' images')
2019-12-27 09:08:03.709609 ('we have processed ', 5999, ' images')
2019-12-27 09:08:17.049025 ('we have processed ', 6999, ' images')
2019-12-27 09:08:30.820594 ('we have processed ', 7999, ' images')
2019-12-27 09:08:45.526375 ('we have processed ', 8999, ' images')
2019-12-27 09:09:00.821283 ('we have processed ', 9999, ' image

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-27 09:11:03.315239 ('we have processed ', 16999, ' images')
2019-12-27 09:11:22.559482 ('we have processed ', 17999, ' images')
2019-12-27 09:11:42.268956 ('we have processed ', 18999, ' images')
2019-12-27 09:12:02.884221 ('we have processed ', 19999, ' images')
2019-12-27 09:12:23.740358 ('we have processed ', 20999, ' images')
2019-12-27 09:12:45.107418 ('we have processed ', 21999, ' images')
2019-12-27 09:13:07.041587 ('we have processed ', 22999, ' images')
2019-12-27 09:13:29.138179 ('we have processed ', 23999, ' images')
2019-12-27 09:13:29.166672 End get_similarity_vectors() processing
The training accuracy is:  0.9537916666666667
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-27 09:15:20.012908 Start get_similarity_vectors() processing
2019-12-27 09:15:30.252874 ('we have processed ', 999, ' images')
2019-12-27 09:15:40.546432 ('we have processed ', 1999, ' images')
2019-12-27 09:15:51.239700 ('we have processed ', 2999, ' images')
2019

25203/25203 [==============================] - 13s 516us/sample - loss: 0.0362 - acc: 0.9887 - loss: 0.0357 - acc: 0.98 - ETA: 0s - loss: 0.0364 -
Epoch 5/5
25203/25203 [==============================] - 13s 516us/sample - loss: 0.0258 - acc: 0.9918 - loss: 0.0225 - acc: 0.99 - ETA: 7s - loss: 0.0231 - a - ETA: 6s - l  - ETA: 0s - loss: 0.0257 - acc: 0.991
2019-12-27 09:25:31.355218 End of fit
acc: 98.39%
2019-12-27 09:25:36.411198 Start of fit
Epoch 1/5
25203/25203 [==============================] - 15s 576us/sample - loss: 0.2863 - acc: 0.9095 - loss - ETA: 4s - loss: 0.36 - ETA: 3s - loss - ETA: 1s - loss: 0.3 - ETA: 0s - loss: 0.2906 - acc: 0. - ETA: 0s - loss: 0.2868 - acc: 0.90
Epoch 2/5
25203/25203 [==============================] - 13s 518us/sample - loss: 0.0727 - acc: 0.9777 4s - loss: 0.0784 - acc: 0 - ETA: 3s - loss: 0.0788 - ETA: 3s - loss: 0.0778 - ETA: 2s - loss: 0.0773 - acc: 0.976 - ETA: 2s - loss: 0.0774 - acc - ETA: 1s - l
Epoch 3/5
25203/25203 [=====================

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


...

2019-12-27 10:19:20.974860 ('we have processed ', 16999, ' images')
2019-12-27 10:19:39.718140 ('we have processed ', 17999, ' images')
2019-12-27 10:19:59.164583 ('we have processed ', 18999, ' images')
2019-12-27 10:20:19.519192 ('we have processed ', 19999, ' images')
2019-12-27 10:20:40.038085 ('we have processed ', 20999, ' images')
2019-12-27 10:21:01.309819 ('we have processed ', 21999, ' images')
2019-12-27 10:21:23.215002 ('we have processed ', 22999, ' images')
2019-12-27 10:21:45.424568 ('we have processed ', 23999, ' images')
2019-12-27 10:21:45.440169 End get_similarity_vectors() processing
The training accuracy is:  0.9537916666666667
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-27 10:23:35.336767 Start get_similarity_vectors() processing
2019-12-27 10:23:45.670128 ('we have processed ', 999, ' images')
2019-12-27 10:23:55.719840 ('we have processed ', 1999, ' images')
2019-12-27 10:24:06.348780 ('we have processed ', 2999, ' images')
2019

25204/25204 [==============================] - 17s 656us/sample - loss: 0.2685 - acc: 0.9144A: 2s - loss: 0.2984 - acc: 0.9 - ETA: 2s 
Epoch 2/5
25204/25204 [==============================] - 14s 554us/sample - loss: 0.0693 - acc: 0.9786s - loss: 0. - ETA: 6s - loss: 
Epoch 3/5
25204/25204 [==============================] - 14s 561us/sample - loss: 0.0448 - acc: 0.9862 - loss: 0.0487 - ac - ETA: 8s - loss: 0 - ETA: 7s - ETA: 5s - loss: 0. - ETA: 4s - loss: 0.0453 - acc - ETA: 3s - loss: 0.
Epoch 4/5
25204/25204 [==============================] - 14s 571us/sample - loss: 0.0325 - acc: 0.9899 - l
Epoch 5/5
25204/25204 [==============================] - 14s 572us/sample - loss: 0.0243 - acc: 0.9924
2019-12-27 10:37:16.947598 End of fit
acc: 98.71%
2019-12-27 10:37:21.447932 Start of fit
Epoch 1/5
25204/25204 [==============================] - 16s 635us/sample - loss: 0.2776 - acc: 0.9130
Epoch 2/5
25204/25204 [==============================] - 14s 560us/sample - loss: 0.0710 - acc: 0.9785

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-27 10:42:59.074003 ('we have processed ', 16999, ' images')
2019-12-27 10:43:18.028683 ('we have processed ', 17999, ' images')
2019-12-27 10:43:37.376225 ('we have processed ', 18999, ' images')
2019-12-27 10:43:57.925245 ('we have processed ', 19999, ' images')
2019-12-27 10:44:18.744319 ('we have processed ', 20999, ' images')
2019-12-27 10:44:40.240974 ('we have processed ', 21999, ' images')
2019-12-27 10:45:02.414269 ('we have processed ', 22999, ' images')
2019-12-27 10:45:24.912433 ('we have processed ', 23999, ' images')
2019-12-27 10:45:24.928018 End get_similarity_vectors() processing
The training accuracy is:  0.9537916666666667
The range (min num reps of a digit, max num reps of a digit):
370 450
2019-12-27 10:47:15.314258 Start get_similarity_vectors() processing
2019-12-27 10:47:25.990038 ('we have processed ', 999, ' images')
2019-12-27 10:47:36.402361 ('we have processed ', 1999, ' images')
2019-12-27 10:47:47.401094 ('we have processed ', 2999, ' images')
2019

25203/25203 [==============================] - 15s 580us/sample - loss: 0.0463 - acc: 0.9860 - 
Epoch 4/5
25203/25203 [==============================] - 14s 567us/sample - loss: 0.0307 - acc: 0.9900 - loss: 0.0308 - acc: 0.9 - ETA: 0s - loss: 0.0308 - acc: 0.9
Epoch 5/5
25203/25203 [==============================] - 15s 596us/sample - loss: 0.0256 - acc: 0.9920
2019-12-27 11:00:10.413493 End of fit
acc: 98.82%
2019-12-27 11:00:15.590294 Start of fit
Epoch 1/5
25204/25204 [==============================] - 17s 660us/sample - loss: 0.2862 - acc: 0.9099 - loss: 0. - ETA: 0s - loss: 0.2904 - acc: 0
Epoch 2/5
25204/25204 [==============================] - 14s 566us/sample - loss: 0.0752 - acc: 0.9758 - loss: 0.0832 - acc: 0. - ETA: 8s - loss: 0.0838 - ETA: 7s - ETA: 3s - loss: 0.0774 - acc: 0.975 - ETA: 3s - loss: - ETA: 1s - loss: 0.0768 - ETA: 0s - loss: 0.0767 - a
Epoch 3/5
25204/25204 [==============================] - 14s 569us/sample - loss: 0.0484 - acc: 0.9851: 0s - loss: 0.0484 - a

# comparison summary of accuracy of 5 reps of 10-fold cross-validation

### similarity vector type is 1 unless otherwise stated

12_000 
- 97.78% (+/- 0.54%) verbatim_from_book_CNN
- 97.89% (+/- 0.43%) test_CNN_with_similarity_vector
- 98.13% (+/- 0.49%) random numbers appended to the image
- 97.98% (+/- 0.54%) similarity vector and random numbers appended
- 97.94% (+/- 0.58%) random numbers equal in size to type=-3 appended to the image
- 98.09% (+/- 0.54%) similarity vector type 0


24_000
- 98.55% (+/- 0.25%) verbatim_from_book_CNN
- 98.47% (+/- 0.41%) test_CNN_with_similarity_vector
- 98.60% (+/- 0.23%) random numbers appended to the image
- 98.56% (+/- 0.36%) similarity vector and random numbers appended
- 98.58% (+/- 0.29%) random numbers equal in size to type=-3 appended to the image
- 98.57% (+/- 0.24%) similarity vector type 0

36_000 
- 98.78% (+/- 0.26%) verbatim_from_book_CNN
- 98.78% (+/- 0.30%) test_CNN_with_similarity_vector
- 98.80% (+/- 0.26%) random numbers appended to the image
- 98.85% (+/- 0.22%) similarity vector and random numbers appended
- 98.79% (+/- 0.26%) random numbers equal in size to type=-3 appended to the image
- similarity vector type 0

48_000
- 98.92% (+/- 0.16%) verbatim_from_book_CNN
- 98.95% (+/- 0.20%) test_CNN_with_similarity_vector
- 98.99% (+/- 0.12%) random numbers appended to the image
- 98.92% (+/- 0.18%) similarity vector and random numbers appended
- 98.98% (+/- 0.14%) random numbers equal in size to type=-3 appended to the image
- similarity vector type 0

60_000
- 98.98% (+/- 0.20%) verbatim_from_book_CNN
- 99.01% (+/- 0.18%) test_CNN_with_similarity_vector
- 99.05% (+/- 0.14%) random numbers appended to the image
- 99.04% (+/- 0.16%) similarity vector and random numbers appended
- random numbers equal in size to type=-3 appended to the image
- similarity vector type 0


In [ ]:
Config.SIMILARITY_VECTOR_TYPE=0

Config.NUM_KERAS_TRAIN_IMAGES=36_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=6_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-27 11:03:12.403893 ('Start of CNN model with similarity_vector TYPE=', 0)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(36000, 28, 28) (36000,)
image_label,image_no_label 36000 36000
The range (min num reps of a digit, max num reps of a digit):
3230 4113
2019-12-27 11:03:13.474457 Start get_similarity_vectors() processing
2019-12-27 11:03:24.240738 ('we have processed ', 999, ' images')
2019-12-27 11:03:34.966843 ('we have processed ', 1999, ' images')
2019-12-27 11:03:46.142720 ('we have processed ', 2999, ' images')
2019-12-27 11:03:57.649491 ('we have processed ', 3999, ' images')
2019-12-27 11:04:10.005664 ('we have processed ', 4999, ' images')
2019-12-27 11:04:23.031505 ('we have processed ', 5999, ' images')
2019-12-27 11:04:36.368174 ('we have processed ', 6999, ' images')
2019-12-27 11:04:50.058274 ('we have processed ', 7999, ' images')
2019-12-27 11:05:04.975270 ('we have processed ', 8999, ' images')
2019-12-27 11:05:20.395850 ('we have processed ', 9999, ' image

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-27 11:07:22.623654 ('we have processed ', 16999, ' images')
2019-12-27 11:07:41.717537 ('we have processed ', 17999, ' images')
2019-12-27 11:08:01.276368 ('we have processed ', 18999, ' images')
2019-12-27 11:08:21.817250 ('we have processed ', 19999, ' images')
2019-12-27 11:08:42.610393 ('we have processed ', 20999, ' images')
2019-12-27 11:09:04.203109 ('we have processed ', 21999, ' images')
2019-12-27 11:09:26.323275 ('we have processed ', 22999, ' images')
2019-12-27 11:09:48.854770 ('we have processed ', 23999, ' images')
2019-12-27 11:10:11.664943 ('we have processed ', 24999, ' images')
2019-12-27 11:10:35.134654 ('we have processed ', 25999, ' images')
2019-12-27 11:10:58.864666 ('we have processed ', 26999, ' images')
2019-12-27 11:11:23.256805 ('we have processed ', 27999, ' images')
2019-12-27 11:11:48.507857 ('we have processed ', 28999, ' images')
2019-12-27 11:12:13.791012 ('we have processed ', 29999, ' images')
2019-12-27 11:12:39.494842 ('we have processed '

37802/37802 [==============================] - 21s 562us/sample - loss: 0.0568 - acc: 0.9823 - loss: 0.0583 - acc: 0.9 - ETA: 2s - loss: 0.0583 - acc: 0. -
Epoch 3/5
37802/37802 [==============================] - 22s 573us/sample - loss: 0.0368 - acc: 0.9885A: 4s - loss: 0.0367 - acc: 0.9 - ETA: 
Epoch 4/5
37802/37802 [==============================] - 21s 557us/sample - loss: 0.0278 - acc: 0.9910s -  - ETA: 11s - loss: 0.0275 - acc - ETA: 10s - loss: 0.02 - ETA: 4s - loss: 0.0276 -  - ETA: 3s - loss: 0.0
Epoch 5/5
37802/37802 [==============================] - 21s 544us/sample - loss: 0.0204 - acc: 0.9933 - loss: 0
2019-12-27 11:38:13.429655 End of fit
acc: 99.31%
2019-12-27 11:38:18.888863 Start of fit
Epoch 1/5
37803/37803 [==============================] - 24s 636us/sample - loss: 0.2204 - acc: 0.9313s - loss: 0 - ETA: 8s - loss: 0.2852  - ETA: 7s -  - ETA: 5s - loss: - ETA: 3s - loss: 0.2451 - ac
Epoch 2/5
37803/37803 [==============================] - 22s 573us/sample - loss: 0.0

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


...

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-27 12:26:36.943771 ('we have processed ', 16999, ' images')
2019-12-27 12:26:56.457582 ('we have processed ', 17999, ' images')
2019-12-27 12:27:16.372874 ('we have processed ', 18999, ' images')
2019-12-27 12:27:37.481907 ('we have processed ', 19999, ' images')
2019-12-27 12:27:58.858968 ('we have processed ', 20999, ' images')
2019-12-27 12:28:21.011673 ('we have processed ', 21999, ' images')
2019-12-27 12:28:43.640948 ('we have processed ', 22999, ' images')
2019-12-27 12:29:06.689689 ('we have processed ', 23999, ' images')
2019-12-27 12:29:29.742718 ('we have processed ', 24999, ' images')
2019-12-27 12:29:53.721436 ('we have processed ', 25999, ' images')
2019-12-27 12:30:17.963014 ('we have processed ', 26999, ' images')
2019-12-27 12:30:42.797141 ('we have processed ', 27999, ' images')
2019-12-27 12:31:08.607047 ('we have processed ', 28999, ' images')
2019-12-27 12:31:34.491229 ('we have processed ', 29999, ' images')
2019-12-27 12:32:00.705627 ('we have processed '

37803/37803 [==============================] - 22s 591us/sample - loss: 0.0543 - acc: 0.9823  - ETA: 0s - loss: 0.0552 
Epoch 3/5
37803/37803 [==============================] - 22s 575us/sample - loss: 0.0360 - acc: 0.9886 - loss: 0.0 
Epoch 4/5
37803/37803 [==============================] - 21s 568us/sample - loss: 0.0267 - acc: 0.9913
Epoch 5/5
37803/37803 [==============================] - 21s 565us/sample - loss: 0.0188 - acc: 0.9939
2019-12-27 13:00:49.761066 End of fit
acc: 98.86%
2019-12-27 13:00:56.043481 Start of fit
Epoch 1/5
37806/37806 [==============================] - 24s 645us/sample - loss: 0.2138 - acc: 0.9331
Epoch 2/5
37806/37806 [==============================] - 22s 577us/sample - loss: 0.0544 - acc: 0.9827
Epoch 3/5
37806/37806 [==============================] - 22s 569us/sample - loss: 0.0366 - acc: 0.9885
Epoch 4/5
37806/37806 [==============================] - 22s 571us/sample - loss: 0.0265 - acc: 0.9919
Epoch 5/5
37806/37806 [==============================] -

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-27 13:07:22.091130 ('we have processed ', 16999, ' images')
2019-12-27 13:07:41.712426 ('we have processed ', 17999, ' images')
2019-12-27 13:08:02.169811 ('we have processed ', 18999, ' images')
2019-12-27 13:08:23.837039 ('we have processed ', 19999, ' images')
2019-12-27 13:08:45.602424 ('we have processed ', 20999, ' images')
2019-12-27 13:09:07.975696 ('we have processed ', 21999, ' images')
2019-12-27 13:09:30.648270 ('we have processed ', 22999, ' images')
2019-12-27 13:09:53.549338 ('we have processed ', 23999, ' images')
2019-12-27 13:10:16.891262 ('we have processed ', 24999, ' images')
2019-12-27 13:10:40.737696 ('we have processed ', 25999, ' images')


In [ ]:
Config.SIMILARITY_VECTOR_TYPE=0

Config.NUM_KERAS_TRAIN_IMAGES=48_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=8_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

In [ ]:
Config.SIMILARITY_VECTOR_TYPE=0

Config.NUM_KERAS_TRAIN_IMAGES=60_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=10_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

In [18]:
Config.SIMILARITY_VECTOR_TYPE=2

Config.NUM_KERAS_TRAIN_IMAGES=12_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES
Config.NUM_KERAS_TEST_IMAGES=2_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

all_cvscores=[]
for _ in range(5):
    cvscores=test_CNN_with_similarity_vector()
    all_cvscores.append(cvscores)
print("%.2f%% (+/- %.2f%%)" % (np.mean(all_cvscores), np.std(all_cvscores)))
print('all_cvscores:',all_cvscores)

Config.SIMILARITY_VECTOR_TYPE=1

2019-12-27 08:02:15.250924 ('Start of CNN model with similarity_vector TYPE=', 2)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(12000, 28, 28) (12000,)
image_label,image_no_label 12000 12000
The range (min num reps of a digit, max num reps of a digit):
1048 1351
2019-12-27 08:02:15.843576 Start get_similarity_vectors() processing
2019-12-27 08:02:26.048038 ('we have processed ', 999, ' images')
2019-12-27 08:02:36.741217 ('we have processed ', 1999, ' images')
2019-12-27 08:02:47.115588 ('we have processed ', 2999, ' images')
2019-12-27 08:02:57.523703 ('we have processed ', 3999, ' images')
2019-12-27 08:03:08.444630 ('we have processed ', 4999, ' images')
2019-12-27 08:03:19.830415 ('we have processed ', 5999, ' images')
2019-12-27 08:03:31.277749 ('we have processed ', 6999, ' images')
2019-12-27 08:03:42.554272 ('we have processed ', 7999, ' images')
2019-12-27 08:03:54.652768 ('we have processed ', 8999, ' images')
2019-12-27 08:04:06.767738 ('we have processed ', 9999, ' image

12604/12604 [==============================] - 6s 480us/sample - loss: 0.1168 - acc: 0.9645
Epoch 3/5
12604/12604 [==============================] - 6s 490us/sample - loss: 0.0696 - acc: 0.9797s - loss: 0.0721 - acc: - ETA: 0s - loss: 0.0717 - acc: - ETA: 0s - loss: 0.0701 - acc: 0
Epoch 4/5
12604/12604 [==============================] - 6s 506us/sample - loss: 0.0494 - acc: 0.9849
Epoch 5/5
12604/12604 [==============================] - 6s 512us/sample - loss: 0.0358 - acc: 0.9902
2019-12-27 08:10:54.831246 End of fit
acc: 97.64%
2019-12-27 08:10:57.103238 Start of fit
Epoch 1/5
12605/12605 [==============================] - 7s 559us/sample - loss: 0.4686 - acc: 0.8526s - loss: 0.7179 - - ETA: 2s - loss: 0.6223 - acc: 0.802 - ETA
Epoch 2/5
12605/12605 [==============================] - 6s 489us/sample - loss: 0.1217 - acc: 0.9627s - loss: 0.1410 - - ETA: 3s - loss: 0.1403 - acc: 0.9 - ETA: 3s - loss: 0.1376 - - ETA: 2s - loss: 0. - ETA: 1s - loss: 0.1289 - acc: 0.960 - ETA: 1s - loss 

12598/12598 [==============================] - 6s 494us/sample - loss: 0.0354 - acc: 0.9892s - loss: 0.0361 -  - ETA: 1s
2019-12-27 08:17:56.135438 End of fit
acc: 98.07%
2019-12-27 08:17:58.463089 Start of fit
Epoch 1/5
12601/12601 [==============================] - 7s 566us/sample - loss: 0.4508 - acc: 0.8541
Epoch 2/5
12601/12601 [==============================] - 6s 484us/sample - loss: 0.1165 - acc: 0.9625s - loss: 0.1 - ETA: 4s - loss: 0.1268  - ETA: 3s - loss: 0 - ETA: 2s - loss: 0.1267 - acc: 0 - ETA: 2s - loss: 0.1267 - acc: 0. - ET
Epoch 3/5
12601/12601 [==============================] - 6s 485us/sample - loss: 0.0714 - acc: 0.9778s - loss: 0.091 - ETA: 4s - loss: - ETA: 3s - loss: 0. - ETA: 2s - lo - ETA: 0s - loss: 0.0719 - acc: 0.977 - ETA: 0s - loss: 0.0715 - acc: 0.977 - ETA: 0s - loss: 0.0720 - ac
Epoch 4/5
12601/12601 [==============================] - 6s 479us/sample - loss: 0.0509 - acc: 0.9833s - loss: 0.0 - ETA: 0s - loss: 0.0533 -
Epoch 5/5
12601/12601 [==========

12597/12597 [==============================] - 6s 492us/sample - loss: 0.0513 - acc: 0.9834
Epoch 5/5
12597/12597 [==============================] - 6s 476us/sample - loss: 0.0380 - acc: 0.9888s - loss: 0.0371
2019-12-27 08:25:36.542812 End of fit
acc: 97.72%
2019-12-27 08:25:39.023645 Start of fit
Epoch 1/5
12597/12597 [==============================] - 7s 566us/sample - loss: 0.4380 - acc: 0.8612
Epoch 2/5
12597/12597 [==============================] - ETA: 0s - loss: 0.1200 - acc: 0.9643- ET - ET - 6s 481us/sample - loss: 0.1195 - acc: 0.9644
Epoch 3/5
12597/12597 [==============================] - 6s 480us/sample - loss: 0.0744 - acc: 0.9762s - loss:  - ETA: 3s - loss: 0.0766 - acc: 0.9 - ETA: 3s - los - ETA
Epoch 4/5
12597/12597 [==============================] - 6s 482us/sample - loss: 0.0526 - acc: 0.9828s - loss: - ETA: 2s - loss: 0. - ETA: 0s - loss: 0.0526
Epoch 5/5
12597/12597 [==============================] - 6s 480us/sample - loss: 0.0388 - acc: 0.9883
2019-12-27 08:26:15

2019-12-27 08:31:49.558226 ('we have processed ', 4999, ' images')
2019-12-27 08:32:00.872832 ('we have processed ', 5999, ' images')
2019-12-27 08:32:12.218629 ('we have processed ', 6999, ' images')
2019-12-27 08:32:23.443076 ('we have processed ', 7999, ' images')
2019-12-27 08:32:35.489752 ('we have processed ', 8999, ' images')
2019-12-27 08:32:47.751893 ('we have processed ', 9999, ' images')
2019-12-27 08:33:00.560902 ('we have processed ', 10999, ' images')
2019-12-27 08:33:13.450757 ('we have processed ', 11999, ' images')
2019-12-27 08:33:13.466378 End get_similarity_vectors() processing
The training accuracy is:  0.9414166666666667
The range (min num reps of a digit, max num reps of a digit):
175 234
2019-12-27 08:33:36.691569 Start get_similarity_vectors() processing
2019-12-27 08:33:47.014881 ('we have processed ', 999, ' images')
2019-12-27 08:33:57.549536 ('we have processed ', 1999, ' images')
2019-12-27 08:33:57.565156 End get_similarity_vectors() processing
The testin

12604/12604 [==============================] - 6s 493us/sample - loss: 0.0506 - acc: 0.9838s - loss: 0.0539 - acc:  - ETA: 5s - loss: 0 - ETA: 1s - loss: 0. - ETA: 0s - loss: 0.0499 - acc: 0.98 - ETA: 0s - loss: 0.0499 - acc: 0.
Epoch 5/5
12604/12604 [==============================] - 6s 487us/sample - loss: 0.0376 - acc: 0.9878s - l - ETA: 4s - loss: 0.0343 - - ETA: 3s - loss: 0. - ETA: 2s - loss: 0.0384 - a - ETA: 1s - loss: 0.037 - ETA: 0s - loss: 0.0370 -  - ETA: 0s - loss: 0.0370 - acc: 0.9
2019-12-27 08:39:25.193938 End of fit
acc: 97.71%
2019-12-27 08:39:28.018609 Start of fit
Epoch 1/5
12604/12604 [==============================] - 7s 591us/sample - loss: 0.4613 - acc: 0.8546
Epoch 2/5
12604/12604 [==============================] - 6s 491us/sample - loss: 0.1195 - acc: 0.9641
Epoch 3/5
12604/12604 [==============================] - 6s 490us/sample - loss: 0.0753 - acc: 0.9772s - loss: 0.0831 - acc: 0 - ETA: 3s - loss: 0.0824 - acc: 0. - ETA: 0s - loss: 0.0761 - acc: - ETA: 0s -

acc: 98.00%
2019-12-27 08:46:43.689771 Start of fit
Epoch 1/5
12598/12598 [==============================] - 8s 598us/sample - loss: 0.4806 - acc: 0.8470s - l - ETA: 2s - loss: 0.6046 - a - ETA: 1s - loss: 
Epoch 2/5
12598/12598 [==============================] - 6s 492us/sample - loss: 0.1275 - acc: 0.9593s - loss: - ETA: 1s - loss: 0.131 - ETA: 0s - loss: 0.1278 - acc: 0.
Epoch 3/5
12598/12598 [==============================] - 6s 496us/sample - loss: 0.0822 - acc: 0.9758s - loss: 0.0 - ETA: 2s - loss: 0.0807 - acc - ETA: 2s - loss: 0.0841 - acc: 0.975 
Epoch 4/5
12598/12598 [==============================] - 6s 496us/sample - loss: 0.0568 - acc: 0.9817
Epoch 5/5
12598/12598 [==============================] - 6s 505us/sample - loss: 0.0414 - acc: 0.9867ETA: 4s - los
2019-12-27 08:47:22.640278 End of fit
acc: 97.50%
2019-12-27 08:47:25.664891 Start of fit
Epoch 1/5
12601/12601 [==============================] - 8s 610us/sample - loss: 0.4453 - acc: 0.8584s - loss: 0.6508 - acc: 0 - ET

# Some Config choice tuning: NUM_SPHERES aka donuts or shells

In [19]:
#     elif Config.SIMILARITY_VECTOR_TYPE==1:
#         return similarity_vector
Config.SIMILARITY_VECTOR_TYPE=1
Config.NUM_SPHERES=10

Config.NUM_KERAS_TRAIN_IMAGES=60_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES

Config.NUM_KERAS_TEST_IMAGES=10_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

for _ in range(3):
    test_CNN_with_similarity_vector()

2019-12-21 06:05:49.373857 ('Start of CNN model with similarity_vector TYPE=', 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(60000, 28, 28) (60000,)
image_label,image_no_label 60000 60000
The range (min num reps of a digit, max num reps of a digit):
5421 6742
2019-12-21 06:05:49.664289 Start get_similarity_vectors() processing
2019-12-21 06:05:59.814109 ('we have processed ', 999, ' images')
2019-12-21 06:06:09.720527 ('we have processed ', 1999, ' images')
2019-12-21 06:06:19.850344 ('we have processed ', 2999, ' images')
2019-12-21 06:06:29.781841 ('we have processed ', 3999, ' images')
2019-12-21 06:06:39.985378 ('we have processed ', 4999, ' images')
2019-12-21 06:06:50.442977 ('we have processed ', 5999, ' images')
2019-12-21 06:07:00.593918 ('we have processed ', 6999, ' images')
2019-12-21 06:07:10.447121 ('we have processed ', 7999, ' images')
2019-12-21 06:07:21.283871 ('we have processed ', 8999, ' images')
2019-12-21 06:07:31.376651 ('we have processed ', 9999, ' image

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-21 06:08:44.481838 ('we have processed ', 16999, ' images')
2019-12-21 06:08:54.869093 ('we have processed ', 17999, ' images')
2019-12-21 06:09:05.578084 ('we have processed ', 18999, ' images')
2019-12-21 06:09:16.652687 ('we have processed ', 19999, ' images')
2019-12-21 06:09:27.409086 ('we have processed ', 20999, ' images')
2019-12-21 06:09:38.376113 ('we have processed ', 21999, ' images')
2019-12-21 06:09:49.423758 ('we have processed ', 22999, ' images')
2019-12-21 06:10:00.440183 ('we have processed ', 23999, ' images')
2019-12-21 06:10:11.412404 ('we have processed ', 24999, ' images')
2019-12-21 06:10:22.378165 ('we have processed ', 25999, ' images')
2019-12-21 06:10:33.401426 ('we have processed ', 26999, ' images')
2019-12-21 06:10:44.531960 ('we have processed ', 27999, ' images')
2019-12-21 06:10:56.023386 ('we have processed ', 28999, ' images')
2019-12-21 06:11:07.017853 ('we have processed ', 29999, ' images')
2019-12-21 06:11:18.110791 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-21 06:43:05.749653 ('we have processed ', 16999, ' images')
2019-12-21 06:43:16.219384 ('we have processed ', 17999, ' images')
2019-12-21 06:43:26.670180 ('we have processed ', 18999, ' images')
2019-12-21 06:43:37.755549 ('we have processed ', 19999, ' images')
2019-12-21 06:43:48.540626 ('we have processed ', 20999, ' images')
2019-12-21 06:43:59.377397 ('we have processed ', 21999, ' images')
2019-12-21 06:44:10.418298 ('we have processed ', 22999, ' images')
2019-12-21 06:44:21.411347 ('we have processed ', 23999, ' images')
2019-12-21 06:44:32.395279 ('we have processed ', 24999, ' images')
2019-12-21 06:44:43.379023 ('we have processed ', 25999, ' images')
2019-12-21 06:44:54.296197 ('we have processed ', 26999, ' images')
2019-12-21 06:45:05.478177 ('we have processed ', 27999, ' images')
2019-12-21 06:45:17.034450 ('we have processed ', 28999, ' images')
2019-12-21 06:45:28.163379 ('we have processed ', 29999, ' images')
2019-12-21 06:45:39.533664 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-21 07:17:40.596317 ('we have processed ', 16999, ' images')
2019-12-21 07:17:51.078721 ('we have processed ', 17999, ' images')
2019-12-21 07:18:01.798245 ('we have processed ', 18999, ' images')
2019-12-21 07:18:12.948918 ('we have processed ', 19999, ' images')
2019-12-21 07:18:23.875840 ('we have processed ', 20999, ' images')
2019-12-21 07:18:34.827077 ('we have processed ', 21999, ' images')
2019-12-21 07:18:45.975379 ('we have processed ', 22999, ' images')
2019-12-21 07:18:57.095073 ('we have processed ', 23999, ' images')
2019-12-21 07:19:08.081136 ('we have processed ', 24999, ' images')
2019-12-21 07:19:19.191043 ('we have processed ', 25999, ' images')
2019-12-21 07:19:30.227911 ('we have processed ', 26999, ' images')
2019-12-21 07:19:41.478947 ('we have processed ', 27999, ' images')
2019-12-21 07:19:53.179939 ('we have processed ', 28999, ' images')
2019-12-21 07:20:04.441724 ('we have processed ', 29999, ' images')
2019-12-21 07:20:15.691454 ('we have processed '

In [20]:
#     elif Config.SIMILARITY_VECTOR_TYPE==1:
#         return similarity_vector
Config.SIMILARITY_VECTOR_TYPE=1
Config.NUM_SPHERES=16

Config.NUM_KERAS_TRAIN_IMAGES=60_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES

Config.NUM_KERAS_TEST_IMAGES=10_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

for _ in range(3):
    test_CNN_with_similarity_vector()

2019-12-21 08:24:55.543284 ('Start of CNN model with similarity_vector TYPE=', 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(60000, 28, 28) (60000,)
image_label,image_no_label 60000 60000
The range (min num reps of a digit, max num reps of a digit):
5421 6742
2019-12-21 08:24:55.877184 Start get_similarity_vectors() processing
2019-12-21 08:25:06.440410 ('we have processed ', 999, ' images')
2019-12-21 08:25:16.833962 ('we have processed ', 1999, ' images')
2019-12-21 08:25:27.541936 ('we have processed ', 2999, ' images')
2019-12-21 08:25:37.970733 ('we have processed ', 3999, ' images')
2019-12-21 08:25:48.730582 ('we have processed ', 4999, ' images')
2019-12-21 08:25:59.501134 ('we have processed ', 5999, ' images')
2019-12-21 08:26:10.242445 ('we have processed ', 6999, ' images')
2019-12-21 08:26:20.590156 ('we have processed ', 7999, ' images')
2019-12-21 08:26:31.272865 ('we have processed ', 8999, ' images')
2019-12-21 08:26:42.009235 ('we have processed ', 9999, ' image

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-21 08:27:59.675672 ('we have processed ', 16999, ' images')
2019-12-21 08:28:10.849842 ('we have processed ', 17999, ' images')
2019-12-21 08:28:22.168334 ('we have processed ', 18999, ' images')
2019-12-21 08:28:34.132439 ('we have processed ', 19999, ' images')
2019-12-21 08:28:45.924016 ('we have processed ', 20999, ' images')
2019-12-21 08:28:57.898460 ('we have processed ', 21999, ' images')
2019-12-21 08:29:09.952844 ('we have processed ', 22999, ' images')
2019-12-21 08:29:21.751336 ('we have processed ', 23999, ' images')
2019-12-21 08:29:33.500859 ('we have processed ', 24999, ' images')
2019-12-21 08:29:45.395203 ('we have processed ', 25999, ' images')
2019-12-21 08:29:57.160940 ('we have processed ', 26999, ' images')
2019-12-21 08:30:09.290305 ('we have processed ', 27999, ' images')
2019-12-21 08:30:21.973683 ('we have processed ', 28999, ' images')
2019-12-21 08:30:34.201307 ('we have processed ', 29999, ' images')
2019-12-21 08:30:46.293511 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-21 09:05:48.937370 ('we have processed ', 16999, ' images')
2019-12-21 09:06:00.208915 ('we have processed ', 17999, ' images')
2019-12-21 09:06:11.540493 ('we have processed ', 18999, ' images')
2019-12-21 09:06:23.471196 ('we have processed ', 19999, ' images')
2019-12-21 09:06:35.322173 ('we have processed ', 20999, ' images')
2019-12-21 09:06:47.174117 ('we have processed ', 21999, ' images')
2019-12-21 09:06:59.238313 ('we have processed ', 22999, ' images')
2019-12-21 09:07:11.288212 ('we have processed ', 23999, ' images')
2019-12-21 09:07:23.257457 ('we have processed ', 24999, ' images')
2019-12-21 09:07:35.347941 ('we have processed ', 25999, ' images')
2019-12-21 09:07:47.382989 ('we have processed ', 26999, ' images')
2019-12-21 09:07:59.653175 ('we have processed ', 27999, ' images')
2019-12-21 09:08:12.324701 ('we have processed ', 28999, ' images')
2019-12-21 09:08:24.671664 ('we have processed ', 29999, ' images')
2019-12-21 09:08:36.880530 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-21 09:44:28.924613 ('we have processed ', 16999, ' images')
2019-12-21 09:44:40.265891 ('we have processed ', 17999, ' images')
2019-12-21 09:44:51.689638 ('we have processed ', 18999, ' images')
2019-12-21 09:45:03.746328 ('we have processed ', 19999, ' images')
2019-12-21 09:45:15.561749 ('we have processed ', 20999, ' images')
2019-12-21 09:45:27.595455 ('we have processed ', 21999, ' images')
2019-12-21 09:45:39.611459 ('we have processed ', 22999, ' images')
2019-12-21 09:45:51.579766 ('we have processed ', 23999, ' images')
2019-12-21 09:46:03.799476 ('we have processed ', 24999, ' images')
2019-12-21 09:46:15.924791 ('we have processed ', 25999, ' images')
2019-12-21 09:46:27.941740 ('we have processed ', 26999, ' images')
2019-12-21 09:46:40.254597 ('we have processed ', 27999, ' images')
2019-12-21 09:46:53.082848 ('we have processed ', 28999, ' images')
2019-12-21 09:47:05.584727 ('we have processed ', 29999, ' images')
2019-12-21 09:47:17.898476 ('we have processed '

In [18]:
#     elif Config.SIMILARITY_VECTOR_TYPE==1:
#         return similarity_vector
Config.SIMILARITY_VECTOR_TYPE=1
Config.NUM_SPHERES=6

Config.NUM_KERAS_TRAIN_IMAGES=60_000
Config.NUM_KERAS_TRAIN_LABELS=Config.NUM_KERAS_TRAIN_IMAGES

Config.NUM_KERAS_TEST_IMAGES=10_000
Config.NUM_KERAS_TEST_LABELS=Config.NUM_KERAS_TEST_IMAGES

for _ in range(3):
#     test_CNN_with_similarity_vector()

2019-12-21 04:25:26.161926 ('Start of CNN model with similarity_vector TYPE=', 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(60000, 28, 28) (60000,)
image_label,image_no_label 60000 60000
The range (min num reps of a digit, max num reps of a digit):
5421 6742
2019-12-21 04:25:26.783742 Start get_similarity_vectors() processing
2019-12-21 04:25:36.768766 ('we have processed ', 999, ' images')
2019-12-21 04:25:46.559483 ('we have processed ', 1999, ' images')
2019-12-21 04:25:56.502243 ('we have processed ', 2999, ' images')
2019-12-21 04:26:06.064670 ('we have processed ', 3999, ' images')
2019-12-21 04:26:16.058429 ('we have processed ', 4999, ' images')
2019-12-21 04:26:26.385210 ('we have processed ', 5999, ' images')
2019-12-21 04:26:37.046660 ('we have processed ', 6999, ' images')
2019-12-21 04:26:46.630950 ('we have processed ', 7999, ' images')
2019-12-21 04:26:57.095567 ('we have processed ', 8999, ' images')
2019-12-21 04:27:06.894281 ('we have processed ', 9999, ' image

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-21 04:28:19.541368 ('we have processed ', 16999, ' images')
2019-12-21 04:28:29.332543 ('we have processed ', 17999, ' images')
2019-12-21 04:28:39.292313 ('we have processed ', 18999, ' images')
2019-12-21 04:28:49.763018 ('we have processed ', 19999, ' images')
2019-12-21 04:28:59.831328 ('we have processed ', 20999, ' images')
2019-12-21 04:29:10.444167 ('we have processed ', 21999, ' images')
2019-12-21 04:29:21.143918 ('we have processed ', 22999, ' images')
2019-12-21 04:29:31.975639 ('we have processed ', 23999, ' images')
2019-12-21 04:29:42.424205 ('we have processed ', 24999, ' images')
2019-12-21 04:29:52.907427 ('we have processed ', 25999, ' images')
2019-12-21 04:30:03.173454 ('we have processed ', 26999, ' images')
2019-12-21 04:30:13.743324 ('we have processed ', 27999, ' images')
2019-12-21 04:30:24.689999 ('we have processed ', 28999, ' images')
2019-12-21 04:30:35.338865 ('we have processed ', 29999, ' images')
2019-12-21 04:30:45.812390 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-21 05:01:45.616377 ('we have processed ', 16999, ' images')
2019-12-21 05:01:55.674760 ('we have processed ', 17999, ' images')
2019-12-21 05:02:05.462341 ('we have processed ', 18999, ' images')
2019-12-21 05:02:15.723603 ('we have processed ', 19999, ' images')
2019-12-21 05:02:25.812777 ('we have processed ', 20999, ' images')
2019-12-21 05:02:36.240554 ('we have processed ', 21999, ' images')
2019-12-21 05:02:46.767959 ('we have processed ', 22999, ' images')
2019-12-21 05:02:57.152699 ('we have processed ', 23999, ' images')
2019-12-21 05:03:07.487982 ('we have processed ', 24999, ' images')
2019-12-21 05:03:17.837203 ('we have processed ', 25999, ' images')
2019-12-21 05:03:28.153759 ('we have processed ', 26999, ' images')
2019-12-21 05:03:38.863758 ('we have processed ', 27999, ' images')
2019-12-21 05:03:49.719815 ('we have processed ', 28999, ' images')
2019-12-21 05:04:00.439733 ('we have processed ', 29999, ' images')
2019-12-21 05:04:11.081111 ('we have processed '

C:\Users\scott\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in greater_equal


2019-12-21 05:35:30.543380 ('we have processed ', 16999, ' images')
2019-12-21 05:35:40.337685 ('we have processed ', 17999, ' images')
2019-12-21 05:35:50.382146 ('we have processed ', 18999, ' images')
2019-12-21 05:36:00.677884 ('we have processed ', 19999, ' images')
2019-12-21 05:36:10.942085 ('we have processed ', 20999, ' images')
2019-12-21 05:36:21.095677 ('we have processed ', 21999, ' images')
2019-12-21 05:36:31.629370 ('we have processed ', 22999, ' images')
2019-12-21 05:36:42.152161 ('we have processed ', 23999, ' images')
2019-12-21 05:36:52.514882 ('we have processed ', 24999, ' images')
2019-12-21 05:37:03.071856 ('we have processed ', 25999, ' images')
2019-12-21 05:37:13.392875 ('we have processed ', 26999, ' images')
2019-12-21 05:37:23.978413 ('we have processed ', 27999, ' images')
2019-12-21 05:37:34.897592 ('we have processed ', 28999, ' images')
2019-12-21 05:37:45.363581 ('we have processed ', 29999, ' images')
2019-12-21 05:37:55.831803 ('we have processed '

#### ......................................NUM_POWER_SUMS=4....NUM_POWER_SUMS=1

600/100 verbatim_from_book_CNN 85.41% (+/- 4.90%); 86.18% (+/- 3.88%)

6_000/1_000 verbatim_from_book_CNN 96.74% (+/- 1.50%); 97.07% (+/- 0.74%)

#### SIMILARITY_VECTOR_TYPE=1
600/100 test_CNN_with_similarity_vector 85.88% (+/- 4.85%); 86.26% (+/- 4.10%)

#### SIMILARITY_VECTOR_TYPE=1
6_000/1_000 test_CNN_with_similarity_vector 97.10% (+/- 0.87%); 96.84% (+/- 0.69%)

#### SIMILARITY_VECTOR_TYPE=0
6_000/1_000 test_CNN_with_similarity_vector 97.23% (+/- 0.58%); 97.07% (+/- 0.66%)

#### SIMILARITY_VECTOR_TYPE=1
6_000/1_000 test_CNN_with_similarity_vector 96.68% (+/- 1.44%); 96.46% (+/- 1.12%)

#### SIMILARITY_VECTOR_TYPE=2
6_000/1_000 test_CNN_with_similarity_vector 97.57% (+/- 0.63%); 96.93% (+/- 0.85%)

#### SIMILARITY_VECTOR_TYPE=3
6_000/1_000 test_CNN_with_similarity_vector 97.01% (+/- 0.79%); 97.14% (+/- 1.01%)

#### SIMILARITY_VECTOR_TYPE=4
6_000/1_000 test_CNN_with_similarity_vector 96.73% (+/- 0.96%); 97.04% (+/- 1.31%)

#### SIMILARITY_VECTOR_TYPE=5
6_000/1_000 test_CNN_with_similarity_vector 97.13% (+/- 1.05%); 96.70% (+/- 1.17%)

#### SIMILARITY_VECTOR_TYPE=1
6_000/1_000 test_CNN_with_similarity_vector 96.06% (+/- 1.53%); 96.70% (+/- 0.58%)

12_000/2_000 verbatim_from_book_CNN 97.94% (+/- 0.50%); 98.00% (+/- 0.37%)

24_000/4_000 verbatim_from_book_CNN 98.51% (+/- 0.23%); 98.49% (+/- 0.28%)

36_000/6_000 verbatim_from_book_CNN 98.76% (+/- 0.19%); 98.79% (+/- 0.24%)

6_000/1_000 verbatim_from_book_CNN 96.40% (+/- 1.24%); 97.08% (+/- 0.76%)

#### SIMILARITY_VECTOR_TYPE=1
12_000/2_000 test_CNN_with_similarity_vector 98.14% (+/- 0.51%); 98.11% (+/- 0.49%)

#### SIMILARITY_VECTOR_TYPE=1
24_000/4_000 test_CNN_with_similarity_vector 98.49% (+/- 0.37%); 98.59% (+/- 0.25%)

#### SIMILARITY_VECTOR_TYPE=1
36_000/6_000 test_CNN_with_similarity_vector 98.67% (+/- 0.49%); 98.72% (+/- 0.28%)

#### SIMILARITY_VECTOR_TYPE=1
48_000/8_000 test_CNN_with_similarity_vector 99.07% (+/- 0.10%); 98.88% (+/- 0.22%)

#### SIMILARITY_VECTOR_TYPE=1
60_000/10_000 test_CNN_with_similarity_vector 99.04% (+/- 0.11%); 99.03% (+/- 0.14%)

48_000/8_000 verbatim_from_book_CNN...................; 98.84% (+/- 0.29%)

60_000/10_000 verbatim_from_book_CNN...................;99.06% (+/- 0.18%)

#### SIMILARITY_VECTOR_TYPE=1 NUM_POWER_SUMS=1 repeated 3 times
60_000/10_000 test_CNN_with_similarity_vector: 99.08% (+/- 0.17%); 98.95% (+/- 0.22%); 99.00% (+/- 0.10%); 99.05% (+/- 0.10%)

#### SIMILARITY_VECTOR_TYPE=-1 NUM_POWER_SUMS=1 repeated 3 times and then again
60_000/10_000 test_CNN_with_similarity_vector: 99.09% (+/- 0.12%); 99.12% (+/- 0.11%); 99.04% (+/- 0.19%); 99.04% (+/- 0.11%); 99.09% (+/- 0.10%); 99.06% (+/- 0.13%)

60_000/10_000 verbatim_from_book_CNN repeated 3 times: 99.12% (+/- 0.06%); 99.10% (+/- 0.11%); 99.04% (+/- 0.10%)


# Areas for further work

Phase 1 of 3-part investigation:
- do performance analysis and speed up get_similarity_vector (avoid append; Cpython; vectorize; use profiler)
- make a module that you can import and pip install
- make docstrings
- tune scikit-Learn hyperparameters and Config. constants
- document alternative attempts e.g. USE_AREA_INVAR; the weaker model of kMeans and their centroids 
- AUC-ROC curve for points <tp rate,tn rate> by taking one Config. constant and varying it
- bar charts or box plots for (1 - accuracy) error rate reduction
- are there uses for scatterplots? heatmaps? line graphs?

Phase 2 of 3-part investigation:
- rotate each image by Config.SIN_ALPHA and Config.COS_ALPHA radians
- add smoke or noise to test image
- Invariant for perspectivity (if not projectivity): try adding order of sectors ordered by num_sphere_sector_np to see if it is a useful measure
- try other datasets (COIL, ImageNet)
- use similarity vector with XGBoost
- comparing kNN with similarity vector to CNN with similarity vector

Phase 3 of 3-part investigation:
